In [ ]:
#|default_exp core

# Claudette's source

This is the 'literate' source code for Claudette. You can view the fully rendered version of the notebook [here](https://claudette.answer.ai/core.html), or you can clone the git repo and run the [interactive notebook](https://github.com/AnswerDotAI/claudette/blob/main/00_core.ipynb) in Jupyter. The notebook is converted the [Python module claudette/core.py](https://github.com/AnswerDotAI/claudette/blob/main/claudette/core.py) using [nbdev](https://nbdev.fast.ai/). The goal of this source code is to both create the Python module, and also to teach the reader *how* it is created, without assuming much existing knowledge about Claude's API.

Most of the time you'll see that we write some source code *first*, and then a description or discussion of it *afterwards*.

## Setup

In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

To print every HTTP request and response in full, uncomment the above line. This functionality is provided by Anthropic's SDK.

In [ ]:
#| export
import inspect, typing, json
from collections import abc
from dataclasses import dataclass
from typing import get_type_hints, Any
from functools import wraps

from anthropic import Anthropic, AnthropicBedrock, AnthropicVertex
from anthropic.types import (Usage, TextBlock, ServerToolUseBlock,
                             WebSearchToolResultBlock, Message, ToolUseBlock,
                             ThinkingBlock, ServerToolUsage)
from anthropic.resources import messages

import toolslm
from toolslm.funccall import *

from fastcore.meta import delegates
from fastcore.utils import *
from fastcore.xtras import save_iter
from msglm import mk_msg_anthropic as mk_msg, mk_msgs_anthropic as mk_msgs

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
from anthropic.types import Model
from claudette.text_editor import *
from typing import get_args
from datetime import datetime
from pprint import pprint
from IPython.display import Image

import warnings

In [ ]:
warnings.filterwarnings("ignore", message="Pydantic serializer warnings")

In [ ]:
#| export
_all_ = ['mk_msg', 'mk_msgs']

In [ ]:
#| export
empty = inspect.Parameter.empty

:::{.callout-tip}

If you're reading the rendered version of this notebook, you'll see an "Exported source" collapsible widget below. If you're reading the source notebook directly, you'll see `#| exports` at the top of the cell. These show that this piece of code will be exported into the python module that this notebook creates. No other code will be included -- any other code in this notebook is just for demonstration, documentation, and testing.

You can toggle expanding/collapsing the source code of all exported sections by using the `</> Code` menu in the top right of the rendered notebook page.

:::

In [ ]:
#| exports

model_types = {
    # Anthropic
    'claude-opus-4-20250514': 'opus',
    'claude-sonnet-4-20250514': 'sonnet',
    'claude-3-opus-20240229': 'opus-3',
    'claude-3-7-sonnet-20250219': 'sonnet-3-7',
    'claude-3-5-sonnet-20241022': 'sonnet-3-5',
    'claude-3-haiku-20240307': 'haiku-3',
    'claude-3-5-haiku-20241022': 'haiku-3-5',
    # AWS
    'anthropic.claude-3-opus-20240229-v1:0': 'opus',
    'anthropic.claude-3-5-sonnet-20241022-v2:0': 'sonnet',
    'anthropic.claude-3-sonnet-20240229-v1:0': 'sonnet',
    'anthropic.claude-3-haiku-20240307-v1:0': 'haiku',
    # Google
    'claude-3-opus@20240229': 'opus',
    'claude-3-5-sonnet-v2@20241022': 'sonnet',
    'claude-3-sonnet@20240229': 'sonnet',
    'claude-3-haiku@20240307': 'haiku',
}

all_models = list(model_types)

In [ ]:
#|hide

# Full list of models via the anthropic SDK for reference. Not all models are surfaced by Claudette.
Model

typing.Union[typing.Literal['claude-3-7-sonnet-latest', 'claude-3-7-sonnet-20250219', 'claude-3-5-haiku-latest', 'claude-3-5-haiku-20241022', 'claude-sonnet-4-20250514', 'claude-sonnet-4-0', 'claude-4-sonnet-20250514', 'claude-3-5-sonnet-latest', 'claude-3-5-sonnet-20241022', 'claude-3-5-sonnet-20240620', 'claude-opus-4-0', 'claude-opus-4-20250514', 'claude-4-opus-20250514', 'claude-3-opus-latest', 'claude-3-opus-20240229', 'claude-3-sonnet-20240229', 'claude-3-haiku-20240307', 'claude-2.1', 'claude-2.0'], str]

In [ ]:
#| export
models = all_models[:5]

In [ ]:
models

['claude-opus-4-20250514',
 'claude-sonnet-4-20250514',
 'claude-3-opus-20240229',
 'claude-3-7-sonnet-20250219',
 'claude-3-5-sonnet-20241022']

In [ ]:
#| export
models_aws = [
    'anthropic.claude-sonnet-4-20250514-v1:0',
    'claude-3-5-haiku-20241022',
    'claude-3-7-sonnet-20250219',
    'anthropic.claude-3-opus-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20241022-v2:0'
]

In [ ]:
#| export
models_goog = ['anthropic.claude-3-sonnet-20240229-v1:0',
 'anthropic.claude-3-haiku-20240307-v1:0',
 'claude-3-opus@20240229',
 'claude-3-5-sonnet-v2@20241022',
 'claude-3-sonnet@20240229',
 'claude-3-haiku@20240307']

In [ ]:
#| exports
text_only_models = ('claude-3-5-haiku-20241022',)

In [ ]:
#| exports
has_streaming_models = set(all_models)
has_system_prompt_models = set(all_models)
has_temperature_models = set(all_models)
has_extended_thinking_models = {'claude-opus-4-20250514', 'claude-sonnet-4-20250514', 'claude-3-7-sonnet-20250219'}

In [ ]:
has_extended_thinking_models

{'claude-3-7-sonnet-20250219',
 'claude-opus-4-20250514',
 'claude-sonnet-4-20250514'}

In [ ]:
#| exports

def can_stream(m): return m in has_streaming_models
def can_set_system_prompt(m): return m in has_system_prompt_models
def can_set_temperature(m): return m in has_temperature_models
def can_use_extended_thinking(m): return m in has_extended_thinking_models

We include these functions to provide a uniform library interface with cosette since openai models such as o1 do not have many of these capabilities.

In [ ]:
assert can_stream('claude-3-5-sonnet-20241022') and can_set_system_prompt('claude-3-5-sonnet-20241022') and can_set_temperature('claude-3-5-sonnet-20241022')

These are the current versions and [prices](https://www.anthropic.com/pricing#anthropic-api) of Anthropic's models at the time of writing.

In [ ]:
model = models[1]; model

'claude-sonnet-4-20250514'

For examples, we'll use the latest Sonnet, since it's awesome.

## Antropic SDK

In [ ]:
cli = Anthropic()

This is what Anthropic's SDK provides for interacting with Python. To use it, pass it a list of *messages*, with *content* and a *role*. The roles should alternate between *user* and *assistant*.

:::{.callout-tip}

After the code below you'll see an indented section with an orange vertical line on the left. This is used to show the *result* of running the code above. Because the code is running in a Jupyter Notebook, we don't have to use `print` to display results, we can just type the expression directly, as we do with `r` here.

:::

In [ ]:
m = {'role': 'user', 'content': "I'm Jeremy"}
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Nice to meet you, Jeremy! How are you doing today? Is there anything I can help you with?

<details>

- id: `msg_01KpToJafReCnzTVLSCHVJpz`
- content: `[{'citations': None, 'text': 'Nice to meet you, Jeremy! How are you doing today? Is there anything I can help you with?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 10, 'output_tokens': 25, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

### Formatting output

That output is pretty long and hard to read, so let's clean it up. We'll start by pulling out the `Content` part of the message.
To do that, we're going to write our first function which will be included to the `claudette/core.py` module.

:::{.callout-tip}

This is the first exported public function or class we're creating (the previous export was of a variable). In the rendered version of the notebook for these you'll see 4 things, in this order (unless the symbol starts with a single `_`, which indicates it's *private*):

- The signature (with the symbol name as a heading, with a horizontal rule above)
- A table of paramater docs (if provided)
- The doc string (in italics).
- The source code (in a collapsible "Exported source" block)

After that, we generally provide a bit more detail on what we've created, and why, along with a sample usage.

:::

In [ ]:
#| exports
def _type(x):
    try: return x.type
    except AttributeError: return x.get('type')

def find_block(r:abc.Mapping, # The message to look in
               blk_type:type|str=TextBlock  # The type of block to find
              ):
    "Find the first block of type `blk_type` in `r.content`."
    f = (lambda x:_type(x)==blk_type) if isinstance(blk_type,str) else (lambda x:isinstance(x,blk_type))
    return first(o for o in r.content if f(o))

This makes it easier to grab the needed parts of Claude's responses, which can include multiple pieces of content. By default, we look for the first text block. That will generally have the content we want to display.

In [ ]:
find_block(r)

TextBlock(citations=None, text='Nice to meet you, Jeremy! How are you doing today? Is there anything I can help you with?', type='text')

In [ ]:
def contents(r):
    "Helper to get the contents from Claude response `r`."
    blk = find_block(r)
    if not blk and r.content: blk = r.content[0]
    return blk.text.strip() if hasattr(blk,'text') else str(blk)

For display purposes, we often just want to show the text itself.

In [ ]:
contents(r)

'Nice to meet you, Jeremy! How are you doing today? Is there anything I can help you with?'

In [ ]:
#| exports
@patch
def _repr_markdown_(self:(Message)):
    det = '\n- '.join(f'{k}: `{v}`' for k,v in self.model_dump().items())
    cts = re.sub(r'\$', '&#36;', contents(self))  # escape `$` for jupyter latex
    return f"""{cts}

<details>

- {det}

</details>"""

Jupyter looks for a `_repr_markdown_` method in displayed objects; we add this in order to display just the content text, and collapse full details into a hideable section. Note that `patch` is from [fastcore](https://fastcore.fast.ai/), and is used to add (or replace) functionality in an existing class. We pass the class(es) that we want to patch as type annotations to `self`. In this case, `_repr_markdown_` is being added to Anthropic's `Message` class, so when we display the message now we just see the contents, and the details are hidden away in a collapsible details block.

In [ ]:
r

Nice to meet you, Jeremy! How are you doing today? Is there anything I can help you with?

<details>

- id: `msg_01KpToJafReCnzTVLSCHVJpz`
- content: `[{'citations': None, 'text': 'Nice to meet you, Jeremy! How are you doing today? Is there anything I can help you with?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 10, 'output_tokens': 25, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

One key part of the response is the `usage` key, which tells us how many tokens we used by returning a `Usage` object.

We'll add some helpers to make things a bit cleaner for creating and formatting these objects.

In [ ]:
r.usage

In: 10; Out: 25; Cache create: 0; Cache read: 0; Total Tokens: 35; Search: 0

In [ ]:
#| exports
def server_tool_usage(web_search_requests=0):
    'Little helper to create a server tool usage object'
    return ServerToolUsage(web_search_requests=web_search_requests)

In [ ]:
#| exports
def usage(inp=0, # input tokens
          out=0,  # Output tokens
          cache_create=0, # Cache creation tokens
          cache_read=0, # Cache read tokens
          server_tool_use=server_tool_usage() # server tool use
         ):
    'Slightly more concise version of `Usage`.'
    return Usage(input_tokens=inp, output_tokens=out, cache_creation_input_tokens=cache_create,
                 cache_read_input_tokens=cache_read, server_tool_use=server_tool_use)

The constructor provided by Anthropic is rather verbose, so we clean it up a bit, using a lowercase version of the name.

In [ ]:
usage(5)

In: 5; Out: 0; Cache create: 0; Cache read: 0; Total Tokens: 5; Search: 0

In [ ]:
#| exports
def _dgetattr(o,s,d): 
    "Like getattr, but returns the default if the result is None"
    return getattr(o,s,d) or d

@patch(as_prop=True)
def total(self:Usage): return self.input_tokens+self.output_tokens+_dgetattr(self, "cache_creation_input_tokens",0)+_dgetattr(self, "cache_read_input_tokens",0)

Adding a `total` property to `Usage` makes it easier to see how many tokens we've used up altogether.

In [ ]:
usage(5,1).total

6

In [ ]:
#| exports
@patch
def __repr__(self:Usage):
    io_toks = f'In: {self.input_tokens}; Out: {self.output_tokens}'
    cache_toks = f'Cache create: {_dgetattr(self, "cache_creation_input_tokens",0)}; Cache read: {_dgetattr(self, "cache_read_input_tokens",0)}'
    server_tool_use = _dgetattr(self, "server_tool_use",server_tool_usage())
    server_tool_use_str = f'Search: {server_tool_use.web_search_requests}'
    total_tok = f'Total Tokens: {self.total}'
    return f'{io_toks}; {cache_toks}; {total_tok}; {server_tool_use_str}'

In python, patching `__repr__` lets us change how an object is displayed. (More generally, methods starting and ending in `__` in Python are called `dunder` methods, and have some `magic` behavior -- such as, in this case, changing how an object is displayed.) We won't be directly displaying ServerToolUsage's, so we can handle its display behavior in the same Usage `__repr__`

In [ ]:
usage(5)

In: 5; Out: 0; Cache create: 0; Cache read: 0; Total Tokens: 5; Search: 0

In [ ]:
#| exports
@patch
def __add__(self:ServerToolUsage, b):
    "Add together each of the server tool use counts"
    return ServerToolUsage(web_search_requests=self.web_search_requests+b.web_search_requests)

And, patching `__add__` lets `+` work on a `ServerToolUsage` as well as a `Usage` object.

In [ ]:
server_tool_usage(1) + server_tool_usage(2)

ServerToolUsage(web_search_requests=3)

In [ ]:
#| exports
@patch
def __add__(self:Usage, b):
    "Add together each of `input_tokens` and `output_tokens`"
    return usage(self.input_tokens+b.input_tokens, self.output_tokens+b.output_tokens,
                 _dgetattr(self,'cache_creation_input_tokens',0)+_dgetattr(b,'cache_creation_input_tokens',0),
                 _dgetattr(self,'cache_read_input_tokens',0)+_dgetattr(b,'cache_read_input_tokens',0),
                 _dgetattr(self,'server_tool_use',server_tool_usage())+_dgetattr(b,'server_tool_use',server_tool_usage()))

In [ ]:
r.usage+r.usage + usage(server_tool_use=server_tool_usage(1))

In: 20; Out: 50; Cache create: 0; Cache read: 0; Total Tokens: 70; Search: 1

### Creating messages

Creating correctly formatted `dict`s from scratch every time isn't very handy, so we'll import a couple of helper functions from the `msglm` library.

Let's use `mk_msg` to recreate our msg `{'role': 'user', 'content': "I'm Jeremy"}` from earlier.

In [ ]:
prompt = "I'm Jeremy"
m = mk_msg(prompt)
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Hi Jeremy! Nice to meet you. How are you doing today? Is there anything I can help you with?

<details>

- id: `msg_01A1C9UvfEqHrk86eCJgFE5E`
- content: `[{'citations': None, 'text': 'Hi Jeremy! Nice to meet you. How are you doing today? Is there anything I can help you with?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 10, 'output_tokens': 26, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

We can pass more than just text messages to Claude. As we'll see later we can also pass images, SDK objects, etc. To handle these different data types we need to pass the type along with our content to Claude. 

Here's an example of a multimodal message containing text and images. 

```json
{
    'role': 'user', 
    'content': [
        {'type':'text', 'text':'What is in the image?'},
        {
            'type':'image', 
            'source': {
                'type':'base64', 'media_type':'media_type', 'data': 'data'
            }
        }
    ]
}
```

`mk_msg` infers the type automatically and creates the appropriate data structure. 

LLMs, don't actually have state, but instead dialogs are created by passing back all previous prompts and responses every time. With Claude, they always alternate *user* and *assistant*. We'll use `mk_msgs` from `msglm` to make it easier to build up these dialog lists.

In [ ]:
msgs = mk_msgs([prompt, r, "I forgot my name. Can you remind me please?"]) 
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='Hi Jeremy! Nice to meet you. How are you doing today? Is there anything I can help you with?', type='text')]},
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

In [ ]:
cli.messages.create(messages=msgs, model=model, max_tokens=200)

Your name is Jeremy - you introduced yourself to me just a moment ago in your first message.

<details>

- id: `msg_016metiVT3rZtA6vKZXWfRUv`
- content: `[{'citations': None, 'text': 'Your name is Jeremy - you introduced yourself to me just a moment ago in your first message.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 50, 'output_tokens': 22, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Client

In [ ]:
#| exports
class Client:
    def __init__(self, model, cli=None, log=False, cache=False):
        "Basic Anthropic messages client."
        self.model,self.use = model,usage()
        self.text_only = model in text_only_models
        self.log = [] if log else None
        self.c = (cli or Anthropic(default_headers={'anthropic-beta': 'prompt-caching-2024-07-31'}))
        self.cache = cache

We'll create a simple `Client` for `Anthropic` which tracks usage stores the model to use. We don't add any methods right away -- instead we'll use `patch` for that so we can add and document them incrementally.

In [ ]:
c = Client(model)
c.use

In: 0; Out: 0; Cache create: 0; Cache read: 0; Total Tokens: 0; Search: 0

In [ ]:
#| exports
@patch
def _r(self:Client, r:Message, prefill=''):
    "Store the result of the message and accrue total usage."
    if prefill:
        blk = find_block(r)
        if blk: blk.text = prefill + (blk.text or '')
    self.result = r
    self.use += r.usage
    self.stop_reason = r.stop_reason
    self.stop_sequence = r.stop_sequence
    return r

We use a `_` prefix on private methods, but we document them here in the interests of literate source code.

`_r` will be used each time we get a new result, to track usage and also to keep the result available for later.

In [ ]:
c._r(r)
c.use

In: 10; Out: 26; Cache create: 0; Cache read: 0; Total Tokens: 36; Search: 0

Whereas OpenAI's models use a `stream` parameter for streaming, Anthropic's use a separate method. We implement Anthropic's approach in a private method, and then use a `stream` parameter in `__call__` for consistency:

In [ ]:
#| exports
@patch
def _log(self:Client, final, prefill, msgs, **kwargs):
    self._r(final, prefill)
    if self.log is not None: self.log.append({
        "msgs": msgs, **kwargs,
        "result": self.result, "use": self.use, "stop_reason": self.stop_reason, "stop_sequence": self.stop_sequence
    })
    return self.result

Once streaming is complete, we need to store the final message and call any completion callback that's needed.

In [ ]:
#| exports
@save_iter
def _stream(o, cm, prefill, cb):
    with cm as s:
        yield prefill
        yield from s.text_stream
        o.value = s.get_final_message()
        cb(o.value)

In [ ]:
#| export
def get_types(msgs):
    types = []
    for m in msgs:
        content = m.get('content', [])
        if isinstance(content, list): types.extend(getattr(c, 'type', None) or c['type'] for c in content)
        else: types.append('text')
    return types

In [ ]:
get_types(msgs)

['text', 'text', 'text']

In [ ]:
#| export
def mk_tool_choice(choose:Union[str,bool,None])->dict:
    "Create a `tool_choice` dict that's 'auto' if `choose` is `None`, 'any' if it is True, or 'tool' otherwise"
    return {"type": "tool", "name": choose} if isinstance(choose,str) else {'type':'any'} if choose else {'type':'auto'}

In [ ]:
print(mk_tool_choice('sums'))
print(mk_tool_choice(True))
print(mk_tool_choice(None))

{'type': 'tool', 'name': 'sums'}
{'type': 'any'}
{'type': 'auto'}


Claude can be forced to use a particular tool, or select from a specific list of tools, or decide for itself when to use a tool. If you want to force a tool (or force choosing from a list), include a `tool_choice` param with a dict from `mk_tool_choice`.

Claude supports adding an extra `assistant` message at the end, which contains the *prefill* -- i.e. the text we want Claude to assume the response starts with. However Claude doesn't actually repeat that in the response, so for convenience we add it.

In [ ]:
#| exports
@patch
def _precall(self:Client, msgs, prefill, sp, temp, maxtok, maxthinktok, stream,
             stop, tools, tool_choice, kwargs):
    if tools: kwargs['tools'] = [get_schema(o) if callable(o) else o for o in listify(tools)]
    if tool_choice: kwargs['tool_choice'] = mk_tool_choice(tool_choice)
    if maxthinktok: 
        kwargs['thinking'] = {'type':'enabled', 'budget_tokens':maxthinktok} 
        temp,prefill = 1,''
    pref = [prefill.strip()] if prefill else []
    if not isinstance(msgs,list): msgs = [msgs]
    if stop is not None:
        if not isinstance(stop, (list)): stop = [stop]
        kwargs["stop_sequences"] = stop
    msgs = mk_msgs(msgs+pref, cache=self.cache, cache_last_ckpt_only=self.cache)
    assert not ('image' in get_types(msgs) and self.text_only), f"Images not supported by: {self.model}"
    kwargs |= dict(max_tokens=maxtok, system=sp, temperature=temp)
    return msgs, kwargs

In [ ]:
#| exports
@patch
@delegates(messages.Messages.create)
def __call__(self:Client,
             msgs:list, # List of messages in the dialog
             sp='', # The system prompt
             temp=0, # Temperature
             maxtok=4096, # Maximum tokens
             maxthinktok=0, # Maximum thinking tokens
             prefill='', # Optional prefill to pass to Claude as start of its response
             stream:bool=False, # Stream response?
             stop=None, # Stop sequence
             tools:Optional[list]=None, # List of tools to make available to Claude
             tool_choice:Optional[dict]=None, # Optionally force use of some tool
             cb=None, # Callback to pass result to when complete
             **kwargs):
    "Make a call to Claude."
    msgs,kwargs = self._precall(msgs, prefill, sp, temp, maxtok, maxthinktok, stream,
                                stop, tools, tool_choice, kwargs)
    m = self.c.messages
    f = m.stream if stream else m.create
    res = f(model=self.model, messages=msgs, **kwargs)
    def _cb(v):
        self._log(v, prefill=prefill, msgs=msgs, **kwargs)
        if cb: cb(v)
    if stream: return _stream(res, prefill, _cb)
    try: return res
    finally: _cb(res)

Defining `__call__` let's us use an object like a function (i.e it's *callable*). We use it as a small wrapper over `messages.create`.

In [ ]:
c = Client(model, log=True)
c.use

In: 0; Out: 0; Cache create: 0; Cache read: 0; Total Tokens: 0; Search: 0

In [ ]:
c('Hi')

Hello! How are you doing today? Is there anything I can help you with?

<details>

- id: `msg_01A8tepLcMRXgtjsbTjrg2d7`
- content: `[{'citations': None, 'text': 'Hello! How are you doing today? Is there anything I can help you with?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 8, 'output_tokens': 20, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Usage details are automatically updated after each call:

In [ ]:
c.use

In: 8; Out: 20; Cache create: 0; Cache read: 0; Total Tokens: 28; Search: 0

A log of all messages is kept if `log=True` is passed:

In [ ]:
pprint(c.log)

[{'max_tokens': 4096,
  'msgs': [{'content': 'Hi', 'role': 'user'}],
  'result': Message(id='msg_01A8tepLcMRXgtjsbTjrg2d7', content=[TextBlock(citations=None, text='Hello! How are you doing today? Is there anything I can help you with?', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 8; Out: 20; Cache create: 0; Cache read: 0; Total Tokens: 28; Search: 0),
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'system': '',
  'temperature': 0,
  'use': In: 8; Out: 20; Cache create: 0; Cache read: 0; Total Tokens: 28; Search: 0}]


Let's try out *prefill*:

In [ ]:
q = "Very concisely, what is the meaning of life?"
pref = 'According to Douglas Adams, '

In [ ]:
c(q, prefill=pref)

According to Douglas Adams, 42. But seriously, there's no universal answer - it's deeply personal. Many find meaning through relationships, purpose, growth, helping others, or spiritual beliefs. The search itself might be part of the point.

<details>

- id: `msg_01CG2iga8Zz2tdzG3Wgu2ofV`
- content: `[{'citations': None, 'text': "According to Douglas Adams, 42. But seriously, there's no universal answer - it's deeply personal. Many find meaning through relationships, purpose, growth, helping others, or spiritual beliefs. The search itself might be part of the point.", 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 24, 'output_tokens': 46, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
c.use

In: 32; Out: 66; Cache create: 0; Cache read: 0; Total Tokens: 98; Search: 0

We can pass `stream=True` to stream the response back incrementally:

In [ ]:
r = c('Hi', stream=True)
for o in r: print(o, end='')

Hello! How are you doing today? Is there anything I can help you with?

In [ ]:
c.use

In: 40; Out: 86; Cache create: 0; Cache read: 0; Total Tokens: 126; Search: 0

The full final message after completion of streaming is in the `value` attr of the response:

In [ ]:
r.value

Hello! How are you doing today? Is there anything I can help you with?

<details>

- id: `msg_01DTwQqmBxg38bhpkjVb8iHM`
- content: `[{'citations': None, 'text': 'Hello! How are you doing today? Is there anything I can help you with?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 8, 'output_tokens': 20, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
for o in c(q, prefill=pref, stream=True): print(o, end='')

According to Douglas Adams, 42

. But seriously, there's no universal

 answer - it's deeply personal. Many find meaning through relationships, purpose

, growth, helping others, or spiritual

 beliefs. The search itself might be part of the point.

In [ ]:
c.use

In: 64; Out: 132; Cache create: 0; Cache read: 0; Total Tokens: 196; Search: 0

Pass a stop sequence if you want claude to stop generating text when it encounters it.

In [ ]:
c("Count from 1 to 10", stop="5")

1, 2, 3, 4,

<details>

- id: `msg_01U1hyKamhJ93NSgNZwGXeic`
- content: `[{'citations': None, 'text': '1, 2, 3, 4, ', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `stop_sequence`
- stop_sequence: `5`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 14, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

This also works with streaming, and you can pass more than one stop sequence:

In [ ]:
for o in c("Count from 1 to 10", stop=["3", "yellow"], stream=True): print(o, end='')
print()
print(c.stop_reason, c.stop_sequence)

1, 2, 
stop_sequence 3


We've shown the token usage but we really care about is pricing. Let's extract the latest [pricing](https://www.anthropic.com/pricing#anthropic-api) from Anthropic into a `pricing` dict.

In [ ]:
#| export
pricing = {  # model type: $ / million tokens (input, output, cache write, cache read)
    'opus': (15, 75, 18.75, 1.5),
    'sonnet': (3, 15, 3.75, 0.3),
    'haiku-3': (0.25, 1.25, 0.3, 0.03),
    'haiku-3-5': (1, 3, 1.25, 0.1),
}

In [ ]:
#| exports
def get_pricing(m, u):
    return pricing[m][:3] if u.prompt_token_count < 128_000 else pricing[m][3:]

Similarly, let's get the pricing for the latest [server tools]():

In [ ]:
#| export
server_tool_pricing = {
    'web_search_requests': 10, # $10 per 1,000
}

We'll patch `Usage` to enable it compute the cost given pricing.

In [ ]:
#| exports
@patch
def cost(self:Usage, costs:tuple) -> float:
    cache_w, cache_r = _dgetattr(self, "cache_creation_input_tokens",0), _dgetattr(self, "cache_read_input_tokens",0)
    tok_cost = sum([self.input_tokens * costs[0] +  self.output_tokens * costs[1] +  cache_w * costs[2] + cache_r * costs[3]]) / 1e6
    server_tool_use = _dgetattr(self, "server_tool_use",server_tool_usage())
    server_tool_cost = server_tool_use.web_search_requests * server_tool_pricing['web_search_requests'] / 1e3
    return tok_cost + server_tool_cost

In [ ]:
#| exports
@patch(as_prop=True)
def cost(self: Client) -> float: return self.use.cost(pricing[model_types[self.model]])

In [ ]:
#| exports
def get_costs(c):
    costs = pricing[model_types[c.model]]
    
    inp_cost = c.use.input_tokens * costs[0] / 1e6
    out_cost = c.use.output_tokens * costs[1] / 1e6

    cache_w = c.use.cache_creation_input_tokens   
    cache_r = c.use.cache_read_input_tokens
    cache_cost = (cache_w * costs[2] + cache_r * costs[3]) / 1e6

    server_tool_use = c.use.server_tool_use
    server_tool_cost = server_tool_use.web_search_requests * server_tool_pricing['web_search_requests'] / 1e3
    return inp_cost, out_cost, cache_cost, cache_w + cache_r, server_tool_cost

The markdown repr of the client itself will show the latest result, along with the usage so far.

In [ ]:
#| exports
@patch
def _repr_markdown_(self:Client):
    if not hasattr(self,'result'): return 'No results yet'
    msg = contents(self.result)
    inp_cost, out_cost, cache_cost, cached_toks, server_tool_cost = get_costs(self)
    return f"""{msg}

| Metric | Count | Cost (USD) |
|--------|------:|-----:|
| Input tokens | {self.use.input_tokens:,} | {inp_cost:.6f} |
| Output tokens | {self.use.output_tokens:,} | {out_cost:.6f} |
| Cache tokens | {cached_toks:,} | {cache_cost:.6f} |
| Server tool use | {self.use.server_tool_use.web_search_requests:,} | {server_tool_cost:.6f} |
| **Total** | **{self.use.total:,}** | **${self.cost:.6f}** |"""

In [ ]:
c

1, 2,

| Metric | Count | Cost (USD) |
|--------|------:|-----:|
| Input tokens | 94 | 0.000282 |
| Output tokens | 154 | 0.002310 |
| Cache tokens | 0 | 0.000000 |
| Server tool use | 0 | 0.000000 |
| **Total** | **248** | **$0.002592** |

Pass a list of alternating user/assistant messages to give Claude a "dialog".

In [ ]:
c(["My name is Jeremy", "Hi Jeremy!", "Can you remind me what my name is?"])

Your name is Jeremy.

<details>

- id: `msg_018tNGndAgrjqotBPftbgjiG`
- content: `[{'citations': None, 'text': 'Your name is Jeremy.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 29, 'output_tokens': 8, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Tool use

Let's now look more at tool use (aka *function calling*).

For testing, we need a function that Claude can call; we'll write a simple function that adds numbers together, and will tell us when it's being called:

In [ ]:
@dataclass
class MySum: val:int

def sums(
    a:int,  # First thing to sum
    b:int=1 # Second thing to sum
) -> int: # The sum of the inputs
    "Adds a + b."
    print(f"Finding the sum of {a} and {b}")
    return MySum(a + b)

In [ ]:
a,b = 604542,6458932
pr = f"What is {a}+{b}?"
sp = "Always use tools when calculations are required."

Claudette can autogenerate a schema thanks to the `toolslm` library. We'll force the use of the tool using the function we created earlier.

In [ ]:
tools=[get_schema(sums)]
choice = mk_tool_choice('sums')

We'll start a dialog with Claude now. We'll store the messages of our dialog in `msgs`. The first message will be our prompt `pr`, and we'll pass our `tools` schema.

In [ ]:
msgs = mk_msgs(pr)
r = c(msgs, sp=sp, tools=tools, tool_choice=choice)
r

ToolUseBlock(id='toolu_01KpUQoReZfMSqMR6eeeRsie', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')

<details>

- id: `msg_01Qd2neoMVF8iEcNMy8wqopn`
- content: `[{'id': 'toolu_01KpUQoReZfMSqMR6eeeRsie', 'input': {'a': 604542, 'b': 6458932}, 'name': 'sums', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 440, 'output_tokens': 57, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

When Claude decides that it should use a tool, it passes back a `ToolUseBlock` with the name of the tool to call, and the params to use.

We don't want to allow it to call just any possible function (that would be a security disaster!) so we create a *namespace* -- that is, a dictionary of allowable function names to call.

In [ ]:
ns = mk_ns(sums)
ns

{'sums': <function __main__.sums(a: int, b: int = 1) -> int>}

`ToolResult` is used for two special cases:

1) When tool calls are RPCs with claudette running on an application server and code execution happening elsewhere, wrapping with a `result_type` field is used as a type descriptor for the claudette client. 

2) Different types are handled in message history with specific format, so `mk_funcres` branches the Anthropic representation (see depending on the `result_type`.

Currently images are the only supported tool result type - see https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/implement-tool-use#example-of-tool-result-with-images for the format implemented in `mk_funcres`.

In [ ]:
#| export

class ToolResult(BasicRepr):
    def __init__(self, result_type: str, data): store_attr()
    def __str__(self): return str(self.data)

In [ ]:
#| export

def _img_content(b64data):
    return [{"type": "image",
             "source":{"type": "base64", "media_type": "image/png", "data": b64data}},
            {"type": "text", "text": "Captured screenshot."}]

def mk_funcres(fc, ns):
    "Given tool use block 'fc', get tool result, and create a tool_result response."
    res = call_func(fc.name, fc.input, ns=ns, raise_on_err=False)
    if isinstance(res, ToolResult) and res.result_type=="image/png": res = _img_content(res.data) # list
    else: res = str(res.data) if isinstance(res, ToolResult) else str(res)
    return {"type": "tool_result", "tool_use_id": fc.id, "content": res}

We can now use the function requested by Claude. We look it up in `ns`, and pass in the provided parameters.

In [ ]:
fcs = [o for o in r.content if isinstance(o,ToolUseBlock)]
fcs

[ToolUseBlock(id='toolu_01KpUQoReZfMSqMR6eeeRsie', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')]

In [ ]:
res = [mk_funcres(fc, ns=ns) for fc in fcs]
res

Finding the sum of 604542 and 6458932


[{'type': 'tool_result',
  'tool_use_id': 'toolu_01KpUQoReZfMSqMR6eeeRsie',
  'content': 'MySum(val=7063474)'}]

In [ ]:
def contents(r):
    "Helper to get the contents from Claude response `r`."
    blk = find_block(r)
    if not blk and r.content: blk = r.content[0]
    if hasattr(blk,'text'): return blk.text.strip()
    elif hasattr(blk,'content'): return blk.content.strip()
    return str(blk)

In [ ]:
#| exports

def mk_toolres(
    r:abc.Mapping, # Tool use request response from Claude
    ns:Optional[abc.Mapping]=None # Namespace to search for tools
    ):
    "Create a `tool_result` message from response `r`."
    cts = getattr(r, 'content', [])
    res = [mk_msg(r.model_dump(), role='assistant')]
    if ns is None: ns=globals()
    tcs = [mk_funcres(o, ns) for o in cts if isinstance(o,ToolUseBlock)]
    if tcs: res.append(mk_msg(tcs))
    return res

In [ ]:
foo = []
foo.append({})
foo.append({})
foo

[{}, {}]

In order to tell Claude the result of the tool call, we pass back the tool use assistant request and the `tool_result` response.

In [ ]:
tr = mk_toolres(r, ns=ns)
tr

Finding the sum of 604542 and 6458932


[{'role': 'assistant',
  'content': [{'id': 'toolu_01KpUQoReZfMSqMR6eeeRsie',
    'input': {'a': 604542, 'b': 6458932},
    'name': 'sums',
    'type': 'tool_use'}]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01KpUQoReZfMSqMR6eeeRsie',
    'content': 'MySum(val=7063474)'}]}]

In [ ]:
msgs

[{'role': 'user', 'content': 'What is 604542+6458932?'}]

We add this to our dialog, and now Claude has all the information it needs to answer our question.

In [ ]:
msgs += tr
contents(c(msgs, sp=sp, tools=tools))

'The sum of 604542 + 6458932 is 7,063,474.'

In [ ]:
contents(msgs[-1])

'MySum(val=7063474)'

In [ ]:
msgs

[{'role': 'user', 'content': 'What is 604542+6458932?'},
 {'role': 'assistant',
  'content': [{'id': 'toolu_01KpUQoReZfMSqMR6eeeRsie',
    'input': {'a': 604542, 'b': 6458932},
    'name': 'sums',
    'type': 'tool_use'}]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01KpUQoReZfMSqMR6eeeRsie',
    'content': 'MySum(val=7063474)'}]}]

### Text editing

Anthropic also has a special tool type specific to text editing.

In [ ]:
tools = [text_editor_conf['sonnet']]
tools

[{'type': 'text_editor_20250429', 'name': 'str_replace_based_edit_tool'}]

In [ ]:
pr = 'Could you please explain my _quarto.yml file?'
msgs = [mk_msg(pr)]
r = c(msgs, sp=sp, tools=tools)
find_block(r, ToolUseBlock)

ToolUseBlock(id='toolu_01UsV3G1KddykspfZ7wfWFq4', input={'command': 'view', 'path': '_quarto.yml'}, name='str_replace_based_edit_tool', type='tool_use')

We've gone ahead and create a reference implementation that you can directly use from our `text_editor` module. Or use as reference for creating your own. 

In [ ]:
ns = mk_ns(str_replace_based_edit_tool)
tr = mk_toolres(r, ns=ns)
msgs += tr
print(contents(c(msgs, sp=sp, tools=tools))[:128])

It looks like the `_quarto.yml` file isn't in the current directory. Let me check what files are available in the current direct


## Structured data

In [ ]:
a,b = 604542,6458932
pr = f"What is {a}+{b}?"
sp = "Always use your tools for calculations."

In [ ]:
for tools in [sums, [get_schema(sums)]]:
    r = c(pr, tools=tools, tool_choice='sums')
    print(r)

Message(id='msg_01Bpa4mLb1YfeUQDahxeFqkR', content=[ToolUseBlock(id='toolu_01B4sKNW2fJrJFK1SajS8Cuo', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 435; Out: 53; Cache create: 0; Cache read: 0; Total Tokens: 488; Search: 0)


Message(id='msg_01SqdVRiqnAnkwxJTozR2X8Z', content=[ToolUseBlock(id='toolu_01VHsBEvAKYuVLdx3Z1R7o3N', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 435; Out: 53; Cache create: 0; Cache read: 0; Total Tokens: 488; Search: 0)


In [ ]:
ns = mk_ns(sums)
tr = mk_toolres(r, ns=ns)

Finding the sum of 604542 and 6458932


In [ ]:
#| exports
@patch
@delegates(Client.__call__)
def structured(self:Client,
               msgs:list, # List of messages in the dialog
               tools:Optional[list]=None, # List of tools to make available to Claude
               ns:Optional[abc.Mapping]=None, # Namespace to search for tools
               **kwargs):
    "Return the value of all tool calls (generally used for structured outputs)"
    tools = listify(tools)
    res = self(msgs, tools=tools, tool_choice=tools, **kwargs)
    if ns is None: ns=mk_ns(*tools)
    cts = getattr(res, 'content', [])
    tcs = [call_func(o.name, o.input, ns=ns) for o in cts if isinstance(o,ToolUseBlock)]
    return tcs

Anthropic's API does not support response formats directly, so instead we provide a `structured` method to use tool calling to achieve the same result. The result of the tool is not passed back to Claude in this case, but instead is returned directly to the user. 

In [ ]:
c.structured(pr, tools=[sums])

Finding the sum of 604542 and 6458932


[MySum(val=7063474)]

In [ ]:
c

ToolUseBlock(id='toolu_018uwqb7sK31r4G2A6aQ1QCR', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')

| Metric | Count | Cost (USD) |
|--------|------:|-----:|
| Input tokens | 4,754 | 0.014262 |
| Output tokens | 630 | 0.009450 |
| Cache tokens | 0 | 0.000000 |
| Server tool use | 0 | 0.000000 |
| **Total** | **5,384** | **$0.023712** |

## Custom Types with Tools Use

We need to add tool support for custom types too. Let's test out custom types using a minimal example.

In [ ]:
class Book(BasicRepr):
    def __init__(self, title: str, pages: int): store_attr()
    def __repr__(self):
        return f"Book Title : {self.title}\nNumber of Pages : {self.pages}"

In [ ]:
Book("War and Peace", 950)

Book Title : War and Peace
Number of Pages : 950

In [ ]:
def find_page(book: Book, # The book to find the halfway point of
              percent: int, # Percent of a book to read to, e.g. halfway == 50, 
) -> int:
    "The page number corresponding to `percent` completion of a book"
    return round(book.pages * (percent / 100.0))

In [ ]:
get_schema(find_page)

{'name': 'find_page',
 'description': 'The page number corresponding to `percent` completion of a book\n\nReturns:\n- type: integer',
 'input_schema': {'type': 'object',
  'properties': {'book': {'type': 'object',
    'description': 'The book to find the halfway point of',
    '$ref': '#/$defs/Book'},
   'percent': {'type': 'integer',
    'description': 'Percent of a book to read to, e.g. halfway == 50,'}},
  'required': ['book', 'percent'],
  '$defs': {'Book': {'type': 'object',
    'properties': {'title': {'type': 'string', 'description': ''},
     'pages': {'type': 'integer', 'description': ''}},
    'title': 'Book',
    'required': ['title', 'pages']}}}}

In [ ]:
choice = mk_tool_choice('find_page')
choice

{'type': 'tool', 'name': 'find_page'}

Claudette will pack objects as dict, so we'll transform tool functions with user-defined types into tool functions that accept a dict in lieu of the user-defined type.

First let's convert a single argument:

In [ ]:
#| export
def _is_builtin(tp: type):
    "Returns True for built in primitive types or containers"
    return (tp in (str, int, float, bool, complex) or tp is None
        or getattr(tp, '__origin__', None) is not None)  # Pass through all container types

def _convert(val: Dict, # dictionary argument being passed in
            tp: type): # type of the tool function input
    "Convert converts a single argument"
    if val is None or _is_builtin(tp) or not isinstance(val, dict): return val
    return tp(**val)

`_is_builtin` decides whether to pass an argument through as-is. Let's check the argument conversion:

In [ ]:
(_is_builtin(int), _is_builtin(Book), _is_builtin(List))

(True, False, True)

In [ ]:
(_convert(555, int),
 _convert({"title": "War and Peace", "pages": 923}, Book),
 _convert([1, 2, 3, 4], List))

(555,
 Book Title : War and Peace
 Number of Pages : 923,
 [1, 2, 3, 4])

To apply `tool()` to a function is to return a new function where the user-defined types are replaced with dictionary inputs.

In [ ]:
#| export
def tool(func):
    if isinstance(func, dict): return func # it's a schema, so don't change
    hints = get_type_hints(func)
    @wraps(func)
    def wrapper(*args, **kwargs):
        new_args = [_convert(arg, hints[p]) for p,arg in zip(inspect.signature(func).parameters, args)]
        new_kwargs = {k: _convert(v, hints[k]) if k in hints else v for k,v in kwargs.items()}
        return func(*new_args, **new_kwargs)
    return wrapper

A function is transformed into a function with dict arguments substituted for user-defined types. Built-in types such as `percent` here are left untouched.

In [ ]:
find_page(book=Book("War and Peace", 950), percent=50)

475

In [ ]:
tool(find_page)({"title": "War and Peace", "pages": 950}, percent=50)

475

By passing tools wrapped by `tool()`, user-defined types now work completes without failing in tool calls.

In [ ]:
pr = "How many pages do I have to read to get halfway through my 950 page copy of War and Peace"
tools = tool(find_page)
tools

<function __main__.find_page(book: __main__.Book, percent: int) -> int>

In [ ]:
r = c(pr, tools=[tools])
find_block(r, ToolUseBlock)

ToolUseBlock(id='toolu_018mxeP2xsX6L9eZTgst5cxc', input={'book': {'title': 'War and Peace', 'pages': 950}, 'percent': 50}, name='find_page', type='tool_use')

In [ ]:
tr = mk_toolres(r, ns=[tools])
tr

[{'role': 'assistant',
  'content': [{'citations': None,
    'text': "I'll help you find the halfway point of your copy of War and Peace.",
    'type': 'text'},
   {'id': 'toolu_018mxeP2xsX6L9eZTgst5cxc',
    'input': {'book': {'title': 'War and Peace', 'pages': 950}, 'percent': 50},
    'name': 'find_page',
    'type': 'tool_use'}]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_018mxeP2xsX6L9eZTgst5cxc',
    'content': '475'}]}]

In [ ]:
msgs = [pr]+tr
contents(c(msgs, sp=sp, tools=[tools]))

"To get halfway through your 950-page copy of War and Peace, you need to read to page 475. That means you'll have read 475 pages when you reach the halfway point of the book."

## Chat

Rather than manually adding the responses to a dialog, we'll create a simple `Chat` class to do that for us, each time we make a request. We'll also store the system prompt and tools here, to avoid passing them every time.

In [ ]:
#| export
class Chat:
    def __init__(self,
                 model:Optional[str]=None, # Model to use (leave empty if passing `cli`)
                 cli:Optional[Client]=None, # Client to use (leave empty if passing `model`)
                 sp='', # Optional system prompt
                 tools:Optional[list]=None, # List of tools to make available to Claude
                 temp=0, # Temperature
                 cont_pr:Optional[str]=None, # User prompt to continue an assistant response
                 cache: bool = False,  # Use Claude cache?
                 hist: list = None,  # Initialize history
                 ns:Optional[abc.Mapping]=None # Namespace to search for tools
                ):
        "Anthropic chat client."
        assert model or cli
        assert cont_pr != "", "cont_pr may not be an empty string"
        self.c = (cli or Client(model, cache=cache))
        if hist is None: hist=[]
        if tools: tools = [tool(t) for t in listify(tools)]
        if ns is None: ns=tools
        self.h,self.sp,self.tools,self.cont_pr,self.temp,self.cache,self.ns = hist,sp,tools,cont_pr,temp,cache,ns

    @property
    def use(self): return self.c.use

The class stores the `Client` that will provide the responses in `c`, and a history of messages in `h`.

In [ ]:
sp = "Never mention what tools you use."
chat = Chat(model, sp=sp)
chat.c.use, chat.h

(In: 0; Out: 0; Cache create: 0; Cache read: 0; Total Tokens: 0; Search: 0, [])

In [ ]:
chat.c.use.cost(pricing[model_types[chat.c.model]])

0.0

This is clunky. Let's add `cost` as a property for the `Chat` class. It will pass in the appropriate prices for the current model to the usage cost calculator.

In [ ]:
#| exports
@patch(as_prop=True)
def cost(self: Chat) -> float: return self.c.cost

In [ ]:
chat.cost

0.0

In [ ]:
#| exports
@patch
def _post_pr(self:Chat, pr, prev_role):
    if pr is None and prev_role == 'assistant':
        if self.cont_pr is None:
            raise ValueError("Prompt must be given after completion, or use `self.cont_pr`.")
        pr = self.cont_pr # No user prompt, keep the chain
    if pr: self.h.append(mk_msg(pr, cache=self.cache))

In [ ]:
#| exports
@patch
def _append_pr(self:Chat, pr=None):
    prev_role = nested_idx(self.h, -1, 'role') if self.h else 'assistant' # First message should be 'user'
    if pr and prev_role == 'user': self() # already user request pending
    self._post_pr(pr, prev_role)

In [ ]:
#| exports
@patch
def __call__(self:Chat,
             pr=None,  # Prompt / message
             temp=None, # Temperature
             maxtok=4096, # Maximum tokens
             maxthinktok=0, # Maximum thinking tokens
             stream=False, # Stream response?
             prefill='', # Optional prefill to pass to Claude as start of its response
             tool_choice:Optional[dict]=None, # Optionally force use of some tool
             **kw):
    if temp is None: temp=self.temp
    self._append_pr(pr)
    def _cb(v):
        self.last = mk_toolres(v, ns=self.ns)
        self.h += self.last
    return self.c(self.h, stream=stream, prefill=prefill, sp=self.sp, temp=temp, maxtok=maxtok, maxthinktok=maxthinktok,
                 tools=self.tools, tool_choice=tool_choice, cb=_cb, **kw)

The `__call__` method just passes the request along to the `Client`, but rather than just passing in this one prompt, it appends it to the history and passes it all along. As a result, we now have state!

In [ ]:
chat = Chat(model, sp=sp)

In [ ]:
chat("I'm Jeremy")
chat("What's my name?")

Your name is Jeremy.

<details>

- id: `msg_01MmB2RTGWux22AqEfGeiUDB`
- content: `[{'citations': None, 'text': 'Your name is Jeremy.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 42, 'output_tokens': 8, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
chat.use, chat.cost

(In: 59; Out: 25; Cache create: 0; Cache read: 0; Total Tokens: 84; Search: 0,
 0.000552)

Let's try out prefill too:

In [ ]:
q = "Very concisely, what is the meaning of life?"
pref = 'According to Douglas Adams,'

In [ ]:
chat(q, prefill=pref)

According to Douglas Adams,42. But seriously: to find purpose, connect with others, and create meaning through your choices and relationships.

<details>

- id: `msg_01J83eXfS8APbRGhXd8t7bd9`
- content: `[{'citations': None, 'text': 'According to Douglas Adams,42. But seriously: to find purpose, connect with others, and create meaning through your choices and relationships.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 70, 'output_tokens': 25, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

By default messages must be in user, assistant, user format. If this isn't followed (aka calling `chat()` without a user message) it will error out:

In [ ]:
try: chat()
except ValueError as e: print("Error:", e)

Error: Prompt must be given after completion, or use `self.cont_pr`.


Setting `cont_pr` allows a "default prompt" to be specified when a prompt isn't specified. Usually used to prompt the model to continue.

In [ ]:
chat.cont_pr = "Tell me a little more..."
chat()

The meaning of life is deeply personal and has been pondered for millennia. Some common perspectives:

**Philosophical**: Create your own meaning through authentic choices and taking responsibility for your existence (existentialism).

**Religious/Spiritual**: Serve a higher purpose, grow spiritually, or fulfill your role in a divine plan.

**Humanistic**: Reduce suffering, increase happiness, love others, and contribute to human flourishing.

**Biological**: Survive, reproduce, and pass on your genes - though humans transcend this basic drive.

**Practical**: Find what gives you a sense of purpose - whether that's relationships, creativity, helping others, learning, or building something lasting.

Most people find meaning through some combination: deep connections with others, work that feels worthwhile, personal growth, and leaving the world a bit better than they found it.

What resonates with you, Jeremy?

<details>

- id: `msg_01FD4D1mWfwWpyRL1nGhFK8b`
- content: `[{'citations': None, 'text': "The meaning of life is deeply personal and has been pondered for millennia. Some common perspectives:\n\n**Philosophical**: Create your own meaning through authentic choices and taking responsibility for your existence (existentialism).\n\n**Religious/Spiritual**: Serve a higher purpose, grow spiritually, or fulfill your role in a divine plan.\n\n**Humanistic**: Reduce suffering, increase happiness, love others, and contribute to human flourishing.\n\n**Biological**: Survive, reproduce, and pass on your genes - though humans transcend this basic drive.\n\n**Practical**: Find what gives you a sense of purpose - whether that's relationships, creativity, helping others, learning, or building something lasting.\n\nMost people find meaning through some combination: deep connections with others, work that feels worthwhile, personal growth, and leaving the world a bit better than they found it.\n\nWhat resonates with you, Jeremy?", 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 104, 'output_tokens': 194, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

We can also use streaming:

In [ ]:
chat = Chat(model, sp=sp)
for o in chat("I'm Jeremy", stream=True): print(o, end='')

Hi Jeremy! Nice to meet you. How

 are you doing today?

In [ ]:
r = chat(q, prefill=pref, stream=True)
for o in r: print(o, end='')
r.value

According to Douglas Adams,it

's 42. But seriously

: to find purpose, connect with others, and create

 meaning through your choices and relationships.

According to Douglas Adams,it's 42. But seriously: to find purpose, connect with others, and create meaning through your choices and relationships.

<details>

- id: `msg_01RbkyNHxj6FEXfCef8AaHXp`
- content: `[{'citations': None, 'text': "According to Douglas Adams,it's 42. But seriously: to find purpose, connect with others, and create meaning through your choices and relationships.", 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 54, 'output_tokens': 28, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

You can provide a history of messages to initialise `Chat` with:

In [ ]:
chat = Chat(model, sp=sp, hist=["Can you guess my name?", "Hmmm I really don't know. Is it 'Merlin G. Penfolds'?"])
chat('Wow how did you know?')

I have to be honest - I was just making a playful, completely random guess! I actually have no way of knowing your real name unless you tell me directly. I don't have access to any information about who you are beyond what you share in our conversation.

So if "Merlin G. Penfolds" actually is your name, that would be an absolutely incredible coincidence! But I suspect you might be playing along with my silly guess. Either way, it's nice to meet you! What would you like me to call you?

<details>

- id: `msg_01VGkeCFm6T6NzPtJfVqusKu`
- content: `[{'citations': None, 'text': 'I have to be honest - I was just making a playful, completely random guess! I actually have no way of knowing your real name unless you tell me directly. I don\'t have access to any information about who you are beyond what you share in our conversation.\n\nSo if "Merlin G. Penfolds" actually is your name, that would be an absolutely incredible coincidence! But I suspect you might be playing along with my silly guess. Either way, it\'s nice to meet you! What would you like me to call you?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 58, 'output_tokens': 117, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

### Chat tool use

We automagically get streamlined tool use as well:

In [ ]:
pr = f"What is {a}+{b}?"
pr

'What is 604542+6458932?'

In [ ]:
chat = Chat(model, sp=sp, tools=[sums])
r = chat(pr)
r

Finding the sum of 604542 and 6458932


ToolUseBlock(id='toolu_01C6trhovgB1vnR8B9H17Ksr', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')

<details>

- id: `msg_016PmAoJoSK7JZ2GErbXvo1w`
- content: `[{'id': 'toolu_01C6trhovgB1vnR8B9H17Ksr', 'input': {'a': 604542, 'b': 6458932}, 'name': 'sums', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 437, 'output_tokens': 72, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Now we need to send this result to Claude—calling the object with no parameters tells it to return the tool result to Claude:

In [ ]:
chat()

604542 + 6458932 = 7,063,474

<details>

- id: `msg_012xKEBqBkFpvBSG8LPREReY`
- content: `[{'citations': None, 'text': '604542 + 6458932 = 7,063,474', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 530, 'output_tokens': 19, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

It should be correct, because it actually used our Python function to do the addition. Let's check:

In [ ]:
a+b

7063474

Let's try the same thing with streaming:

In [ ]:
chat = Chat(model, sp=sp, tools=[sums])
r = chat(pr, stream=True)
for o in r: print(o, end='')

Finding the sum of 604542 and 6458932


The full message, including tool call details, are in `value`:

In [ ]:
r.value

ToolUseBlock(id='toolu_01G3xuThSjLqQmdrWqYNgw3x', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')

<details>

- id: `msg_01UT7X8rSctsaNt84pceWiqj`
- content: `[{'id': 'toolu_01G3xuThSjLqQmdrWqYNgw3x', 'input': {'a': 604542, 'b': 6458932}, 'name': 'sums', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 437, 'output_tokens': 72, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
r = chat(stream=True)
for o in r: print(o, end='')

604542 + 6458932 = 7,063,474

In [ ]:
r.value

604542 + 6458932 = 7,063,474

<details>

- id: `msg_014hAjEm37yMq9kPwyhcA9dt`
- content: `[{'citations': None, 'text': '604542 + 6458932 = 7,063,474', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 530, 'output_tokens': 19, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

The history shows both the tool_use and tool_result messages:

In [ ]:
chat.h

[{'role': 'user', 'content': 'What is 604542+6458932?'},
 {'role': 'assistant',
  'content': [{'id': 'toolu_01G3xuThSjLqQmdrWqYNgw3x',
    'input': {'a': 604542, 'b': 6458932},
    'name': 'sums',
    'type': 'tool_use'}]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01G3xuThSjLqQmdrWqYNgw3x',
    'content': 'MySum(val=7063474)'}]},
 {'role': 'assistant',
  'content': [{'citations': None,
    'text': '604542 + 6458932 = 7,063,474',
    'type': 'text'}]}]

Let's test a function with user defined types.

In [ ]:
chat = Chat(model, sp=sp, tools=[find_page])
r = chat("How many pages is three quarters of the way through my 80 page edition of Tao Te Ching?")
r

ToolUseBlock(id='toolu_011BWtSseHviFcsuxg8gg1tW', input={'book': {'title': 'Tao Te Ching', 'pages': 80}, 'percent': 75}, name='find_page', type='tool_use')

<details>

- id: `msg_01R1JJNcNC9rakXswLyN3ghU`
- content: `[{'id': 'toolu_011BWtSseHviFcsuxg8gg1tW', 'input': {'book': {'title': 'Tao Te Ching', 'pages': 80}, 'percent': 75}, 'name': 'find_page', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 547, 'output_tokens': 86, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
chat()

Three quarters of the way through your 80-page edition of Tao Te Ching would be page 60.

<details>

- id: `msg_01JsSNkyz92fMaTJnAXZtWdk`
- content: `[{'citations': None, 'text': 'Three quarters of the way through your 80-page edition of Tao Te Ching would be page 60.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 647, 'output_tokens': 29, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
#| exports
@patch
def _repr_markdown_(self:Chat):
    if not hasattr(self.c, 'result'): return 'No results yet'
    last_msg = contents(self.c.result)
    
    def fmt_msg(m):
        t = contents(m)
        if isinstance(t, dict): return t['content']
        return t
        
    history = '\n\n'.join(f"**{m['role']}**: {fmt_msg(m)}" 
                         for m in self.h)
    det = self.c._repr_markdown_().split('\n\n')[-1]
    if history: history = f"""
<details>
<summary>► History</summary>

{history}

</details>
"""

    return f"""{last_msg}
{history}
{det}"""

In [ ]:
# TODO: fix history format

In [ ]:
chat

Three quarters of the way through your 80-page edition of Tao Te Ching would be page 60.

<details>
<summary>► History</summary>

**user**: H

**assistant**: {'id': 'toolu_011BWtSseHviFcsuxg8gg1tW', 'input': {'book': {'title': 'Tao Te Ching', 'pages': 80}, 'percent': 75}, 'name': 'find_page', 'type': 'tool_use'}

**user**: 60

**assistant**: Three quarters of the way through your 80-page edition of Tao Te Ching would be page 60.

</details>

| Metric | Count | Cost (USD) |
|--------|------:|-----:|
| Input tokens | 1,194 | 0.003582 |
| Output tokens | 115 | 0.001725 |
| Cache tokens | 0 | 0.000000 |
| Server tool use | 0 | 0.000000 |
| **Total** | **1,309** | **$0.005307** |

In [ ]:
chat = Chat(model, tools=[text_editor_conf['sonnet']], ns=mk_ns(str_replace_based_edit_tool))

When not providing tools directly as Python functions (like `sum`), you **must** create and pass a namespace dictionary (mapping the tool name string to the function object) using the `ns` parameter to methods like `mk_toolres` or `toolloop`. `toolslm` cannot automatically generate the namespace in this case. For schema-based tools (i.e., Python functions), `claudette` handles namespace creation automatically.

In [ ]:
r = chat('Please explain very concisely what my _quarto.yml does. It is in the current path. Use your tools')
find_block(r, ToolUseBlock)

ToolUseBlock(id='toolu_01SoXDAs5oCoDHg1uiMxTsoW', input={'command': 'view', 'path': '_quarto.yml'}, name='str_replace_based_edit_tool', type='tool_use')

In [ ]:
chat()

Let me check the current directory first to see what files are available:

<details>

- id: `msg_01FiC3TVdeH7obe4yRQKnXRk`
- content: `[{'citations': None, 'text': 'Let me check the current directory first to see what files are available:', 'type': 'text'}, {'id': 'toolu_01Br6EMg31cpqkFkqPxsdrsv', 'input': {'command': 'view', 'path': '.'}, 'name': 'str_replace_based_edit_tool', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1242, 'output_tokens': 90, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Images

Claude can handle image data as well. As everyone knows, when testing image APIs you have to use a cute puppy.

In [ ]:
# Image is Cute_dog.jpg from Wikimedia
fn = Path('samples/puppy.jpg')
Image(filename=fn, width=200)

FileNotFoundError: [Errno 2] No such file or directory: 'samples/puppy.jpg'

In [ ]:
img = fn.read_bytes()

FileNotFoundError: [Errno 2] No such file or directory: 'samples/puppy.jpg'

Claude expects an image message to have the following structure

```js
{
    'role': 'user', 
    'content': [
        {'type':'text', 'text':'What is in the image?'},
        {
            'type':'image', 
            'source': {
                'type':'base64', 'media_type':'media_type', 'data': 'data'
            }
        }
    ]
}
```
`msglm` automatically detects if a message is an image, encodes it, and generates the data structure above.
All we need to do is a create a list containing our image and a query and then pass it to `mk_msg`.

Let's try it out...

In [ ]:
q = "In brief, what color flowers are in this image?"
msg = mk_msg([img, q])

NameError: name 'img' is not defined

In [ ]:
c([msg])

NameError: name 'msg' is not defined

You don't need to call `mk_msg` on each individual message before passing them to the `Chat` class. Instead you can pass your messages in a list and the `Chat` class will automatically call `mk_msgs` in the background. 

```python
c(["How are you?", r])
```

For messages that contain multiple content types (like an image with a question), you'll need to enclose the message contents in a list as shown below:

```python
c(["How are you?", r, [img, q]])
```

In [ ]:
c = Chat(model)
c([img, q])

NameError: name 'img' is not defined

In [ ]:
def contents(r):
    "Helper to get the contents from Claude response `r`."
    blk = find_block(r)
    if not blk and r.content: blk = r.content[0]
    if hasattr(blk,'text'): return blk.text.strip()
    elif hasattr(blk,'content'): return blk.content.strip()
    elif hasattr(blk,'source'): return f'*Media Type - {blk.type}*'
    return str(blk)

In [ ]:
contents(c.h[0])

IndexError: list index out of range

In [ ]:
c

No results yet

:::{.callout-note}

Unfortunately, not all Claude models support images 😞. This [table](https://docs.anthropic.com/en/docs/about-claude/models#model-comparison-table) summarizes the capabilities of each Claude model and the different modalities they support.

:::

## Caching

Claude supports context caching by adding a `cache_control` header to the message content.

```js
{
    "role": "user",
    "content": [
        {
            "type": "text", 
            "text": "Please cache my message", 
            "cache_control": {"type": "ephemeral"}
        }
    ]
}
```

To cache a message, we simply set `cache=True` when calling `mk_msg`.

In [ ]:
mk_msg(['hi', 'there'], cache=True)

```json
{ 'content': [ {'text': 'hi', 'type': 'text'},
               { 'cache_control': {'type': 'ephemeral'},
                 'text': 'there',
                 'type': 'text'}],
  'role': 'user'}
```

Claude also now supports smart cache look-ups, so it's very simple to keep an entire conversation in cache by constantly telling it to update the cache with the latest message. To do this, we just need to set `cache=True` when creating a `Chat`.

In [ ]:
chat = Chat(model, sp=sp, cache=True)

Caching has a minimum token limit of 1024 tokens for Sonnet and Opus, and 2048 for Haiku. If your conversation is below this limit, it will not be cached.

In [ ]:
chat("Hi, I'm Jeremy.")

Hi Jeremy! Nice to meet you. How are you doing today?

<details>

- id: `msg_01VwGqwA8LiJpjAdWrXedn4X`
- content: `[{'citations': None, 'text': 'Hi Jeremy! Nice to meet you. How are you doing today?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 20, 'output_tokens': 17, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
chat.use

In: 20; Out: 17; Cache create: 0; Cache read: 0; Total Tokens: 37; Search: 0

Note the usage: no cache is created, nor used. Now, let's send a long enough message to trigger caching.

In [ ]:
chat("""Lorem ipsum dolor sit amet""" * 150)

I see you've sent a very long block of "Lorem ipsum dolor sit amet" repeated many times! Lorem ipsum is that classic placeholder text that's commonly used in design and publishing.

Was this intentional, or did something go wrong with copy-pasting? Either way, no worries - happens to the best of us! Is there something specific you'd like to chat about, Jeremy?

<details>

- id: `msg_01EXHZPSmgvWCWts2TkCYJhd`
- content: `[{'citations': None, 'text': 'I see you\'ve sent a very long block of "Lorem ipsum dolor sit amet" repeated many times! Lorem ipsum is that classic placeholder text that\'s commonly used in design and publishing.\n\nWas this intentional, or did something go wrong with copy-pasting? Either way, no worries - happens to the best of us! Is there something specific you\'d like to chat about, Jeremy?', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 1084, 'cache_read_input_tokens': 0, 'input_tokens': 4, 'output_tokens': 84, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
chat.use

In: 24; Out: 101; Cache create: 1084; Cache read: 0; Total Tokens: 1209; Search: 0

The context is now long enough for cache to be used. All the conversation history has now been written to the temporary cache. Any subsequent message will read from it rather than re-processing the entire conversation history.

In [ ]:
chat("Oh thank you! Sorry, my lorem ipsum generator got out of control!")

Haha, no problem at all! Those lorem ipsum generators can definitely get a bit enthusiastic sometimes. It's like they're trying to fill every possible space with placeholder text!

Are you working on some kind of design or layout project? Or were you just experimenting with text generation? I'm curious what you were up to that needed lorem ipsum in the first place.

<details>

- id: `msg_01TPkasxBmaThMsh2xRJeRbR`
- content: `[{'citations': None, 'text': "Haha, no problem at all! Those lorem ipsum generators can definitely get a bit enthusiastic sometimes. It's like they're trying to fill every possible space with placeholder text!\n\nAre you working on some kind of design or layout project? Or were you just experimenting with text generation? I'm curious what you were up to that needed lorem ipsum in the first place.", 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 102, 'cache_read_input_tokens': 1084, 'input_tokens': 4, 'output_tokens': 80, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
chat.use

In: 28; Out: 181; Cache create: 1186; Cache read: 1084; Total Tokens: 2479; Search: 0

## Extended Thinking

Claude >=3.7 Sonnet & Opus have enhanced reasoning capabilities for complex tasks. See [docs](https://docs.anthropic.com/en/docs/build-with-claude/extended-thinking) for more info.

We can enable extended thinking by passing a `thinking` param with the following structure.

```js
thinking={ "type": "enabled", "budget_tokens": 16000 }
```

When extended thinking is enabled a thinking block is included in the response as shown below.

```js
{
  "content": [
    {
      "type": "thinking",
      "thinking": "To approach this, let's think about...",
      "signature": "Imtakcjsu38219c0.eyJoYXNoIjoiYWJjM0NTY3fQ...."
    },
    {
      "type": "text",
      "text": "Yes, there are infinitely many prime numbers such that..."
    }
  ]
}
```

*Note: When thinking is [enabled](https://docs.anthropic.com/en/docs/build-with-claude/extended-thinking#important-considerations-when-using-extended-thinking) `prefill` must be empty and the `temp` must be 1.*

In [ ]:
#| export
def think_md(txt, thk):
    return f"""
{txt}

<details>
<summary>Thinking</summary>
{thk}
</details>
"""

In [ ]:
def contents(r, show_thk=True):
    "Helper to get the contents from Claude response `r`."
    blk = find_block(r)
    if show_thk:
        tk_blk = find_block(r, blk_type=ThinkingBlock)
        if tk_blk: return think_md(blk.text.strip(), tk_blk.thinking.strip())
    if not blk and r.content: blk = r.content[0]
    if hasattr(blk,'text'): return blk.text.strip()
    elif hasattr(blk,'content'): return blk.content.strip()
    elif hasattr(blk,'source'): return f'*Media Type - {blk.type}*'
    return str(blk)

Let's call the model without extended thinking enabled. 

In [ ]:
chat = Chat(model)

In [ ]:
chat("Write a sentence about Python!")

Python is a versatile, high-level programming language known for its clean syntax and readability, making it popular for everything from web development and data science to artificial intelligence and automation.

<details>

- id: `msg_01UNPp4LdXFiXzNskyjYFtYP`
- content: `[{'citations': None, 'text': 'Python is a versatile, high-level programming language known for its clean syntax and readability, making it popular for everything from web development and data science to artificial intelligence and automation.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 40, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Now, let's call the model with extended thinking enabled.

In [ ]:
chat("Write a sentence about Python!", maxthinktok=1024)


Python's extensive library ecosystem and beginner-friendly design have made it one of the most widely-used programming languages for both newcomers learning to code and experienced developers building complex applications.

<details>
<summary>Thinking</summary>
The human is asking me to write a sentence about Python again. I should write a different sentence this time to provide variety, while still being informative about Python the programming language.
</details>


<details>

- id: `msg_01PhhDYxQmsugZ9FA9dWJgw6`
- content: `[{'signature': 'Eu4CCkYIBRgCKkArTux8hUUazB2af12MIUvoDhnszZflI5jnDZxzYKyVhHAOBBWhthY4dF/DdvPXwVDk+Eao4SYA70cwU9MwQ+AeEgwhM0rxypU1Tt5/DxYaDKUht6El9d7ytkR3SSIw7t6H+SUYURFhg3ERjCP8J0K3bk1/YhJa817lfc0EITY22BH3g3yhrwqsSDFDSBOEKtUBxjOJP7MLhQZ4wlkcY4CLAsugOYZW+qMXYExJ94FbylR7sYQhLTxMZa+2yQEcRYqgeh/dqeAwgQVYUZ9LeY3RwLvKhayZuDExxDqFZqFWzaQ0WPJzoTAyP4eTHiBd7INAegjbHwA//mAMTLF2boax5CEttDcAc7//gwlkA0ohqJjgsS9v1LsHEEfVR0Iac4m4ixBP6SHIA7ySH1k/XdxvRpduIoi/U7d+vcPkq8Hgg8rwETCjQYZFMCKrcnJBRu6zvsDXQ3QJZUgYLPGk9IAWZ/Oc0BqoGAE=', 'thinking': 'The human is asking me to write a sentence about Python again. I should write a different sentence this time to provide variety, while still being informative about Python the programming language.', 'type': 'thinking'}, {'citations': None, 'text': "Python's extensive library ecosystem and beginner-friendly design have made it one of the most widely-used programming languages for both newcomers learning to code and experienced developers building complex applications.", 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 90, 'output_tokens': 85, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Server Tools and Web Search

The `str_replace` special tool type is a client side tool, i.e., one where we provide the implementation. However, Anthropic also supports server side tools. The current one available is their search tool, which you can find the documentation for [here](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/web-search-tool). When provided as a tool to claude, claude can decide to search the web in order to answer or solve the task at hand.

In [ ]:
#| export
def search_conf(max_uses:int=None, allowed_domains:list=None, blocked_domains:list=None, user_location:dict=None):
    'Little helper to create a search tool config'
    conf = {'type': 'web_search_20250305', 'name': 'web_search'}
    if max_uses: conf['max_uses'] = max_uses
    if allowed_domains: conf['allowed_domains'] = allowed_domains
    if blocked_domains: conf['blocked_domains'] = blocked_domains
    if user_location: conf['user_location'] = user_location
    return conf

Similar to client side tools, you provide to the `tools` argument in the anthropic api a non-schema dictionary with the tool's name, type, and any additional metadata specific to that tool. Here's a function to make that process easier for the web search tool.

In [ ]:
search_conf()

{'type': 'web_search_20250305', 'name': 'web_search'}

The web search tool returns a list of `TextBlock`s comprised of response text from the model, `ServerToolUseBlock` and server tool results block such as `WebSearchToolResultBlock`. Some of these `TextBlock`s will contain citations with references to the results of the web search tool. Here is what all this looks like:

```js
{
  "content": [
    {
      "type": "text",
      "text": "I'll check the current weather in...",
    },
    {
      "type": "server_tool_use",
      "name": "web_search",
      "input": {"query": "San Diego weather forecast today May 12 2025"},
      "id":"srvtoolu_014t7fS449voTHRCVzi5jQGC"
    },
    {
      "type": "web_search_tool_result",
      "tool_use_id": "srvtoolu_014t7fS449voTHRCVzi5jQGC",
      "content": [
        "type": "web_search_result",
        "title": "Heat Advisory issued May 9...",
        "url": "https://kesq.com/weather/...",
        ...
      ]
    }
    {
      "type": "text",
      "citations": [
        {
            "cited_text": 'The average temperature during this month...',
            "title": "Weather San Diego in May 2025:...",
            "url": "https://en.climate-data.org/...",
            "encrypted_index": "EpMBCioIAxgCIiQ4ODk4YTF..."
        }
      ],
      "text": "The average temperature in San Diego during May is..."
    },
    ...
  ]
}
```

 Let's update our `contents` function to handle these cases. For handling citations, we will use the excellent reference syntax in markdown to make clickable citation links.

In [ ]:
#| export
def find_blocks(r, blk_type=TextBlock, type='text'):
    "Helper to find all blocks of type `blk_type` in response `r`."
    if isinstance(r, dict): f = lambda b: b.get('type') == 'text'
    else: f = lambda b: isinstance(b, TextBlock)
    return [b for b in getattr(r, "content", []) if f(b)]

In [ ]:
#| export
def blks2cited_txt(txt_blks):
    "Helper to get the contents from a list of `TextBlock`s, with citations."
    text_sections, citations = [], []
    for blk in txt_blks:
        if isinstance(blk, dict): blk = AttrDict(blk)
        section = blk.text
        if getattr(blk, 'citations', None):
            markers = []
            for cit in blk.citations:
                citations.append(cit)
                markers.append(f"[^{len(citations)}]")
            section = f"{section} " + " ".join(markers)
        text_sections.append(section)
    body = "".join(text_sections)
    if citations:
        refs = "\n\n".join(f"[^{i+1}]: {c.url}\n\t\"{c.cited_text.replace('\"', '\\\"')}\"" 
                          for i, c in enumerate(citations))
        body = f"{body}\n\n{refs}" if body else refs
    return body

In [ ]:
#| export
def contents(r, show_thk=True):
    "Helper to get the contents from Claude response `r`."
    blks = find_blocks(r, blk_type=TextBlock)
    content = None
    if blks: content = blks2cited_txt(blks)
    if show_thk:
        tk_blk = find_block(r, blk_type=ThinkingBlock)
        if tk_blk: return think_md(content, tk_blk.thinking.strip())
    if not content:
        blk = find_block(r)
        if not blk and getattr(r, "content", None): blk = r.content[0]
        if hasattr(blk, "text"): content = blk.text.strip()
        elif hasattr(blk, "content"): content = blk.content.strip()
        elif hasattr(blk, "source"): content = f"*Media Type - {blk.type}*"
        else: content = str(blk)
    return content

In [ ]:
chat = Chat(model, sp='Be concise in your responses.', tools=[search_conf()], cache=True)
pr = 'What is the weather in San Diego?'
r = chat(pr)
r

Based on the search results, here's the current weather information for San Diego:

Today's weather in San Diego features cloudy skies early, followed by partial clearing, with a high of 71°F and southwest winds at 5 to 10 mph. [^1] Tonight will be cloudy with a low around 65°F. [^2]

The air quality is generally acceptable for most individuals, though sensitive groups may experience minor to moderate symptoms from long-term exposure. [^3]

For more detailed and up-to-date forecasts, you can check the National Weather Service or local San Diego weather sources for the most current conditions and extended forecasts.

[^1]: https://www.wunderground.com/hourly/us/ca/san-diego
	"zoom out ... Cloudy skies early, followed by partial clearing. High 71F. Winds SW at 5 to 10 mph. "

[^2]: https://www.wunderground.com/hourly/us/ca/san-diego
	"... Cloudy. Low around 65F."

[^3]: https://www.accuweather.com/en/us/san-diego/92101/weather-forecast/347628
	"The air quality is generally acceptable for most individuals. However, sensitive groups may experience minor to moderate symptoms from long-term expos..."

<details>

- id: `msg_0188owjz6Y4F9Tgp959ttb1w`
- content: `[{'id': 'srvtoolu_01VA3fKkzcT6G7bLBXDuWA6o', 'input': {'query': 'San Diego weather today'}, 'name': 'web_search', 'type': 'server_tool_use'}, {'content': [{'encrypted_content': 'EqcCCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDOOAYgbj52nPSK3h3xoMUv62cx1MUhp2+jTZIjB/cwkAmEDsUt9s6BTnomsyYN69Vn6/CRM4u+cWzaCSJKd9m1JijsOzkz0rsNrlJdcqqgFijr9R8v6S1D3tdmjsOlyWoAcB2hw+TphU8czb3vBnFHdKj2+HJdvhWMxIoJeAwq+EQrm4biT7xzWIdZjbWrjZxgPa2mnFTSIUk+ZFStbSqLgwDhMdUPMUYburhxkft8CJcd+EfRqcRc08m4x6H0lWZSYqrEPMwJWibrMRVUVrl61678awNujtjYWhOdV4qnN0SCROfOnv6YyWVn00AcaBF6zevrDsePE5aRgD', 'page_age': 'June 18, 2025', 'title': '10-Day Weather Forecast for San Diego, CA - The Weather Channel | weather.com', 'type': 'web_search_result', 'url': 'https://weather.com/weather/tenday/l/San+Diego+CA?canonicalCityId=3b2b39ed755b459b725bf2a29c71d678'}, {'encrypted_content': 'EtoLCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDGspZt6TzalgEm2d1hoM486oqH/NaKUAGoHWIjB2UCPGYAn8tjFy7V97ke3En64y/xYN+jxryGcL4dVPFZe3BLHkqIeQs9qmrFuTQMEq3Qp9p0HX5pye2U/jApN13p/vAdvz1xv/psljKmqwAwuHu/BYHCSLepp1k/4szIKUEkW4HvXcP7CpeNm3EqTxZlpNU2EOuG8WI2k7rs1Mh1fyqO8PQb/e5QKOavJQ5q+6+kZrnZ72j3XMpbK5ghIpb/WcQ89pUeCc6QdMq5gpENbcjO6N5MU9Df3xzkAp0sSVSwpyZbd6yCRJieZm3+fSaWbB+UfWxvggGQr9FEHQ81r/UUURJH+TxmaUKQZ0E9+tKevPC8W8KkgVRTgV1woRtzypvZfgyDyZcx5gt6yWVdyDnVgMxyn9zPB1EQ9+wTqz7XS8Izj1hcrEr9mrIHpOAdKiQofUNhDyKJSItj2NUwwz2TW/QjLn+rf4q1i4jblK4M07WOqF5CMWV2BpEN2jSvtBGcekoNZr+UyTDlV+0r+acGJ5k6Xi4f91MypyPtL/3Rykst7bRgMjsChMKZfyKmfqAXNI6rEg38Q52gj2cRpx1hYmX02sji9OAclNFipZ3GDV3WBJH46sfHdIwfkzaocVyFZqVvwCJ62HUO4+Z/GX6w4PPYZdtNfci0hjG65ylNmMrF+8LZ32UE7iCTmk7igQlu4cOvhlFxWWZC1jZ98Eb8QW9Msh/jH161mJz/t9xV0hLZE+Whk0idBxZ+rhHRwgFClTjQnG3ohG9bsy+KX8Po0Wqf/ZgP2yP97WDE4L7+APeDNnTAevLdasu2hY879vLcVVXqqJA50v+Uhmcb1m/5l4Z4BtsyZhNr5bJ3fuTANSnkrKXmE9btsJzHbLxMhetWz7CP1iDezm1jd3WVEhpcUqhmSqlUrIdAvyl4NOQtwfmu3bUtxJo+f8QAtlKWUw+f0SBE46bnMtMGtgRUgES9cn1VVKgf0dHBw/qZVaQcpwIa6RjvwVsuGl5evrysnmZiNB/RZNxoqVswkIxuaczwa76rOtOXTNRYcUWR84OutEKku19jj775LM2MayNKrlCbDQy70EHPVyfWGlPy0z51XWBDWtXNP26d63J/Ij3JiQ3B86+50n/KULmPEdYRxE64qCnLTGvon6pGd3PxE5vY+cZ5CSrVUR6HiyDGGzqFJq5elOV1DBmNTAmb9FwjUZoTm7ONEbtH3FYlJiLiej4geQr4Qljpfv4TM63mb6r7AlKpFrX+D2JsXXClwmCLsZv3ATnCPhxmV5QtdnBVGqpBLt55FLoXMCY4x2vusdCoGB8214JIMl5MljiDf/8XonyyFBnkgDjjlFmALc1lphI7sLns1GWAAgRzliimDD24jsTYTJa/u2mTnlciJP1g9mE+eyIwZduKBAzFkNEEAXzmsnSE75aJIA/l+37mdpdihxxJAvgHH8wuEsCnGTdfYcQAQspZkMSaRatAZ63MwjhBxz6m7KjzuMlCXAox+dwRW510u6H8UT7WU2Nz2R+FSY2pEH7QNxH0LlvPwsulcdiFAezjdqZjDPXo6eFQOdw9VtcB3KvDRk3CqlZAG8Rt+NnqAKOTuBP50rDOm+5wWX9ZixcPi/hAJA7/eYHLeQzXnlpj4KUnZok01BeMkK58ovNGz9VQQmljxWkfXWfeWWUjS/vJidAE2dWCn5J7PJCRFJ0XVpVLa/jQtcAfDi6o+ZWkO9zXUB34gE7iIyCoEDlID/l95Y2T7IqyG0RSR3moRySSQH68MSxH3aLoOEadL1rk+H0y/XuDw9qwzRRCeuT/rGrY0skmQeApm2BaaVZbgCFlueSw0HX81CuPJVzTL79fzEop8QBSmXzjBRfZqfeDnYieYGuZXZeY62TJrUHIoUMNcCzUlBKk2N0QW4QUhryx5/HbSjTeCIMWDlARgD', 'page_age': 'June 18, 2025', 'title': 'San Diego, CA Weather Forecast | AccuWeather', 'type': 'web_search_result', 'url': 'https://www.accuweather.com/en/us/san-diego/92101/weather-forecast/347628'}, {'encrypted_content': 'EpcCCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDIWGXsYwtxxkieI/qxoMNWgRBj90YdwjJWQpIjAKWuBWe/LcbP1RwhuhwTTq1MQS87z5C3DOFIJIzbPwB5AFUA1HenNb33x2Uo8/3W0qmgF4VhNqj6wZHd6+EvSXDGxFBALTVmW3q9sS8gzuUbG93WgKjHU3QApytNTb0nahcWsj29rYl2RNM/9CsVTDx4P0vOribIq7lDTBGWaWtXXTxA3sbZBG19EqxvRxoqY8GAprl8ZWZjKqYZyHSbLzlgEzxcIHq0a8eOF6RnDgP2JXYfCaxKcO/oh7jLVh3HWoEmRmA+/LQgdNOw9OGAM=', 'page_age': '5 days ago', 'title': 'Weather Forecast and Conditions for San Diego, CA - The Weather Channel | Weather.com', 'type': 'web_search_result', 'url': 'https://weather.com/weather/today/l/San+Diego+CA?canonicalCityId=3b2b39ed755b459b725bf2a29c71d678'}, {'encrypted_content': 'Er0ECioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDHhykSlfj4MAV1+m1RoMsnVYq/521OeJBXQjIjB6aVZqwdTVsVONfjdXdQs1LjcPqlKxSFiock5al3CnAcX7+Mm8jnZgP2vm8mytxLcqwANB9BanO7LV+k4iPJJll+u9Cx2+11WpjnmnactouX3Hn8QtIKiBWyH/UnjgSSv5q7gYPhRyoDSE7AFsX6NAoMXlZ81V30IGyDJyjv/44oHtvYGShdTwECWBETGTvS0W1/8yvhcEVo1S8gSPmo+Bi2bu/qoeB2QrlU4SL19MlU4A8WyyTTgYaizzIKEbGdbBqn1yrEws1aI1fTngQrcJ2GVsDLdc/K/Owey6X4qDIsqFT3gEgtS9mRzYCOtgwTwg1TpZ/dG8SB1sCgmU6UMHk+XOjq8qtzOC3MfMS9d1pKY4PIql+FxUNHcBlbfUMXPTVtqR1Pt8fRB6oUbOkqrIyT8FTpnZt5BVIpCx4Owre/0mxCbLmIwjKSRjdI3NW3sR1nGlNHckr/UzLUgLubhtKMazbmKgKMFdqLjbUd+C7dFowCAUGK2zhMZlTrinRE6NTG97Cjzj7PjJEtfGEUr/NYw9yly4qyeREAc2nynGbxucFCTlNfa/VlqarpbUqUD6luzKeu/wtcxxwRoAIAtK90n4EL4wsTtrd5sxYAw907rfn0hJX84teqdsVycPBbYZF+JoBPGJDUcaPCRYoeR1HZqeGAM=', 'page_age': '1 week ago', 'title': 'San Diego weather forecast – NBC 7 San Diego', 'type': 'web_search_result', 'url': 'https://www.nbcsandiego.com/weather/'}, {'encrypted_content': 'EsoCCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDALMEssLh2cscqaq1xoMNGTaBC9MBRUbe2BWIjDaxbjAJCSuNU8KHEdzVjfLGdqKCGLEg92xwbdaInRdVRIbgtFF5Q6WOCLS60uSiPUqzQHJov4iN2uZfEmT0/sIKvo1E41CnFGFE6N7AfLZ68Kfi88IrZc4EWVFBBzumvCYpSnpUdc12e7R9HSc4Q35Vm9j7GuTMX+8qMLqBztuUX2Mb7TNtdzfeK1DzRYUiw6NCSKu21V5yaGpFkMN9R8y17tDA911nYbKL+Pip8CoX13Avj8U3tciSG6Zuy0jsjbAg6MVl1cPyVfrvJ3xp5CohYpam3bouUdmOldp4F/lwiu5DNOQJseOwhFghEtLHSEFQFRiBqca1CEe4GJ2Ur44GAM=', 'page_age': '3 days ago', 'title': 'San Diego, CA', 'type': 'web_search_result', 'url': 'https://www.weather.gov/sgx/'}, {'encrypted_content': 'EoQHCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDCaTUuMpn0vDqYfPyhoM/BjHsOzEZvphHllrIjDtndEaHG9jYIUJGRldEYKC4zc/n/5X0rkfcVR6SdqMPeBi+EXSXjI6XIfP2fXtNU0qhwYGM3DppjqD93bl6giibA1aYWtGVrtxTBOReAN6pvsbgAKd0j7/SL3CAKzMtvOV80q0fFD17sHqBvnsb4pLq4aVv8ZcWRgPH/WVahkeYwVhnE5jyT3Jtgq1YZ/P3dzp6VjNl8A7C23ra30a+Ul2cJjuoN2UtZIXhPbZngsDXgL3kSr55QA3BM99jq2G9OHxb5XYPajsed382LbmHm4y2MgwbJA9ydfLnovcc/FTf3JD64UbGui0w7uuj7QtUqNPHCrApM/FsvhEQHjocJ/iE8Y4Fm7joO6znsS5A0FNCvYFPfIAFQrDnenV1g+x9K9uEt7WNBfP39se56Eav7ppNiVvDWf4chX9oe+yZG7iDJYot+B9XPgL19kK8pl9LdRML2uh0ozssTtRCcU1mnnOuHFbkDSgFKMO5hmzNnf707nW8HPUedW+V4KJ/Ax+LS8qTA67sJ459y7QmIT6qKd2Z4/4kaeIDpdb6NNWcNZtRDxYJkAR4f29PNLo1bscpTnuy0xsC1yErrHb/vUlmx6WMcFks5NdxoQBRcV9EqGcDllvURvay7ODtJjs77d2x519A7xGYRh/pmAPqdfbcXIoxqaJDnrtnb1wRNjcO8vDw2FSZ5wx/eHN/5c9p7JsdzbWkTqwxYTWkUMJJbShuaGgfbAOcOQv/FbWNfTFg+GLtGj4HIedNdlqN+NEZkk1gw7Bg49mUbWjCis/BHi+FUtfVk8f1HOhMzkXfKOVd2qpCLWfaxZ0kQj3ZBa/yIMfOnFYh0roknqj5hqz7Zw0Wo31y+P+7S5ZUyIeGKR8KRd5Uu/9I96IRDsefoyRnZB8wpQbfmir51Mi2QrEA8INZ0L9vl+ds2Bm0EWmFRbLKigV035GMJK7Ru6aLYjb3S1EdK33l6iBXyzrBPa765Wmn2+JjvURwLreHKQyP09Ib8v01KrakSlRcbR3cKF+2jLjNmng3Nhfrzb4qvMEG9LrpfBaf1KsA8Qtj4lFFoVMm2hOBQQUnJo1mLIXpg7WHDHx2Mts95AtU3SU4n7HGAM=', 'page_age': None, 'title': 'San Diego, CA Hourly Weather Forecast | Weather Underground', 'type': 'web_search_result', 'url': 'https://www.wunderground.com/hourly/us/ca/san-diego'}, {'encrypted_content': 'Ev8CCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDP+kQooFhSlL6Bb/WxoMIfIVg6EKLrgqWHNXIjAMC6wPSSee3/jOaP/U7nyotTAg7SDovxxgIaqkgn+4viR51H8BwctMgMWDrOkY7J0qggKlI0NjfdwVX8P4GKalxdM8j1FpVuIo4k1WX3MffULccNZIUoiPADetUrxWwNYLfmAw1NaPd0XZNKx9E4IUMaiRSUEalQIqdCogfYFAFlZW0JI2ZBgVeOIiM2rRmUgrbKBSEe/TzGbBzTpyHVHxjqC7m3c9XzF0B9H6fgcHD8cURchCnaUFq2rogmqRVBTN4vYF3z+fzL8vMLH9nXdI4zeKj2qWg/zoBp5+sQTizjkQ0a/baVH7YQNUP+VlJ+eKyKI5w+qhP+MHNkqLz9zkdqbOesg6I4EEKbMFK4iPNTBvIBwrgtj9uvck+d/2TpmywwD9RaOMDEFRJyoKuh0tYMeis8AYAw==', 'page_age': 'June 16, 2025', 'title': 'National Weather Service', 'type': 'web_search_result', 'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San+Diego,CA/'}, {'encrypted_content': 'ErYCCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDGB2+yWsweRO1NJ9jhoMNPFOL+R9XLcK0TnXIjCmzje8Flojaf/wsUQSur93gsuKu9KxILXVAmaxH6VlnSzmfVudL6vAXiPezrUn17IquQHab/Cm1NpvFn0BJj/4i8eu2+AvV8tc0ES5nc2ukBD/GKJnlCLfxD1xe97Mv9QfTvl5yElUs5eM6m3WKrX3wSr7lG48lT/WXLTaXVw+6lK6tSBCa6RRfjFmsSiqPbkh9c/TZWNDBOJzMtbgOfhzv4TAi2ps/rpKp2Drbkhv7srHLk57kCVVRpBQewFtjrkwGMofcqP0Fs7be7/+eqd2HR9PlxG1G096dfkTCaUfDICZboauyx1Sje1CvhgD', 'page_age': None, 'title': '10-day weather forecast for San Diego, CA, United States - The Weather Channel | weather.com', 'type': 'web_search_result', 'url': 'https://weather.com/en-GB/weather/tenday/l/San+Diego+CA+United+States?canonicalCityId=3b2b39ed755b459b725bf2a29c71d678'}, {'encrypted_content': 'Ev4LCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDHy5Tajrid9RXi86hxoMwUHZeIooHqNP9PfnIjDiNGVU/aQgRe4wzB+Z+QvnIJX6TCzroPBIWo3fpXgZNSJUmt2KetESuz93SLnxmNoqgQvfVMSifSguG+KDzs2s9AP+BUuYUZYiWJsSNrltmaJYDuydV4yxDwpW9zdFTmbPM0bqYhq+DZe36ItOZ/3KfDi8/s7FDY0eaqG9KtldySMfieOnS4x4ZyHPZEN7aaZOYzywq4KHDUy2TC+fhNnOwXgM+L5zJkY3r5LZ510ZIRWISv97m2lTpgYa6ioFUN5XaaqcL7wUwb0feQ9+eT29xeCYet+mE19BLE9P8vWHuDsdZhA521EmftMxXZ1c/A8k+f0nZkJiQ8IW8uFddFcFBoR/2qjHAVW6c7p6bupELOF6JdPiDIn3PE44LTm2oW7VoOR3+MBYfStQSMwLN19u5e34vcn+y14HYsSkznkqF+QNb31gKJCEItSxkHIRWjvN+Etzb7EfoSZQB+wSlrBYNSiySsnAxwbQ9GIx2ATY7sIHKZbzqiilJdApZou7VBeVMH9XdlqoCS+Y19merJwdnAFM2BY8YGQDtb+yNCBcUKboTmGtiUpBAkDlf+Ur0F/QsZ2fOSEMJVY41Kaw5bC6HMvy90dxhgZ3NWmo8j5eDxZiMOvST8gK+rp1qmqNUF7rtLGJg8iiNk2LPW3RXw/tcgTYHWum3DY4w/az3Q0mKMaQroxErB3AN8Coq/V995rehWc33IFPn7GuY1anA4ZZVlSw6obNT1V4jBHD2FvUbFK3CKpzDX7NugrMfE/+uT1y5Tubo8nrOfqIl6ECGTfE7zCM5HXz+eHO0WV9kMC3M/OkMlmokpHk6N4rp3fTMCWVxt0foMvDpXj9S69JxQH1C3TP9qCFpIJ6FEo78UvTpxRosKfz1Moe07tnEFoSMAvzXvzHM4IdidwsbRYc7CHLaVDVCNBG6RwkvSet906mGnJaUIZ4dyiHgui2yBi13lNGtc8JnkQ7wcmbNxwkwvH7loolUmYK99Q3Er5A7DlvscnJuwtx/7Yu67hqmRoRp/NxbFk22vBgJ1Qz2brDsFfWpXujV4yiF93sVDjg42I6bPda6xvpEjHBl/zAE2UAnwt6Y41+nVRqM2rjdN2xpBVOg26mFSocdBGnOZJ4l4D9CW8Cht1OpRfAXJvodO7uwvhMZsNzdxM1tkVI9c4p2pC+CAq4x7ktPoPsuPbxAIf317CDe0rcPF8g72mjN9G9xRh2hIY28i9cKRyeSCHp2qwunzfq0ZZWFcxxxgJGIoqCq8WCI1LmaQUHyJVPDbasMu9PJ9XGHLq7fvzVSEBdB73NB1c/PwbMw9CQYETjTmo1bPfJBaJoaa9M5Qmib+zka7rYYm78IeMVwuEVSXc6T1pQff3XXslYrnXfW+HDXejA6b+PH17yQXH84z439GEXZsMJUX9Imypj0MoNPFTgb+PmIBYBL8DvKpzWslYTOWq53CNwWZEwaG6Nl01UdRcJgo2XOQLtazuNDP5WJP1KCihbKJGRgfd0j1baLi2WGVtTCIbPvd9mxEnFrAi7RYAKPSrRuVNXEBClte0zzjgH3Nvp4al8jDeVSv1sgLtbgeV+uI7pru3PTjr8Se34P/Q0QpyUavop6IrNpu5MZp/FPud0Q9kB6DTVPLBNODW/ooUGW1IFjArSaeGs+ebdKlQYVRMHMl3aLukc5+9EWIHUMiENE9tUaAKX/EIqzb8Go5ZNhUX9kz/wPujNPMm3CG1WMft2Y4IOXTfBH3L18D5yxU3hVzv13tvsXPljskJfzKRLUBzdQEDiuaoBkvrASi1Yc7CEJ/Ixa3Nn37sOewZKxQ0MnoDrFEuS/Dg9kJ+ZNrE2StTBQtA1imk1mdNY/L07Ir64XADiYES07v/YCAaimki8wekwMrhGkofMsnrv0YR1OCh7kc+rvvdho9bmnSpfaSQmV7Dz56lxh05HdHnlyT/0z650qhgD', 'page_age': None, 'title': 'San Diego, CA 10-Day Weather Forecast | Weather Underground', 'type': 'web_search_result', 'url': 'https://www.wunderground.com/forecast/us/ca/san-diego'}, {'encrypted_content': 'EpkCCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDKEHZkASky3eSxAA4RoM8kUmlWKEHl36WFfuIjDMH3voln2NLyRDKdAnduVI11ga8fd1NGV2zyuEW884sUMthHKYnA4TrLU/wZMEnI0qnAEQg7nKwOF00UHAj7bKvRfSXIQpKj8yW5Dzmt2XBfY7rWditEbFkCQkKZVw4eHjCR8V4x2lG6B+xdwOCdGfw9OFloS6UA5B6bpGfFMhNGB+XqaQhFRDrVW2vnF02qckWeXSHDCr8zcD5nl7RlgKNRTIc6vo9ZWFsyo57mZbHZteRExKngSYo/sCxpeMYeaaouw0ilBDV49w1sFTTN4YAw==', 'page_age': None, 'title': 'San Diego, CA Weather Forecast | KGTV | kgtv.com', 'type': 'web_search_result', 'url': 'https://www.10news.com/weather'}], 'tool_use_id': 'srvtoolu_01VA3fKkzcT6G7bLBXDuWA6o', 'type': 'web_search_tool_result'}, {'citations': None, 'text': "Based on the search results, here's the current weather information for San Diego:\n\n", 'type': 'text'}, {'citations': [{'cited_text': 'zoom out ... Cloudy skies early, followed by partial clearing. High 71F. Winds SW at 5 to 10 mph. ', 'encrypted_index': 'EpIBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDABStTAq31Nlum6NKxoMiXYgHJcwMf8WVMh4IjBjiTgYGczfJMgZQov1TWlo+GafNzAMVP3hbsnUruguXr1vifjTNfVfL4K3/7J8eEgqFumYOK0SjhzD6q23sDzw7/S6A3nNd34YBA==', 'title': 'San Diego, CA Hourly Weather Forecast | Weather Underground', 'type': 'web_search_result_location', 'url': 'https://www.wunderground.com/hourly/us/ca/san-diego'}], 'text': "Today's weather in San Diego features cloudy skies early, followed by partial clearing, with a high of 71°F and southwest winds at 5 to 10 mph.", 'type': 'text'}, {'citations': None, 'text': ' ', 'type': 'text'}, {'citations': [{'cited_text': '... Cloudy. Low around 65F.', 'encrypted_index': 'EpMBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDAhCIP8he32EUZ4sLBoMY4reP5dMegHKZCpmIjAFuLKRAM6CkCwZIPXL6J4CIChh6aS5ARbTgMx6lIf8ajPvu12c+hhiEiZrEAE1AOMqF/8b9Fnl1eFvMRRFadOiFFiwnk17oAxZGAQ=', 'title': 'San Diego, CA Hourly Weather Forecast | Weather Underground', 'type': 'web_search_result_location', 'url': 'https://www.wunderground.com/hourly/us/ca/san-diego'}], 'text': 'Tonight will be cloudy with a low around 65°F.', 'type': 'text'}, {'citations': None, 'text': '\n\n', 'type': 'text'}, {'citations': [{'cited_text': 'The air quality is generally acceptable for most individuals. However, sensitive groups may experience minor to moderate symptoms from long-term expos...', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDI+/6ggPgXsbcc9GLRoMrPmJ3gNYcLdjXOcEIjCZlcUY1Ke95rjn4hLubJ1HmIpNRlN1ojLwdPYniBMy7HkgNkFtkwwiXdjoJ3cZ5xIqFXyW+vTEfVoqM9FDO6RQ0skXJOA/ZhgE', 'title': 'San Diego, CA Weather Forecast | AccuWeather', 'type': 'web_search_result_location', 'url': 'https://www.accuweather.com/en/us/san-diego/92101/weather-forecast/347628'}], 'text': 'The air quality is generally acceptable for most individuals, though sensitive groups may experience minor to moderate symptoms from long-term exposure.', 'type': 'text'}, {'citations': None, 'text': '\n\nFor more detailed and up-to-date forecasts, you can check the National Weather Service or local San Diego weather sources for the most current conditions and extended forecasts.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 5178, 'cache_read_input_tokens': 2042, 'input_tokens': 12, 'output_tokens': 255, 'server_tool_use': {'web_search_requests': 1}, 'service_tier': 'standard'}`

</details>

In [ ]:
chat = Chat(model, sp='Be concise in your responses.', tools=[search_conf()], cache=True)
pr = 'Tell me about otters, find quotes from famous scientists'
r = chat(pr)
r

I'll search for information about otters and quotes from famous scientists who have studied them.Let me search more specifically for scientific research quotes about otters.Based on my search results, I can provide you with information about otters and some quotes from scientists who have studied them. Here's what I found:

## About Otters

Otters are fluffy and adorable mammals born for the aquatic lifestyle with their slender bodies, thick fur, and webbed feet. [^1] [^2] They inhabit all continents except Australia and Antarctica, living in rivers, lakes, and coastal areas. [^3] The 13 species of otters live in various habitats. For example, sea otters feed on marine invertebrates and use tools to open clams as they inhabit ocean coasts. [^4]

Scientists define animal play as pure enjoyment and not connected with survival. For example, otters play by sliding, somersaulting, belly-flopping, and wrestling. They love playing so much that an early study even observed a group of otters slide 16 times in less than a minute. [^5]

## Quotes from Scientists About Otters

Here are some notable quotes from scientists and researchers who have studied otters:

**From Marine Biologists and Conservationists:**

"The sea otter may be one of the most widely studied and intensively managed marine mammals." [^6] - Sea Otter Conservation

"Healthy otters tell us we have a healthy countryside." [^7] - J. C. Tregarthen

"Even an otter picks up a rock once in a while to open a clam." [^8] - Chuck Lorre

"These little fur balls are very good swimmers and can stay underwater for three to four minutes at a time. Otters are very much like you boys and girls, because they spend most of their time playing." [^9] - Michael R. Greyson

"Few people actually see the Eurasian otter in the wild, yet it has become one of the most popular species." [^10] - Daniel Allen

**From Historical Scientific Observations:**

"Yet otters have not been hunters in water long enough for the habit to become an instinct." [^11] [^12] - Henry Williamson

**From Contemporary Research:**

"I wish there was a verb to otter, ottering around in pure play, to honour Otter ludens, which plays in my mind long after I've seen one." [^13] - From a contemporary scientist writing about otter behavior

The research shows that working in federal science made it possible to maintain a long-term research program that provided a deep understanding of just how important sea otters are to marine ecosystems. [^14] Scientists like Jim Estes have been instrumental in understanding otters' role as keystone species in marine environments.

[^1]: https://www.trvst.world/biodiversity/otter-quotes/
	"Otters, which are fluffy and adorable, are born for the aquatic lifestyle with their slender bodies, thick fur, and webbed feet. "

[^2]: https://www.trvst.world/biodiversity/otter-quotes/
	"They are born for the aquatic lifestyle with their slender bodies, thick fur, and webbed feet. "

[^3]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	"John Steinbeck had an interest in marine science before he met Ricketts. "

[^4]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	"Quoted in Larry R. Squire (ed.), The History of Neuroscience in Autobiography (1996), Vol. "

[^5]: https://www.trvst.world/biodiversity/otter-quotes/
	"Scientists define animal play as pure enjoyment and not connected with survival. For example, otters play by sliding, somersaulting, belly-flopping, a..."

[^6]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	""

[^7]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	""

[^8]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	""

[^9]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	""

[^10]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	""

[^11]: https://www.azquotes.com/quotes/topics/otters.html
	"Yet otters have not been hunters in water long enough for the habit to become an instinct. "

[^12]: https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm
	""

[^13]: https://www.seaottersavvy.org/
	"In 2018 we asked Jim to talk about his picks for the most important milestones in sea otter research over the span of his career. "

[^14]: https://www.trvst.world/biodiversity/otter-quotes/
	"- Chuck Lorre · \"These little fur balls are very good swimmers and can stay underwater for three to four minutes at a time. "

<details>

- id: `msg_0176EyQ2L64jxsppotrxdt5b`
- content: `[{'citations': None, 'text': "I'll search for information about otters and quotes from famous scientists who have studied them.", 'type': 'text'}, {'id': 'srvtoolu_013GLmxiDRgqR9hda6yHPEsk', 'input': {'query': 'otters famous scientists quotes research'}, 'name': 'web_search', 'type': 'server_tool_use'}, {'content': [{'encrypted_content': 'EtsdCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDJ5Mnul7+BDMvxexdBoMehcivNLvnzGX/1Z8IjBTnD3yMo8/dMx2BAmH7BUa9cLamseo6sf1P1SRNSWR8ilT7r07czd0AHTPDHZJyjoq3hyXgwUxDvtgpaO57F76rbOTL3sJB1C5141ECT+HNM6Lh6Azyt4ZS+EsTwylCa68DG1qCYe7niH6uM4a5qwx1ic04ltqXZ5kVQAxPq3cQPiIoJNked3MJfAgY6RBZBbJTHvHcbSMUu0g7ZXuCMzQQHqfwgC2iK8r4+vsTBt/RW5YmJ8Xgi3elF54ZD7Rfld2I9vikakUk2yqgsLoqnOXRzyvzGicPjZJ5NxMYLOOBocNcK/55SyVgBhpQ2j9e4BeMii2XHTa7dLgxnuRW13zdD4xCiRtL/XQGtRLPytDzxGT+tRhRdSYZ6sgK61lo/URPNmmQJwqUh+Z6mWgxIvujGfW+3WF0zfEnB4sotmezdK45Y6APXX4YRqHwSi2Z65HcfhZABSj6YiOBn9n1mmBXxYqQ9yMGQQ/H83uyj46WvPwhem79rTHL7+dq/c6H6smhRdy9TxuZRibMXoGM/3r5hQJ6kcMl6AB2xrFkdnPe9yuOYpWDYmQDrYhEfMZikRUYcSsyl33stI7UPIpi9x4EZNjJVhsWasBNm8d6YRUE+Fzt8NZ1WvJDHw9cvU/IBO/GxCJVVR6kTaE56KyWCgpxk5ezjS9/l1dlQK5gouU1lGE/ycHR2tzMfgrWCOoqvdCFjNwLoML9gM8Gfbu9WeYWvyfQw7cdFuHJFDuRi1oXglwhYFD0MmT61jLDa5DCn3XIvXff9tHaVHOZXA1AlQX6gBc5ORmrnsm+kUu8BH4fG2ibgByJDtxXH/dlCi2+zpeB4beE2NsGRElPF9oMZ+9ltntP2JqmWiLsrOEg4Rh0vNFc3EAwUKtlOo0ADonNr2zrRtZOor3Bg8gfwOZ6zClmDqZ82TiQ3dnE7IHRmDymDpDTS1fsUIsiaXVhppL43uc29/GxX8tDsbbIgceOH/EAjST09SIudLqy7QTvV5H6MZ8PeLKiSXf4R6piK8DlgfvM/+K8D7HiErWdeHWXdt25WkZKQ1DmXICmO4wQLZtMj5QSEJPqJHqi0fUNeN6LsWtwhsRjVL9pPwZ4T3ov4gn/agRFmXS5vJO1fmTc1XyU+NbBoqUGfW1C2/SV7cOaeq0FD0xm7CboczRNdNNhhsMRm2ASxI/qDkH+SwAyuNiCEOnYNp3KJaR4TkMGxGFaIksMB9evJxRPjbHpwx9nkyl1nQ+9FujNr0v5xoIxKnfGlGcaeGIajCs7um0JrleX2Eap8BAZq+FUbxq7K9k3/mEDKICag0+yLI2NioISff+wOUX6GMnB3LumEvIeKmi/QEq2QYerHLXQti6fGIKeW+Rc/qPdDNSBYDd6pIZAEIBZQGDG08uZWkz0Iu3Z+RBP5VefypuBnNeqzRbcyYApQs4pO55xgQKBPISMO2nY9PMi+SmKfvAgvvn6na1K2awF5oUJRH/sxbZ3CKbVvxzlOX4O3c0moVrbx1utdH+jFDazaNFcKbJS9Bcrwoans3OGxrixVGoW+I9r4jkgZB2xlsE1S60CP5VVaS+n2IN1+rTuuToHxc8j4lxqPhaAlZSKw2e6lFm6N3oG1A/MDmqOwsqOKxgvrMjMDcG4/4e/Ni6tGIcIiE7r62HJPjxxfPyK8TMcmQmPvJhsuDKRJpWt0vz/4YVm+yJKw8GcGhyZ+Rg0jNxnZItEhL2JZz51e4emRLo6pqCo8cEnai1fS7uH2AmsYoIbYDnxZ0fUZC1bu/puNvvzjjt+gPG/cMJyQL40EfYtLoSPVsqe63UA63ib5HQu/kJB7TR1Ib166fnDdgcv/gF3hjxcEO0ZglL4A+UBHIpRDLbRmoOTqdlgDne5xeGFzM2OfJ1wbIVi9Evshx2fu1bj27UK6gvp0jurvN8GUVbKLRy93JkpKQcn5wH0SBuF138yOkHl0sCrE0VgPAc1AH634q1pRk3xSqex61KVMeuo7GV6zJGdfmBARYxA62wSmBEbeEKu8J1hSt4NLTnKiLVTKKarucV5glB3C33Qjk2TtGOMrADsL5ywi7VZ57RJyUYNEmZZDaDzDb47cOlSncSrHduGR8SoLigvDStXThwZ6hsImzeBYppON7F6Puz79Q/y0UNjg7Q6bcmBnjkpy+u8YbsxVBoMsyvDTh+knHMQw1L+fecQISD6nVE3tPyDN9IRWWsl35rs/hxfetIT8R7JGvMqsN4Vl6p36juLxYlzwehAmMkTC84ZCw9B25N13Q6cUA0vEFFb1JGhmm/IOEgHWpR09sD7PP96n6m4Zeh0EK+ZXlXYXLzndOdWmg45GdHnCqU1FIpV/4qMd9jBU8KNGa4XWg61rlLBNEqh9dlZJh88mm5Ex25jKVELUC29qxUoNlgNpDVF+7EvMyiY1Jhx+tNiKSMWEpF5+TzVmIj9NQughyzpm58sQDDMN5Czq2upkyIQBGDjaiwZqkl5yqb9+LAWn2iNOGRwtI523JVl001nCLBWdi9znuE2+AM5puD23jMkswOl7uVb61zbNzEDf52SCNyCocjXRROfdm/gpLvR0y/42ayH178w+RxFwYUBQ3heslok5awGOrL36BEo7ShV94Fwa/IbSumr2xqnJ8IicuJxPK9xcuO6QYKpvvRUmC76liHAJeFVBImS8dc3yvGtWpOIOcBXfUTWiJAA3B9IUh+bVrXVSdT6h+GyszFeSYT3/VwOwZp4/69Fitqnlluffp0yUP5Jk/cFstsfD07tiYMffP1BivZyVexAE117EGPhXGc+YD4E6PVRfpcsrbEoZ6DPYUDlwmPqO6Yi5QMXC1Yyb2UzQ7DFRACsB+gjL0AIzHIQWNO8hJ1xaK1mj7Xjg5Hfp1Peoqhq0T358QQNYuDl5dfaJ4tXMBovgilhR7/nfTI3pn6/r2Vcv7fQUjyLShzWisgHjzdF4v02U4WydQlopBr4x8VArvlawLaBReqlo2XlJx2o+deUDZpoVWntMkZGjyWJ7ulVd6b3gwtFTgJ9sgfJLWB6RUtjN4hgMT9V+ye6ZiKzwNSvH7pYyAa1cD5XAFAuyJvbqN/ljIOicIddcTYibYHX1JCo9rKzrgdorgKt4w5iyBZ8XXrPHC6ZlubDKkI21ftnGQwRCYkLm/DuiiWy9UboggCCHtx2W0oFVXBO/icX0QTQlTJefp/dgYLySyj6Qbea3+/f36s11sPzFgzzHoQOAYv+mm+DYwTgjbPwT02iVC+cY6hs9SjyrPosCABC4asV9SWgYEEuBisWkFDKKFjGRlOzdzrYZUSqmjjE0zP0PECetjkHA646ohxQJIpxeOGsxAQziCxh6v7Tc255xwDS0hZnqpC8K1A1q9WnfTM2IpoOuJbHvsNUnKLKGGmUqG6xr/PV07cpReAOLlfc5/zI6zyfhGqRuo0bMBK4jPAI5MGyhSd72ZnhNBSSsxB1HdVMGJxBRmXs13TMx1B4yesXrP5I51pow+BdvNooxnWwP2hP8sPhEtyi3o5w7WeDqGEE5QjidNTVFcrkDqLSz16Yigo0Aq4ZsXluMEFaQbPbo9P8mz2saSyUfP6r/zTGsjiSDwfm6w4N2iW3XJRL0oAkp/Wtszc4XC/9byNz2qWRKNl+2VOohcSk0GnWcyMdKQrchXKYA1GS1w+qrNUKj2IMu+TvwmpRkvCoA0+1Xf/UVsXMq4RNPQ0XdMfSiddvWbFYW/b2739E4toAZoMl6Z+xzVsnCQHHMR7MftwdFThDUWlpwavZcXZrv8m95UcRtrpCXMeZFeaxwDXPohY2aH2WqA0XMvg5EBoqgQtMkIABneq1gd1BZPHMw0Jj4qV8wwc33CpDmnw4x+a5HgcYfO/lU5Uee8Gg/RrhvGjY8eLOb38jnTw6ac95S//81KXQbN5p/IoBB85tqd/kK645lGKkK4Mbdc+j4jOXl5UpO+uY4Zi1l16bt7Dw+NdAXzry6ei5OnUzyO1TCq/04Dnl4UoPFaadhtsyHuZkNbkWqQPO6JOhhqZrML/5wl/73D0AFup0I6uFCgtYZDg0qr64UbIF+BinV0nZPME1qUf23AAvg6vsPZ5DCVgABO1kcdfuWdBIlbF1u6cY7FN6ptYDwH6SbCUUogzQDhfcUi2/8xWErfL3G+FXE6FZdS3MwBlPiVLcOYyWG0fWb1j75eKBBYQpflVuaSiiRCVMcJFyUydeTHXcIXfh3PAlNJzWHtF6i6uuf64OpUWpJ6y7ayiNRZfg3C7JIrxO5n9YxbdsYopgBEgoYfoKs37E0crsU2xj37gNOA0JDMI2kng05G0FJu6kmkr7Zco1CxBa8C2UTfx+79T2eC2V//+wFytrvFbRH90XvBkud60osePbrMJ8ZMWtdogefWsEsj3rA9hhD+jeaRoAl+ArsDXBQqRD4B7H+3LSDS32xU9LBSQPrQvzeEKvwDePvfEz5UCd2Z18ivOvXicBmM6emFGWb5WbumrEGadsUbDou3jePkezM+DTh9REvB1SuNVTljwZRSt6GvQ2XO/rLpihuLhN9f/hm3ZnciKbbGDBc54I9SpgpIALVV4QONKd9xvgFJjNjTYrYXrjdpSkwf/AnSZ1ZLrs+2+5e48UGHcElqkh9V6icPOEh0ZtgOGoR7xw+UDEPUiO2L7ROpcjqZdpdkQOXeFUpJP8Nz7hUHRdCUswNrwtqFuw7T7ej1JCDCVB5rG7boMqwSqIggFUdf2vRsyX5ITz9YanCnLJ5xElxvv6OW3UTtsBixfwAWxLrm8QHyjYEUEfsGjKuyHrcquhZ2OmLVjeWeWrH5Ag9mePYTJ8puTQxoFiwgyMFzO7wzuaZfKZHFyWTuVNkwpwFpS582aZGU8UeWbEMhqC9W9MScw+zX4dYmn0rFF0Rx8QAHHEKMXhjHl5qOhzCbD8E4ijwq2OaLBDPm+fTr1YBZPSfj8hK1aHfR18kNp2h756qyYyBpJyjyCjPpHwkG+wz8YAw==', 'page_age': None, 'title': 'TOP 25 OTTERS QUOTES | A-Z Quotes', 'type': 'web_search_result', 'url': 'https://www.azquotes.com/quotes/topics/otters.html'}, {'encrypted_content': 'EuAfCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDMXWg0Dv5YJl7O6PNRoMXv028vFaw+RCZoZ3IjDK/sDlpA5O4VEYb07zhLMUCx48uB6sC9UM++o9oZfg2aqfMYj5HoFAgQFkkuE/KB0q4x4H7DDkv2r633Iwiu8v0oLcNIgNAuodQqLNw2kLkuBRLSPbqtVGx+1FvOUL1c6txXa+D3RgfRgn4gE3MeyPol486qccfxxAx7v3wzwS9U3Ry7Jl3GWhMW3XSB08cToTBhrd+CRqYwzCsvF85qTiKbjseM8wGLkN8Ne6fzS/k0DLmp7WVYfL/CbvkOPlu4pWRVgivUhvV4mxC7nmkvfJI+bQX00P2GHpa1WTIh2IW8CR0weDL2WKsZPSb0wu6P3cYp/6QXeL+IWNYK+DVqxJXj4Q+bktIKg6boPsg0PIySZamY6OX6jcTlQaotXbdmkvKKWL4wnaioNlV1Bh2FeFdHIqt8eh86mFKu9xRUMjtPR5qno7imcv+32UMGWzhKxXpZrDDKSEy3KbOtsjL1VQNUlQh3HEt9NIKTOyLj66X8reZUEdbsZNQz5g1d/V9alAXjYmWNtnCH+i4YQjjdC5VLDu5WZLaCliaHYow0fzlEm8abFg9yZ4I2/mkEB1wkeEIVd4j85f32OmB5muwFCCknApFWgsqQwlilGkIlu3+M8xVQZ+fLe0FxvG96PNHNhwYyVxvztpKCnLLbz1CFBbZyzLSH2eVb4jFM/tF5y7ZMYdCfhdLW0QixbUbUbjJR6ykh8Epa47WnuJC8k3jVHEfg8LZUhp28qYh5XZixQvBRUwoH1gaNrjT6FyFFwwOMbpSci4l1QFhy5Acv2xIvErUQYLCfa5Mmgj6cxQrGtfNCfiDOdsYwKsT9ZSAeeCLKyXh2u5+dtfxlo4Ah4WsXUB5IA9y5Lz2vkQs1usLVZic4CeHk481v7TEsmy+Y6hdsh/2ls6aDQ3aBc0oktQ9lTkVvDbN2gra5xKof52YIrXaGtV/TzuEDLrONb9H2kO+4OV40wJi8belZ5IIvt0BpxeZWDgJWdoAIjbfJ6z2pLtrk7Y3NCmrILD4wIwUVc7a4gkgfWuyhznDUMJ6bkcJMQc52vW4sLhddSMk2TXJswlNAVV7siE3WGltKHgMLCV7ObDJ1ndJjLVzivT1yMt+XgwAR8QRj5FXq5lgmwz9Cl325beXKr7ESbxAIu2ACf06pVo5c7pvmQ167FphYgEnj6aqvHHrRO0r4zdjVRF4CbiP3uZYBKuG6jSC3ezPheUJjjis5qfD9UvYnOBaYNH0uKZBjIelylzNEq3hbEQx47UESU8xDZJ87DjjOFvkz/KIL8KHRvWzYKIhTtR9eiKeQreBFs47xYWzTtBqhz/dhBtSrYLw8Ri49jHI6yENV6rOrkDOoomVSXA53C96oiuUf1uGzNujdJD9ASxGL3Jk/uSODRL5dWgQEpjknbqHKkW5MftkkXE+KdcTGAvdVZjCz+JHUUAufNPGX27W7tDSOWxNmBO/VnqssWNrk8Sep1312iKBpWxvKmrJpI0xEpA8dfwhibe2CMZvehRHUAuqn281u1ajhTb9iG4fiZ30R/d/K/pCEzgG433I1TfuDmI0M07AzPc8l481S9n89/sSR1yBqFhTR7gwTWpRKL4RWH++qsWKCutZBrGpEdbupbu6Rr47+Swr8PnFQTO49U+hRc+LvZNuCUNjs8RZAjqTj0+zDY7spt+x0mUB0QymPorH4N80xC5CQGGkpxCXCTdHyqD4xQNgcnSerghZmxDQLVAhpSSTD9NetBktdtzWAWUTw+WRwi07ry9MepEbrSq+zfBYsdkSDbGPqfY0lVq+rTSU17VQ2D5r1CdZ6uj1CeV4NRSo8gCIKaIXf6Q0+GMlE4r/TBRLuRN+6uqbHU2LyPKGqOWGUOl6+mfhOVMbFQNolNg/OBvcy5HrJaSP7zaet/DqvHfpQgv8XhLGWcp801Lk8lk9nLiCHv8n8EY06DOeEJsO/yB6r/kSjU6NnT7QdTyIEKRR2YO+ezy2LYLYHwyo+FqgNn+yp2W/TajNHQvwhOol9EjLJVNXyBVrmOGPS6Q4ZefWuY0ZkD55/dRFFtxqeX4N3bacXiMrgioF8IbhuAb0Y85RukEk8dLLU5Ziwic+/v9j5/JuM14P9UqIXiUzXTbqCH/CyVauv85QajJojvmhhu3dbIDdmnxCrLANlsaWlR7hxQzqnwAHag5+GQYQnHbMr17+x5aveYqXBLlO+94thIloYJ+DDZYP0LVBvbLYiucula6MtCme/U/U0BA7mGWsvVJyYLjVNVnTFP5Vr7hNqPO+9CuyM8hV8VA0VXnTvSbIUdtcww0zRXGIKyZQ/I96Ksxj+MFzG+t8ivZcdGvvUhpvLN6M2YRXDc2bKFFil4/56ybM9vfeeleIJeWd3aU5EMfLtIGLu+1KtJOPnfd18IIUaKdrER8XjTOk9ik04dcGozsyI9Qn1awpsjafKzn3r+OlhjFobw39i1mveo1J5NFjoyNWDIQFhc6o0ZZk2lOsn1hz9/yUxjGyHxlFnCBFliEER10Kx/7aZRbSpkIlTTTHSF/DKT9KuLXa6x/deDoKGw5JD6XnLCi416UQRZ2zR0G7RksyCDOEE7aofPn9N/EYG/r5q4XEZjRvgf5We7ZCmM0YdrNAwgz2pxaLXZM+F+qZU+uN3mvaLNwXvuLuTNXhxlQCPdrCpGuZ/TXnOKxBghigHjAweBKYUqYIppxogGoBqskLcWcZVxU5wzC2u12VqV9yZHH2PnaqhzKyewEckzYBXP0gEjU/UkDHXkzO9l/ca7sQFkc5kb44OhWRNC1cFgogEHmuZReupeYBmDjqEt3EfiiOxnu9SdaL+QzUKtWFaKUh4/ZtNk1rpnDhnQ+AyqsnXfu+y0r8h7SLwTj2J9fcS1gghY7/HSHnbC22Mh8RAZGk3pJT434TbXBpHgWHsNSczJqQJISL9gyJMqC4AUIeS6bjb7KNGufDyalAarxKAzjOMSlcje462775RPlve59ZNjIjYwcLFgm1dO/LmGukNQnd+6rCCamvkO83lw5TGJp13w9vvKLOt18yVoZzSrdku4G0xzutoQo9yu2lhb30whmB2zcMTLNV7T8bYObT6WuGqvRK3LHH7sB1riihdVjfNTKyaFX8nELnyolvCx2J1T6x82Ph/ZmAJBlzkleA30ECydOvd7AFSSNzAqX8sbVVnXDOAywRJt0CdctNFxgrERcuSK8tVWvd1EdcUH3jqWFFmlZAIojtzPQbp2gJUOtAupXwjHxAXHIxWKIk4h8hEKqIu4+e0h7h8/a4sdgcAM8ra/BY3bx301ammjAmsiLO0eTNgkvbBhuthWh38blDN+/P0ABV45jPhode5XFSUYjKcaDoXOdTFJ8oTG+g6vWRkVtdqnSi2CMSEb4keEGWeDVgVBj76JEECgpJJLN0NyXyUan8Qr6I7SVeQPqCwFsVikcJwdNtwV2B8xMe2rzOxxTydowm4SKfoVznpBq6hIX8G1rQZFpoQxPyp37AWwegUxvjWtn6TTr4Khg2dxQ6V3omd7cBm88yOEbz29nqeB3bUaJYhnMVyTZmvMZSL3IQanDFpAwJiB1hGmZOmW3jqVAXeAuQwxX0eZ0yTdO458JdtBNZaU2n7XUOl60BGRxlR+l5S3FF6IyaAGpHoeq7PpfM0tuRfSdY/w/O04xirBnJhw1tqPqM26sYEEGqJxheRfixOFfvevNYeWob2G7hi6rpF8FU0U4F6A4d9c2oHccgQZqVvd6oMgQa3j9EdXoeLHSnWbGVRepXtmeXwa7zzf9B0A7YH2zlejknm7G1dzCKpCpfrbxeQL4/u2rV6tZbrVpPEkz9KDDPU7PRMaR84YK1b8VKUnYFmHpKPUWzrUkD1qk9tjVtRV5J3Vub1lO+pSsV3q0BtYnQdn4TnS7A1UcnSZOQrhGb5aLSCNbZL5CMaLY1ihrAMkqCTZBLHtJuQ0UniBPqXRkKnVsZJubXgvf4RMbJv2MFLJ23kauJFI9ywPbvOQzd2gshWF/YA5N7IVIdl2zhwpadHvfcEFOzg5QpGfINAvpJEaEa8G9wDVlJC6YUAQsPgiZ20B/gH70gmRzYlYGsxhbKcU8ZA65qaVk8Ic2bUWve+SeJ2VmpmRf5ZG/DGx07oow3eHKTjnD2Tg23iEwShXOZJnbq9AnDzdLCdyeduMF1dPDGjjzA6rk0ZwiTt4EDT+71zHBW7N3fM7hF9OgO82wBP4AR/kcajxfQilkd63FylX18IReK7WEM9zRxxj5jeZdDJcYhrK6MLf3SDE/JfPeYQeUECy/MwGAtJCVS+FjNmzo+f//vRVTPp6lNV/+MFz8umoF4NOb1IFLpFf1y0cgW7Kf2Q0ATSJhPigYraPHhY9BL/1TmSxoyOX6aMwcXZKAy+4NWSy8WHzQV8G7iMVs2yRNhqigOuZpgm10uw5L6E8YzGcJ53lrpIY+hHKPUcVYxUXaZjBaBnQghEGJFbm1kLEiN01AcOvztApiGDi/uf0Yjwk0MbZBr7tGPICm9Krt/ZL/wETVn+FXFuuR+PfcKjqLLlwb8GS0FCZPiQRbyNolRU8k/Y2fNJZ4lUXleYWomWmWQl4JDZ2t5hqPC3no+KXOopDe+Bk3qjgE/5UQwgKffKvgtMXMj/q4jSfh7vLRUSDXWyUoxXEsajzHBaM5z+EiZq2uCokZmvVID0ykug9CyTot6Vg+Gpu8tr7iPNv541dKRp1QqG64lLDH2g0QyzLtRaRXTRB+qXt5lKe9iuefrzDeg7J1SKqEeb9l6eqiSbcvN1sVgCudfHr03sI8+bbs4A9VdFcNaLP7q8llhWfBNiVxTQ6kfGwvdzIGS12Wl9+T6lIZYg+AMt7/dBUWuNT514pF9qLnSG7a+U5sy+dHT9yaZTUYiIpXEBraJmh2JbD5gNsHRnz7xlnhk9hPe7+5N7AlWD+pdWZp23qCrtT8WZuMg/YLwGu9+r59ztToPn0s/6p6F7BmLzfh5BUhgZks4jWEmr4LQQhse271z5cNNs6RSfqbOmlVsl7eQVnZUGncz1PDTiYpTSRgWJ4UDJIku3mcIWC5xhg8hk8zMY2Byyto3vWTDO9cTQ5lD0V69kk4DprLbjf8e+4ARdm+rfFHKZsmNSiXHU53NtwgSP6bQj0My/qJR3QPty58EutvT8SY+Zpdw81+fVI+qi+Hd7zWdQfI/wfoa7q1U2Cfw4XvkezGK6nau4ZgtvT9gQXPFZWAaecmZY98t4CmRbVxMub/LEIUJLTZN/CnQS65ZvuVKag/GdSZn+raJkH2RoA8+NejNCyImIM1T/0ElShr+leUcNVPeT3lOKUE40qoIqMUUIPZLuIYAw==', 'page_age': 'November 26, 2021', 'title': '25 of the Best Motivational Quotes from Scientists', 'type': 'web_search_result', 'url': 'https://hellobio.com/blog/25-of-the-best-motivational-quotes-from-scientists.html'}, {'encrypted_content': 'EsQaCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDOHC3yedQ/C0a0DTKRoMF8fqo1oSiZAObet/IjDNZj+Ml223dWD+NJSG2LIh+7ik0wZ8XMLgQ45fyJnReITOpSk2xJnRbn+K2/UCH+Yqxxnqw6pJ4l6VH6l3ref8kKjIZhk302Kqc82Eq4DEp6rq3V41sJRneScndbdkKYM/NaLyru9oRwFyg3fajHhEc70JvekDd92i0jVGAbViLuAIaq/BBTQCAf/Qn6M2NEMeBjYkAdciPPz/iWfcGuGpQaXtf0izbluOcVshwG5kqTce6d4cAlGcHpkhsDlxnv/lNr8t3FEExCNRDFZ5xhfhnJTbwwq6B/+1cFlcgf+2/VHUtTPo5qLEHNDcWOWGLvqolDpqXuG2pseMIGYPoqU092SrBShMEiFqxO1cbcbswmPcc7IYWkjyo8xIlndSefwRZOJYXTN5XxFiA2Sl8zaompvn6uTvpaQGMvAguS8pTmibgOPT+TNShgqNanHgDBjtQis9yDzod4S1RFBsftwDB1VFhFB5C3Dzd135m8VW4n2rJagX7cHr8gj5twStVYgKwV5HBQqabzjoJKEvcKQ3Gp4Qir8LuzvBsfxiydI9kY9+B1RF4aKUbXd2P3dwpVuJNb97BozfGDWXRQx3XrvKZUa53JaLPAsoPW4M9CrFnqCqj/zoTAv8psr3g1U4fw3hw3syTBz2awCVXGrBN0Dsqc1OSjhdxZWX1qnSuaeVGOgkymyrrqaZf7o6SDXD27J/u1NYRdxx5bFhVQ0Ib7+tAhCBTfx6/ZlghP+gHZ/xgUdxcOAIOQ5URF8nU4IblsdX+4Jp0+Xsm2cru/LmbaSGkMWsIdH1ff7MVAiE6ysbOjbMVs3qgV4iKFEggBN87V/SlGyhAW/YQajd9KWY4z7f7ETZG+DpHdgMmBuxfDfUezlGSHVRkQl6rOIwxONrD+7FqQufvYRUYxG7lq5rlpyoyaZg0Iz8EuEVu9eRLnyNQlFXQOv07GO0PVGOjMfQ59YGxzRrpynp9xy73CAmTZUVeYuvIRg7wCYA0Ro0F/Qd0g5TlG579LVIt3e6eBGUEoakeTprHE8DWW2ioIlc/sTzvU6BiNvtbz3u6YG/BYvoWgaZSBLeUwJVkZXG4voisWcnY84ZHC3h7gjdSmqILyd0J8xay3/xUTUQi0hwxBnkUb89AmUa4nkR6yOyY67AnYylehuKPH8AWNUn2FVBki07jjtQ8Om6mXdCXW1L2NLraE/fNbsnWtFJVPNWC4mxBduWRE6j2HwA7fFqN0JckxRX6nqxD/JYJlpX3L7Ta1hpJnUy1yhWHFCvTvibYvBvoMug93N+7XxXiwcs8tu053fnkTtnaGj0lDH29eklqwUt249+dvXGNPjqmgCtr6O9TLJaCIW1pUoF6i2F6KTpDbduyuho8TZsM2AprZXHhsbt4AuRF+KRlpdTo6r84MVDO7AhHqF3VpNe+oEUn0KW7Q72AqB2U4jp6GqoeKpTXWe7IVIOCTcG+dTPImD/+Sm/ZLdKBJZbNMIWgpZoBrK2QO1CvDzmluA2K3qV5WF7ezhV9sQxN+G9SUUm6tN4qAA6J5IsS+ZrgiYABhKYjUDhG2xq0fz+jzT2cbChov8u3Vu+ngxkKKxrH+MNcKcj8Av2urpULYGVHpql7cpGoSFJy4kk5QXA/PaJBw+GoF9FzZpVfUPSLCfCLls7e6G6pD/G3e5+Ps0jSdwKAY2JX6ne/DahUde5guz5+z1oxVatnaq260EPtPx2Ia30GYY+vVXLX1qACjzSn8fJVJTm29yU96YCvd6TOz/ANKvexCtBtyNmft5YOVGXJkTLOmuHxaqzf9eom75eGpiU3zKkbnsZlKC1w3EbxmwYSpZaqeKmN0vsKx7IhQNy8l6y09PsyMNZ/eQljO4AzwTIAehuoyJPyR9wbsHLRfm47wE4QxMjdBCOY8fcB6zLC305kbjYGTVnVnj4+qE4BCVtOF2+zZO4KUOO19gI5nWRNB2N4mV0w87vGGEjUVFC7FT5Wje5EGrNt6eyjF5Z4Zucq+njYI+suT9QILZkfiPMHtMzoxHxQCseVJAyq/5jEIb3p31HkJEHA5zGuTHyG6PJJZhXN3DF2jmWPLjfEEN3sAq6YnfQjN9kdqRBDDGACWhuNldaHbwZpXJyg1vb1j/NDr0o/hlJxkfjXNv4mkV4Ip822cLoXqKUk62RLPRD2qFPUb6Jw0KUlUMfSFkYFU9Yci1/8Lgzjij+Kr86W80W1k3e3gr9jix88WdneC5VdZ8q3KZ3EsjvG68D+VpY7yQQrd5ewCCmi2yAT3ptUqSPDMpNIOHxC8PeMteDm/tRuCLBUdG7YDbd21bDnFzuWmz92uOke9TQlfw3/UnuFhdEYazxYertsxiRVXvbTe7KEEs2bHAF7GSYylLdnJ7CFWmLYG/HqdNaC1U9KzG7MknAFKN9xiI/tBZ9TofBpvdmngEAD8UZfuu5VaSxjs1DuhJWimNfcEoNwI3mQP9bW6dsDuVA5ITUWO5ION/KVdMYEEYiyRiApo70kQOkOeXyjWUEIUscS8eroCP2aDZmLC3tNZm+NGGTrO25kjSkk/pnDDPSFgU/MHpG4puDGynxisMev6Hu7IODoHg4KcGGA0aEY7Vag00oW7U4YMGNZZ06csIapSKRT6ZxwzDTXdEnFoARKI6NHffnYVyUrEv1FPDwP/HSTrN3Ttb+3iZUYhu5RlW/ynj2mTx8QXMy48YLjv2p1vKplfFK93LbJSA7ph5UI2A7Y+OEC3zSu35RDbnxuFOpyTUuAUaZwZytj4ZauwDXa9h6FCs46XEYIvrgybGp0SQHlPOIis0YvsKbwjGh8wzjcXgExTnH5MK6zfi767Y/lTm6TWOpZnbamLFYcDFRcIEFrBPhPu+NxDtrvSu/m5N3LAf6QEicpQwbHIHP+d/EUzfq3ah4KF1UcQgfjvsokFm0xTqQEo2sNgBo5IHRAI/gH2RLizWWN6fC9J/mdH/1AsUdJKDddweipVwLljnxQNwKa8gop+rfzLHnpnxMGbJxl4a7dBzGuL3Nu5lMPESGkrz7sc3dFTGvJ+XTV1DqQBavAgZG97cCBWGJq4M6EhhZ5z159JwOK1Y7L1HF5IsFmadhtwBjTnUVKnuGIGQ75vImyjjIi4u/k5BFpUbR8PjO6is/plCYATRapwt/v/689ydXEiGHwyboDKgZ/Ch5Uc9VI5cdhce9UG/a7t22ZC/SLN3QcuVbzRjxVDQomcVLwc6+s2IXEG3YsHet0Or3UpxcPpMsx/so9BEpx5WZmFoHrQFp/goeWoVKx2BFnkGEw+lhmfZKtcusmSHYy9t7hJ0nJ+p8aN+fbjeLKzQknVwxaI63j2WgRi7WovYEJ1Bb+Q94iUffumkTuGHu6NSf76mLwURqLjhTcpJTIJJPw2ETtf6rSHtDjxbHvPrmJo1nb0+/UrtcHwK2RjbKi78oezuORWAwDnSyQxqqR7y5IcKAa1v78S6R7ldOfyGHaFBHcGcQFesPYMqcRL/kIzpwOu3XGnG/uacQFSWUSM40JTKvkLQUC6Fq8gPh/zCTDdbreDf5JVxyOlzU9ZPW2XA3hVveoo8F0whNYZY5DUlUL2/cX4ndX6RkTf3+pAXCVPoWStAQRgYzjrdAc4ybwLn8qxFdiIYLqEMvWL929WZ9/6H1ODeypU7E5uRfOt+EbBMOn0lIfwVBLtjzpTeYVNXCBRkP2Mj3I1LOEWztrA3gakrQpxn8mR6o7FxRVLEpwcy2ziHcQSbO1zwPmGkWABAMudl6zCNvvD3WkA4Oko5Exj/1PBMNvTsaEeTGtibKv06rViARVk7WaPiuQpPuRVZeGX+zhb7en25M7UtZTJroF/2MlYF9kk4g5SOWgKuILYgkZUCUQNyYb3DGnnr+Fcf9j/fWUDQGHItXhHMTP18nQGDBBK/OOjIHOLhre4S7byZrjWbTtCSyg8j97EZU4Npy3GCf21Pgu2Blt+2N4Rjcxd/G6KjCNm57wx6lLLX/FsZa4xSR+BdeQTzzHzxNHSupUdbhZrdHYO6liqj+sftpUcyZVETabnWhco0Pvhv0GmLtEuCDELxRTFV4nTuwZf13A1KlSpcGoDyvDnt9xFkdXGY9g7RWa9mGU8eeO+7pbSFU//Nx3LxbXc2DvEAR1boZSyKiXUAld07vS15mpbfwQE4duOXnWo+6zoQyXXyZyc1G/Co2qsBaChdt3xXYG94czIAJmJIdNaKsWYjMYsV6w2bmapCCgc2hKv8E3+AtZ5eajrduliZQRJXbrz1IHq28yiIyfWF23MOIFE/Rqa2CTe8GuHuy4xI60aYCH/xl7M/pfCl8sAASifQHUe3lfSJjxSCCzX6jyNyda2FMMAec8RtDPcMR3+f3WwexdHqvK04nQDfliVahl3NTVEswX1p1H2dI/nQ5k8NabEgsZFxS7jY+bQPad8nYBv2vOCIAJL8CZQyQGAM=', 'page_age': 'January 20, 2025', 'title': '20 Otter Quotes about the Energetic Slender Mammals (2025)', 'type': 'web_search_result', 'url': 'https://www.trvst.world/biodiversity/otter-quotes/'}, {'encrypted_content': 'EooLCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDA6ZbzkgTbNyDY3eiBoMMijLp7iuZfrB30v4IjC4cIAN499ld3hjNEwAasz559MqAjA/mpph7uk3nGgTSLyDnCjqaWc4+twrGMSEiMwqjQotRcATF59egj0IUXd0M3C5EyRh2cSpjFh5sevm/MYCvurFBrELuKe6tUltQIzSf7Vl1SrRXn6go+uszPqFM3CgL/cWQoa4X2KIxzAeOweFSbIBYQKmmY2cI/qvRk9e3mR+/QOb99bZmozeb8G4506M0oQMqi928N6uCHpm2l6rIEoN7sO3aLG8jm4sc+JkAe5MJp2bphIwTNap0Szt69f0a3snt8d4qNoAFZ/bQS4soz6EFUCnU9F2Wm5EonEL/J0hH+j6yAlhYDhy3OU9Z9S172G7KcGaCpE/NjvvEcAPY8dllz+AyW6UcvrKJB/C+fGAzH+7NQ+HIfGdMi9LkIE48x7ASKB+GsSeVMRap5G6mBHNpWr8cG6GFP4HeaGhlaXLpnodjX9lZO/SBV4+45QKbMFHaVOnLb+hIG1ILIH9K3tFikh08OkkBhmeC19NDDpgXjAl0wpbgTuMFTtH2H2SLW4DwEZAv6paD2HQC9omwFBE5Se7ELBfcBfzTAyH6WrXfWTewim9hYoyV952gfXQgRod4BdIBke5kbpptzUsnvI9kd33Ng0v+EMd99XspqD0figd83MJSUOyq9t1nfkYploaKxAfZvhO05OZyOTmqaeGShktVLP5scODyU6ydSWfPk0ikYQ1FHDleyloBR5eL9OnxuUvV3dHHblyP+QNQmYrcMbk3DbbrYgOQmO0SCjJBoyN/PNIlh+h1fDBv0M/iZZ346ebvkawiEmcL0cm1JO8VKs95GCOxMPFZq3Qj9+kPagv3x/LoHk1yZxzWHvB+/esLVQ83w0pGKKRDtxuNpORBiOGJM4zxP2JPiTwuQk++tApKdnTOlDkGdLkz8g5BndHtUqCmPWY+5PeApOIY/7tJGVqLSLfJajYFruUI1aSRAHEUNkSc1Ujq3bEqEVCU50pPFtSik979OqkiKPtAEuK92dg8O9iTswgHullaG+YIkqrhDtpg63zVoDZt322L9SZBVkoamjNhKhgKm4A79goAGOgEiTqLqlyaP53c/zAxhTelxYvFFWt+wz2M8tCn8KR/0nPr2yMFJKcAeZGn8xtYmXWD+oml1t7klGGCOgy7EFN7tq0JkavojGWR1Tzgu3XvKXfBork/jwiH9G4JVtvJpaCl+Nn7+sQTgfkfX26Fu4WmC4INqThEmawEugrq3CDw7mssngdSGrLOqO13TYkOD7h9fbplCIv93kOTo8TKUgRu2EGteLPJeFT7xFAJC6ZjNkCqWFN4ij1dpmsnuROAAY7MogV9z47A3EG4ays8cnHTdgimdUwX2fB+pThN6y+bVL92ekoUet81WmF6BqQxqCwC+syQoXbjuZU4ZnQrTrbjqSWG/pNoQ8ntb59JN2LF9kXx/Xz4QEV3g0+JKZu3O00kv450kuEZhQF9328k8pU43GtMjR32vqo9NAbnNkEys4WzGJp+BQEMW1527nkU4XEFwXxSTtCfVmSxXDoYzb6YyevMDgtup0VauWGBZ5KIt4/7nf2QslGIsCRiuwOgTtG20y9uzn/1IApDJte/Csdz2X2zIo3soGfp4Kx0NaG383nGRXd6avyKx5UhmzP1MgO0QNCdwTFOlvP6prcsSU7uq/YeZhX7FE6vvgVqnAflW5Pz0APsxLtBV1ZTinnGt/BTfNTSm+VoC6uXNFSm/RIGL4YU/3RiN4D2vO79pg5i7TSpxF1M1FI3kCOssDM9N27pWIJ56aNJNEYAw==', 'page_age': None, 'title': 'Quotes by Scientists - BrainyQuote', 'type': 'web_search_result', 'url': 'https://www.brainyquote.com/profession/quotes-by-scientists'}, {'encrypted_content': 'Ev4aCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDEw6Xz1ECcpCyc+uLxoMzVDpfTuiWLEcXtX8IjCDad6vZ3NN61tqK1GPCVRyLuuqz3zIc0AbrsSlmiOT/RI2GSG43cQv5pIzzz3Qd5gqgRrmyc6phhP14ile5LQl/hTnD0fiELR8k8MAoJ+QFKuY+pCwSltP1TnhDlvk93oim4ftc0J4ZEXP9web9FF7JCej5zyHcDRmXvTK0f/QZxDAYwTKRvM4GgdgC8+Qrxyfom3phEJ0P1LFjOBPnRp0FEqCTttVCTIdXXQR2SsZctYXRXRqHlZZA9F7b4+x2PIrkMlkg9zBdG0axZoJPNZ5eb0JvkOolrr4if0gnNzQ9FO8sJGMOHjvrGSy7my6OytsVIEWw9g5BrmmeeCfFNLIg7gBtoY7n3b8/sW7ssgBGjSIaolNPPlQX3obmw7/In9ZLuVKpiF04DDZmCpijQ7Rk8MDqg2FOKxO3RIqUmNnp07fb6HkPu3YfFYBNTf+dp6Jwy3zsgQB+qIYJvni1MKy+hER16uhLj7AcOfx05/+J2qfhP0qlegaR3vksoUa/+yqG+nASkxtbwqGzJZ5nie/Hc4lA+OQ3RglsSkhHr5wctRcpFtDqULORhKUMn41F/7Q08Q4Ru1p3iTElUSyd2bfBYJaFdfEyuOn73SEFA9uhpLXMk9RfbzQxbce7r6jJ13TBvqIG23dsxmWvP+7qVfbRDaHytsV82A1kR1G31vzJ2/ImVIxyCkcjTrAXZiFN4LUwpXiem2BNjtipBHQN8CWJRtfc1ua4JncUKGDbUpQCafrEu4K6ztcfLp0/nF4A+apufFi7Nlos4wbdVAImeVYRWICo6SkxAaFgMeiN+z8mgnvGF+QBGrj/u2chWTcfHLhWLxJJEhN2Ma30r3QyJYWDUELBHjrHNfddnLeTumnmf+IwW5CarZsJjx8R36c9/yO6Dyx9WcxTee9cP2ioyG3GrAQc8BhBpOpZyK9gbefh49EinnSREV/GYjTqsXidONz9h3a8V1dHS9CmSF8V+nkQNQBeGytxKLkkR1YZzfwJHhWDw+MMxTPKgSc5cJOuACSBxwJwMiRTX1fC8/FWvkMlpksa+eS3Cq7IDeqN6EcZsWBMhXw70esXuHt73rsrhDtd7KvUCFrNX2xna1dTEiwcvDJPrIn+3j0u+kSM8fa2dIq1d6W8zQIPGdb8yHuzWPMkURDPTmRyz2qxELKcOu6FKTMnLl+9AjTDxm4R/8FNM/VfW1yHrzB0mqZX25Oty9xjJ+kAJyilS93O7s+zJ0PjPA5hkFSkHNXaQVc/iEpoR1I8Q4A0suAO+Az1rRakt5hHugkx+P82k1E3cHCg1IxPP7ZgLFOJE8yVbRct4Z39asjfRFxtdwfqzO4oIdM8/4tf1EXttEIGUV20mOSi1+o189FVJPJGg388/wKx3qzm5Ta37yBexvcp4iyoGapmrvWzhxKplBQ09+ZDtyRPEhEf/8iLyaUPoVIxD7qESx/2mB+CLOCCt4cW+GB5/sLFPtPpOLC0an+3uHKXxvtS5VgDUQ0b8riJkFieJ/J2KGuGLTh/FSrkbGkM7mipAR17cxtMQK4qvOah2llhfDEijogu2VCkVD3kr4PJCXp27e5l6AePIqmv+5hZHKf3WSzxVThXgVmV+AZJQMNT/WSNbipJbEMo32Cgdco+0dgBqot6RbN/ryzSkSh+RbzlLK3Ai/TZ2jwDkQ+15rISN5UQ5XsZixWE9hqzsOSYFtqp80jysj99+ef4hs3Ie0T6+7GLZctVNXvgZOOorpYu+dJbkGqyKhpg+l8khnjSedRubwReIhiIavXtbF6bQFghnAq1UGcP3NDl9iGhYF6VTea5YeEkxKHA9zrbi8LvLnTLp+b44jzwP5pDjeP7EkUQaJET36Giz9HttzPzE7Rb1Q3ehOwQZWrLxy2R7IEFnrcrvVC+xjEVRPXbszgwi5pqcmTpNhaskZlK4KiIYlZolzniQEtvvvJi5PdomgeRk3lXo8DmmSVayZnpuhyojO/ka/xbUK3E3AJzG3SCwABn3FKI0avce8/Pa42N/FjJ5r9wdWszVxxz975s6lnsVAbOsn7ahIfrF+WLun7oqdgtAxAX82Anc1gGUBiFmDxCyBJoI89nWy4THbNCLkGNHOeU2DOr4hwfUVxHPkya3i8A90tEH5Y7y2Vz6naa+DaO5rc6HdpcP1vL0HR51qbV7TwpohD/rnLYJqOAKJvg6FEx6rqdsK10LdOWV3ZK4Fq2sToN6DYX3hPsi/Av9NkhtdtcL0HJeJ12o3GMevbq/PalTf42zritQW77XFvThYna1m2lUxteyP0yoDQnUAsNSG5ddnWDQ8plwveoV22IPce2q3oYOjmO1IVhSkxhOWZ8pU0+4liPxqz8B5v1dFsmR5crUkbgtH45r1aGXntXxyXZy62Nu9P9hMhmQ3dQbJOwgTVclKnya1y7JffQ9JSXDcfxypn6iQUoJ0XmoCE8Kl3iL7+Hf/LxLJyma7VOqYjW8Vtf8HLsPVBikE7BvYeI1KgWEthVAfj3EqobO6sXCoYJ5K+0lSHu+eE3VOrbBRD4w3J3ZWjsNeIpRNBG+HF9ItuQVyRdfGIhHUJ3r7z9obHdXfMzIWAzijijIllufHOxDo2oTiX/jKwmziYtWB/ToBfBVUzLYaVrggYy1liWz7Yr5HKkxX6KDJ4PoQA9c/9esHOTwUCPsMbBkK0pGpXgsj0/eG1NKoeNjnJzl/K4g/mrdYdBVwTfurDmNspQvrZT939GoxFnud45iNnoCzxxCSj58CfjFA6JZdsf6KBcXWa/9xOFEE4Ar/oWAXGegMcqj1dm6zuJ49wb3v0+QHq2DVAnD9Afj1Md3ZUyudDFfYxbTwteAfrUi6n9PzWFemOnyXjUUkhSQc2cMZxNI5sLNoTNCw4i++MEm1oOvLeW8adhbZnpgo8FTaEjJ7DBHIAtjWhmcuQPDGWzcvZ1VioH8VuzCMqILamKqw4YvVdcwYHcfigaa/mCTAyyRMg+j4rTyh4XG5K96pqildwZYmywHwLL7jo7RjnKOG66zXmNU6BVQXLkKpRV1qPePY4JTomwQ27wVOXdaw0C2xwmXBDPnbobOAoe4GYh63reEbTgYoKP0EDqUsT1yJLbzapE2iNiZMeBwOe6LwGaMQsEx13yLGWzYhlyhfv4cTRmFYb3DKt8SB/mWn34cR0G33XFL2CIaNMBK49v1grvMp7XaJfArImqyUBAuoeJ+WCHmz2FXFf5mQpmEsiIJaasV1T4yfqpRD6+jiAkgrJbIkGPWKj4aCR6PhYo8ZMN+TdLGUOI5yRxQNTSQLegXrYKsSFuH5f9OShYafgC9GoDYMMwHDyhJ0tEPycGcFPyq0PZEOVcAKVheHABtS1xmvLo8Xuy7EzviOU5M+a2x8/MUu4YjQv5sFIhrmqysGYRYFpV1rAf/JEaqity3Zdw/47xgolZkU9F8qnkcSiO9UqDwKT9JflIzQESGScRmu4kNrmC1Rj4h2ZLm3Aq6ll1ow21f8SdIQNY0a0CKUjW22tw7aoIEeqymXJ+fKZObf259RKCectT7cqxd2acfh/BJ9TZklxre/9oGOLUxYh3L4RTwGZNAgWbhhvrUvuBG8n7/40PoPWeqxfcBiM/f/WquzNP/Ptc6oODs8pUmSr+4tJCZ2gZbFHWoAyteIVTvr2MGappvGzDySrT1jhvs/S50k3KESwuk8mUqx1EHi+Oy4yRLN4d/UtIvI3iqSJGVn1nhAP4K5lRCiapR9muWWIBnZYqHO1waMyGjNj9KVoA3zXfQCjbieifWTsARD+Mvi91/4WpXwmyGzSXVC5VR94W+2Jtpyce9z+vJDUvM6ZSDHSjUM3coVQpq+1Xz08O3E9bv5fjTc/cZX9XxMGtGwyiWIcLwoYLENMTV0eKw2tlgXFaUyq/BtZPmRXTiILUbj/LrgJJUNKFCCitvdFjDRUZ3P4Y6dZH7rOQ/iDOIrhEdCKemYuF4dgd93bT92A9UKqzTu7ib0dnk0CQ2bDT53LHvyXsaj3aSQjDvNE0O4IHjw0snKeMp546Cp7PTlW9oVD1I10wQwQ4Gj/LKIi+OeooNUnz/ugS/HMDVoJ0HSE502Um1ywuL1/MXRDwHKbaanYZbruoZ5jOTmRFv4JR0CMr2HmSa+0F5ZB6xGDKMjF/+rlSQxVU0GCM66Lrl3ZERx8SG70trc2nHk/80GPCoF0dvPbZ01SxVW7kAZB/vWG3KFjOjZwtF7QFY7B56wiIdyjSYR7EtosAVo2ZGfNpQIhWnyWkqKKWQrwDAZ+Re/ybb2qgQ03iRn9c2NPCE8BaXqo6+GhI0Ab9H1QGogL5sSTpgwadLXi7J/CqYvuV+YqZldUYoiJARDU+gX6H1iSlB8MuzH9bfpxmJEWfYZ4DgoXcRgZR/bR09dQQJ0VHYpYc0pmHsd8xwyl0bfXRp14Av1DW7TAx/WEyB1QNdneiqB5U05H7nIlmo/aEdVeTaM5eT/TGuV55DUrcqzFZ6l/2I/h+i2vMcEwkrFH4JivIGsMwLECExgD', 'page_age': 'September 22, 2017', 'title': '30 Brilliant Scientist Quotes', 'type': 'web_search_result', 'url': 'https://www.famousscientists.org/30-brilliant-scientist-quotes/'}, {'encrypted_content': 'ErMZCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDEzN0RMc8VwIRG6m3xoMubAa+bh7o5cAEkxQIjAsGpnJlzDpB9cfhR0mIUk0P1iCr0BZbDdL5IV1HJuSttooR6GsSxXDsKTDUC44N0MqthgkIC9GrnQjoVD6O+NcAON+vij5CwLiXMNC6zX2SN10kw1P65YgYsinx5apNUb/cJGaSaHi3ThITcxPMBIjweKfcJNlsRbCxojE0QxyS9yD5pQUYGin+ScwMaS6BZ/ZFu7FGtq2ZyVevoxLZGKBIrXARESYbMzxP1yEjPHFSOyZyvVcsLls3jYPhLgpT6pZNvoK0nTDMUAm9pOgwcF75Zcwz0J10lCa+EaEPUnbrgPhfUfROIvf8faGRWlAf4obuNcqBiqM8ysA6+86bO8wPEpZCukt0b1zjkXsVLEMq8F/l4EY82plxE9VNXj66G/nEpdfUkAgRqOPEPvJuQf8XMT9E2WpOS5R935Gc5VJMmEAe/ciiMCNtYOWWxNz3lBl4NEbJBU60XQTntkzLbFzLLeiXWOlE7ANd9ezQ8Ucj5R0j3meruPHmJfJdT6dzLnfZBaLX84ujOYtzsZ3LchkfFQ0kDueJFJBkcw3+GwFuRhid5nTXsOx1Pui5+jxXkxYnV5YxBsTzu3g2xLriV377VQa5c5UbvnAU73xeUkkVK/IYORjU5fkxtofBSnMoHDYEGkbd9YeUfVdxuJ4FbXZpCZOgOGZI3swuamQ0Gd/f9fZmpaaOJgadBssNNXD7c/VBjfxHeKlgnBOWHmFDtzCTlUoH9SD2iU+E5/tWMMu/se3i0NYFWA1aAx2VJtMWJF2I7oysamS9UlP53LdV1lH23bJf4jyC9h7bWVY5T4jp787VGN74OdIIahFYYqCkbMNgVg0h5mEFOjlYqozyqufhLq5IlFhFh3An11/kwtUgIkmeTtnzuGcbTxx4gozKqBwk4I5ygKAvLSMyKHhZAv+PVuf2xw3T5WLb7uL25g+F8kZOYwWxSUMJr3D8hkvOtV+mUCsVR3Xm0yXxR5BvW/NKA6wY24bf7OLDULnvPiGcGPGRnepLsHzcxvriRYg00jcM8LVR935Omfor4KAXKP3iMfiSw/eDSmOo6MINwDocLfpUz1i4SOBj26E+Z2rGBNR0C6iZLeQT6LtFw16G5+Wjnd9BCzA6Z+MZwPXXGhsZfNFsFm7cA5pLNjYptNXgHLbDwX87b16SqGiyEGssltZic+AYuX66qI8/c0sE6/5+A6I8RIaYX3HYePvbQDH0MEbiVIZKo7strhAOozx9SONQKW1TJOT/6EklSJtXI6Jh0TgbSTc/Np4mgliAQXEjeoxF4Fv6Xj71bRJOVh/7v94UbFr8QxY5NezPVBY8ee1JX9OpLIQCCZb1u/weCSN4kTaotq202Iskg/257UCEmCwLOeNZpRFiBdZjUwOur8+EvTZb6MMT9RwWe8rdMpcT0x1dZuSNdvd2Jt+B6ZCkR9+Sm28jMvt0TlivqcC96haEdC72j2WvhkGkfZ/S0aGLL4XRbp85mAfzIEHxZdGbDnc7xgzc0iz9BWOsmZk3CdOP15fjKmm4j534CiePAvwvMt1jmWoDA3m8OzC5cPfLl6QLNd+YnaiNxADhTmtq0Xxs+C7avuu/kHGF/a/lAlVZIsnGhna8M6GNfVnBNzk1z/FFnQCMdRb47g50HRGhlCe18k5W8/8SyLmfKUc0deg68JVLskl5UW8NdW5zJHFAWeUtGfRISTkw1m4K6cWxSTQm6gHR3L6WotX7wQC4jJNaFgDY3immlmcIKsW0ohXRXSeYE2d2hH7T2aTmzvusjVXscSYyiV04+ODdYUSmAWQ58U744lzgeB1IJ5XrT1iigd/EDtxD61A79u5jcEi1dIb38w7QbvJ1Pefv0Oi9OMetgik/db5JBifhpQgPEU1DlLEVswITPW8DAWG18ohVg6WVvFetS/yI61pt1rrWUZ4++eVOuaXo9bOzoqFZJOEGOBiSWqbxUaNVGkwzE4/HDoWF3R9kNEBExyhkO2cmPzuyxZPJBAUnQu9RafC+rY5ze8jBuJUWeFqUoWItq0E4z3lr8ceh51/Y7xQpZM5Z/wSFHTnvpJaZNlzhkTZ5HE20YYa5hbCX6mFHq1J3LoQahNmjENtr23CMShGNIXe4Sp/urGkUR+J8nJkbCdJeu60QQcXel143edeis+C5lGftvpQHx0Vd0i1EVI6IV+Bgpj8g8HfQCxy15t3IJO8tJRdirMpuF0tRkCXMQhpZQewsyLc/KSXnBBx2Sa8mGdi4m07EspIpIdQKmlp4ZMNLk1eR6u00IoXT2b7f9+oGLpPGadA/FXe/wiU3nuD2SmY63Vw+ZhYideqMXYolIVNDko11F8pnJZ4bcnWaqOj046Ng9Qc4ahUkqaWkfe17NhdvX9bDixu1Cqh2MlN2ZPoKiieeOH1659v9681QDkJVT0v0i3qyVVnk+69OnU4JZrgSYZfUAtpiPZ1CDkJYRiYxUlExQhQUaCy7ITau91v18twXOYD3h/Kv/IRV9Ldr4YwaqjJCO5O+Wydkl4nSeFPAyuCSev0Gwbu8fx/3w0EF7addS/P/+8G5ciRNS9+1T8P05+6PQfA7DxFZDhAkKFoDpefYwECC1SzSGKRMAEGC8KrbBbQwk0B5/6dLiBFf6u77L9iHArfZIHPdf8j+4l+xj5Q5+1Y0Wj+FO8xq0GjZxXO6vso40V3zXYk0CcCsJpqvd08np8rCIFWWj5L40r+u8BLGzm5nZSRI3kbLMTGuf1VGKZ9h8SSFSblcZq+K4mUqre1Airwt/xdbhVhS/LyHo2kQrre978lTvoq9uTAYUngqxpJVLOwKp2LK8rj6FbkTMbhSpecfO5X7MMP5MaOf9648aN2SFaV6TaG5qcmsy+vBqWtSV4xYfJR9Xrcvp98th1ykN//NGbh4hjtpKnc30KH4zCkvD70c+AKukQoPJ33t5uBb6TbfepEAOE7QmMn/qLMguW7H4zH70JHPCzszfjTh7lLnrOWlf8w2wq8qMzC8yxE/X80gcJe9VZdlzPnoGvEtihzBeP9abt8V+WA5rYUfJukt3eWYF6IwCcGjAgxGTkrAbKwmyNbVmfZOpVtpecVGUJMKF+IUPgrVoTYGhGsYL8qaad9lye7uUE38dWv47Wmi35BqDJ5+YxZGfDk0tpgpknPc/mVuTemgYIDOJq+SEE3BLMzURdRv5soS2TMhKhBOxAJ3B9+xKtxCqioJ6YyrmpL61aB4wDIGtca3Xu692YDNajidJ+IhkhXnSDpBeW5hS8PvC+IYWscGxC3wE5WcxsipLJZ0UgOekHE2eUWO8k/m9CIDODT+Q3MS4r1DsWljkqExNZelAP6X6B/rfL952PWWW7JOcmYG/FnjplDyTUDmooFiTVNlCtjbmLfTrnA5rrybO8pt9udpfEhuY5UYUvPPtCfhmCD9e3JIrCW9X4ZiAteacpIIqWjiIGG/TOy6KwqAaPsgApgn5RjMccZPvq+6Sb3dnmqOHegWOSIQ5XFiCLOx7vI6JTJRglikaPeZto8M2j9RYlQd5WvGxL+mqy1DeLmShiWnrZqyyqypfAxOTVRTCfjOn7PdKXmVBhxudLK/OKU/S9jNPMbemIpujuvIiR84ZOeXdvFXL1zqi+je11riklEphYWklYPqE3gKX39TpI8/UZMJkDRXof6CVp5R6HotX8RcMDNN3eEX4jRbOCVn1tL0cHfMBK9py6Cgu2bhsxTOSSiozdjVM/3SueaF662AWpAXiz/NIQzmvSFcnCzbxo5t2PeG9xppmvtPkcC0tpHmYcRqAqaaqLrvQIQQBaJJAJPPrvLC8dnK4nfOx9jxav8z31Tsy0PYBySttkqJiGVKj57bJMrzBt5j+FaCnoyiGWp0CZ5zoi4MNpbGh+XjbuGarajexJva2TEXoBV6FO4paF+pJ7iNDgxuiKley166nyHSeaw7QuevDwKhxovoZsIczot8W5v23J1/P1eIe8cdxHT+4xX5hY+wkfl2fq8Wkot4WXkBKQHWHDsr6HOu/+cObOLtzulo3VXOKgXVreBe/xnzxSg2quekE4NVy0fblezlwpXJ0GyXV/t7C3r+kV0pAqGsE/StvYgpE9zwVsGW0SbWkOY0Ona66K/1/ruU8rXMdy8CRRxxx6hiI0S7lTTqW+Gn28JuM3+y7/vqsYvok6kCWWb8oDqIyjA0vTmDOEn1VEme3/Erx4fDu4TbhacD05q5XcfwLs16tUcZzBM++mP6p2/AiJANWrREgV0vAOrnE8YAw==', 'page_age': 'July 25, 2021', 'title': 'Science quotations - Paul Strange - Research at Kent', 'type': 'web_search_result', 'url': 'https://research.kent.ac.uk/paulstrange/science-quotations/'}, {'encrypted_content': 'EtUPCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDEvEWEHKMqrILBxT0RoMQ6XFErP82qdVt5+PIjBO/lL+aHvqgd3liF7AuuFuvWfYUf8Q3mcCRro8ZEdr5q+wJfe6ydDqIYi4w/jAw1Uq2A7jLOKCcg8WSa6EMF/z3CioVneisROajLPxd5TtakAwUtHUi3a2ShG4p2VcJaQMNG6DIzXAJHboXsafE7FuMRbl5B7QuWq1kaW3GBRRQG4gAVx2CimWSwu8JH9qlBP8HkMPupftGQJiyprHUiVWBGx+U70wYqBimNKWBgeV/fuWdHDAdRvAd6WXsLJ5o9uUAqc6x2KEZ9Ys/7VGKbS6Xw20lcn6awqnrzZD7vI2Cx6OCWX+lzwADbwY5zqSwN99Ht0fcMo5l/5ZQwXbiZOLHZLSUdWpIowitUEWlZvsqVt2IMl+X8r/sTfkPwaaRByEtNPgHMMPJ44AmfiW83dh/+PyI+XXVJiXzLt2jbhWRs5cH1lf+tnLytWSGDs4IfWwW6ybgaDHfNNpum4P+MreKu+de5vfESHvoYhVG+bpQ8NUUUxsB4Ej7WE1LUQOf6Lbp069iavk/EZUizFR5T7I2fT4MBt2m6zods8dMlU3YNh16wxFl4wzkdzRLxyIM0c+n2XPIS4xefhcq5i9cddnjfDOg+2uctBDJfP+qZQMttTfrvXAOTfqIUbHhpz35fxZTg0Ibm8JWW3cD/LcbFe3Wz12LzlceVCGypZRlTLbldHT64E/VAVgwepax1bX6ZD9m+rnR5VLIotgmndhIVLxWW3+W8H3GQN31T2wyIwcBz/D9OGd+v8OB8HV2ASYBLECxOR8wwLUYAb59NUsbRZqV76Qau5M+f+cy4iZiXhaD6yUu7C+lD8MfOMLTnGvCm1VnEqL7PcL/NJYdvlvuUhKWIuV44eTNC/TFF054w4CX/To8pusxdL3moJeVOcq17/4YgEwR4n/NCHMebvwMUpRc5vtfBMHTtsp63os97uSK/SMBy/0p2I1tfhyqqzyHZFOH3RkNtqZK9d2ZbgZjRvyGzPIoiFZPjcuzBY1ZzXWn9pjPZqZjTjmijSZW1eLeMENLZM76vLPS6i+zU1+547EFRcf21jqMgaIIw8siqFQhlUMKIGGeZCkI+5CYXwGLCuGSIF3TTWQqh6hFKBMyx/eg34tO1F0ChkAh8F8/NcJf7cmB1Ez2RNj08J2pSUZ2qz1ykWXQQUYeumDJAm8GUY9lk5psRfEJ2R0DPKU1vNMD9I5RTvf5fcZJRHI7zGJNBS4nTKFI2dWcajERUWaaIxROd5w+/dNBx3s4uLzDUY4UGFCezFVvhpVmN3KLihvm6rnCCQsYtas9tO/3TkOz77fTxNK3Q5Z48j+BFjG1KtxIMMSQE/zRYqSr7cA0Vaxt4uZtyH5DRlLUj2puoqIeeeX7Mh7Fmfb3KHa6VoqmstcPpdCqA6mGrRiY7rgagAc8NaQ8xn0L4ziDtElpJvqWW1uH+sQfBYyb6PJJ9aDGvKmBZQMZCitcsZoPkygBmTCWwn2RMaqw6b1+Hciy7CZDUQEi9kXG0BWeaLJXywkKrzQlD/8AgCYydHEci08OT9qoeAtr1mP+HEPY7+C2YiWA92c3CT4mJeybBvnDgzQLnbCQYHByayqkBl2+XvyvLMG7fbkhPmAyejx2XmpOlt9OPhGxJ3NbD/oVsUCLhaGL7mWcQY98X2C3hj4XhUWC5olz9FuHnxnE12+jp2k5Xl5Zj7J268jmozAejGPXTgb4lJYzeyryCnvzYOE46Fkuh4VRbmWSczPyetmjRQGAm2laMsbSppmYmP6A+jrvDQIxUHTcb4D1k1UG0zmC/3iM2Gsk1pqwX9itD4QaoPZ52gtFHngxRcTCA1iwJ1IlP44gabcrCmxdZRjq0irAn8pslfzGYMLcWJ4gfxW7V/Cc/tEYamKLWn+sYEqXhmtiiIsT/vkZKqfQl7Tw379EjwDlWg9fi3oN3V++NnOrvUGiHsp6ndfSdfuTYrbS9swm2/alS92gnujHscM4G88Tvrl95KPRlpvmm1LdIjNNDZ987eEVB6sbo25cG7BReLPfnchv+Ijv8IdeILGKqpxR9zB5e4AtqA/r6y80VAqw5kzwkJ5QzWK6wKtTPUxO25pflOW+4zWp6y6gag2lKyPPJpwqWcndKVcM4i9Z7OihSqnT8QSv0vXS0Cnm3+WY20Rfy91B5QQCTl+IMq7qFDd7JVNXp9A3np9bwsuuxnxsyJ3QYExUQkIUaKoacYUSV/GPUi0hZFp2hXNFVg0EK0d2ir2DgnARrr1jUIki9RUsebtdkmdIYbvgigJ5K61axc/eEgSUO7f289sOOY3yMOHrw/Wb15O9LBCu+L+H0gOyxf1xoLc04ETgTSU4KIvn72XZr+H7kfNz3ZmYOQB80vdOEoCULZTdUSVmjvdy7nm7lfBeWU1tEMEPQSg2BUWRwtjWRx/c+kzaGHOB/yavFXVjidO4DbRa0/PGxCmyvgKcCnHT/S+YxkwR5+mrRzykXxeilXwUi6XMPDNdSEBk5PWtqz3YV371rdT6s276keqKHyjT77T+X/GsA7jU4uI/o9YVvkirq9S+xEoIFodo0oTHNE6WApd+U31sn3M+yh/LyKtzhgD', 'page_age': None, 'title': 'Top 50 OTTERS quotes and saying', 'type': 'web_search_result', 'url': 'https://www.inspiringquotes.us/topic/1733-otters'}, {'encrypted_content': 'Er0ZCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDJ3MZsjAd5Jj8/W47RoM7cUMpONgXSIc5VdVIjCoj7J4YqLVrZ1cEo+VL18SqwEe33b/zW13sAfqG063TY8Ec0HhLIg3hxjMbA+IWW0qwBhiCbLp23fjsNQetcKf3tUBiRaZFNSKXIdQ2XTFY9fBaoidfne4Ngryk5+1vJC2oLOqPieKig7Cq5R/HaW6VIIOCQeTKE4T3mUsZM+Aofg7kpK3FS33xCk7ZS/J4X5VlUcZiJZsiDpuiUmx9U3L4WD191MuJiVDx7Wfe3Rzloz80/05/ddF/BbrAM5TKq6I+mceZsNhN8x8rdxV7BtX0E3ZcuI063TJiseolbecQXrxTYagGo/s3A75E/2wSNo1ThvM+jsi79QogZ2jlCi4zy2MFUnhaOyG/+6nZmm3gIEsyzQ6tUloyFd10qQ9Pfnzg+nma6I9mOZU+Y7UPBk55kBTShc/m68LzIW+1+k06HYvl4TKWJ7FuyghSlVzl4LFhVzydHXyaW6I+rBwTTRpPHkqUd9uD6RTEEoUSgylA40Gq8MSIv1D2SbmZ4ShtyCA0skQa6MyzYIbn31Kxk9AtD9lm69uBDdvZ1rCHPxHhO1Ss98L+ku+1c2XcbIv1yaebOuee5EnxRwK2dEsYT7uWqzZ7Ny8ImPS6scNYzT7VuzagSz0D4/bqqKvmRH81ZnrPvUw6grakzrUWHrOFeskbx34I8dtdSlHK1HZUjIOILgGaCSVOB43RfbjgPi+Sd1vYZ2KVM+j/WRiovIv1NjcvFf7cRjB7/DtNxivgNl/8KnnL0C078MmaJUOtoJtQr5DbwtuPXUcGz46Dv6auiOzPnZuYYchq1VAs9HuCLRJNRo2zfgxGRSf/5jayjVoSqREYYlUmYcN7QYYYJXHvPRYCJgrGWdvz1N314UpdTIQBb+dywbSTUyTzFRXgcT7Z2tg17y416lfADJAyBm0OBMtfvUalVB7iqVtNeoAr2TkFfio4HYvTXHSFscIO7PuLhGNPu1h5w57yX8Oz4t5t3Lh4KCZr0Bvfu2K6g+ExOd2BWdTjTk6feODSZ8vcU9Zi6hIvbPW3/4aFkRZuKZEkEEfH176s9QqdVK93jMbGl2mE+utkqf8dnJ4LwmFQAQggVfPWnGqrsILqFh7pJ7OuFKJOyWhwoSSnfSvvD0oODOE4KLw7rlCS9sty5T0kRO7lq18xhCgcnXCyWqc+OKqG9nRLhKg1N+7jLLpKxTnvQ4OfFq4EXlIcIuFXdYdEgxlmR6I5EuXb0IanIxeiD+nLA44q7kFIXrM7uMHqydWn7eZDC8dwNC/gZNx6dHyPVpmz7NAgNXh7qO5OKuN1/wZndiU/WXK5YqpXNb3A1z5ZXN20mCqxH+6k9GRnN55Co82BapPJOMecjxOU2wEKMKzc/YemQsBOpO5UX7g8FE+YcJ4sSl10r6EJcd1WecCXe4dmsZIWi7v5XnMyKrwKkbrXyx+csi4X1EO6nncjDKfH74rCcj2xaNZrDGgcvhsY78DdivIdZ1pj4gryGMF2gl5eJfzm7KvocZfopNHSrzynNFSwLnmptWfg37dw2yR2BJThcpDmYctadIB/nagUbRV1oUu2nMJG6KTJtiXgFk/Tay1jrAR8SYUAm8VXrl8cstOL4e3P1k7PiUQC5aqKn1iHMnHCEKVw4j6iXSL0aTQdnIYL1/eRUYldpLCocGMTotr+eL/hIgl+9j9Tm23LcUGXWkw1xjFJhCLSMWLQqOAnr2OfnjQBfXRUgSX5Iieju/ewgURLVUOq8PKQrgKNQitDPN+QirgkI8Sn3U+IqTcLN9jaGHEe2DpSudzbiMJVOSgwcglmJQ8IDb+8+4uaN96tV1fi2LLqkPmFnwrp/kyM+oQ7fQOwYNMsq1/iITglEOhMEP4uNCXs8DVKFbq8vsEXY1M4BPwPlIHa4OZLLby2gBe4KQnNV8f4Z2RnLNrME5UnVLsFcbUD18/AqvH6jPlFIq8YdW6yD+l9YmnVBEBq14IAAfk2GlgeqcrZk8y+WWy9oqcWwM+cxoX/r93GbHc4CAa1ztZHjBu2POYecicqfbcNeG13OXzj9R1SQN2tCiQYByyK4q6Y681HNmR7JuqaEGokUzyeZsPIatRQLBxXmHNlT+BqnG5HOr15/SwfPFv0L5QwDUkWIrMW+HLjxXjQ/uHDSjsQ75Q6EYQuJNycqK5nq+ek1C0OsQmP7FkhuQWchxj75T3URQ+7ifz40ktc+PeR8fLTAjsdnN/pLqbEY8M4jx/iBCd9c+L1KdkcpKqj7TzY/oZKL6zGLzlc8GfrP70t7LxCKrdikMYfcusWwDl3LAoqDqSth2L9S6Zsxqs2RXgxF7kXWKC6sXYcom2RqvgHWPBwPjDdHw5ahZOHw6y0IX7hwFYtA7sI7g+yAqx3n+eSb7A2LhhNrd7aluiRV8xDXp5EErgwN+3DhcfGEJrUNoGboTCl2nVXzDEsjQLNL1fETE0igOCgjGJPwdwMlFPULr0MH/nr087qdhGiJ2KpxRHphdtKlpAkRlfbWIZhmxHVlDIJJtbH2UQKNsQ7OswFTR14p6ZHHhdqy9o2CyKS+Ap5xv9IgZT2n6891BXZmtnjZz6NoWzYxmE4E9rtItypqL29jpTuXdXtgdoRIDcZ78NceOiY48QORRUtxxXX3k9aaiV+z50snWcU/lZHEVeOwX8+yM63e2sxBj1pWHbo8xxNQgY4yKnjaa0FonFNi6l3ybqlQAKbLK8YkGOW/rWwaKkfCNo+D8+an7xwOHN3HOKPndATBoiawhILxtf+l41+TLIkpubHDuc0Kam5F7fwFpiaPJlwO+mTHNolljr9VGlhvXdlUk4LIuT9xTgsIMz84ORtXhs7nGEHvHibpY8YjEm2AlM1ZRzSZHqGbCMOD+fGbU5GeHZhantLcQYjQj3DmesiXhcecRk4nsGYKp+RnqaQ6IP7y4IylSK47PmpZeSGLJ4eIAvnzt/2FqXJ8nNo0fNOhqGyr2KXisroxnwVU5euxXKOm8ZvtcP3cdx+V4HYry/VeqhelQIDAHxcLD7oJxKXMkYSOdlAkKYOH+zARS5KXz/Ba6Ld8YPHNp3uP/pmjPXP9C1hJuDhOCEIzODFxpi1tuaLzWqKqwlLLNUd2Wil807uK8n410r398to3YIf7g8fxjPLZzHfTPgqyf6I8Kw/pvxfGWeKcBo8qulupji96FGyzAUmHKEYuH219nVjjcv7RkS822B+0p4nBmKuXYiAUEl2kPQPgMAoVAoQXJpQS932eJJA69syOC/Dv4d1MgDDDrax/I75lOuMUnBxv8qe++26U19vpBsagkvDRrTLEeFAagd/nx63n0nUl0ot99FSjgPb2IaBi7WzlZGrSI87eZhhybWAz0G87QbqpVMsLx/ZWTDCxn0lJsWstAwDOqv7TP1/UDGSgkVIBGHlxJQPMsDlhZ15cNuSXnD/dY63SeFMd6evN21waZX1RrMgWeEYxo8KQP0DjreqS1f0LHMaWuKBj3kTOp5DL0C/Klv9Y55tGmHJ64QeqCJuQw6bT/3y6NiyyPUZOtRNZU/Gy+uFIUBG/5VuE0SEe94Oohz+e8t5NJQaOI/PTLahtnMbX7AhpDlEtwbATMXdyhfTAqBzVpaNuuJn1vVVb0aSooSDA5AEsSRxS8+H2mV+g9BAxsi6EdXmcZVHYGk+7if4xIk46UCzXxhf8YVefBq8fOUIUKnF/8f46kF4Eg0uKEQVAamunQp+7UnuFlOBGtROhCJiw1d9QHNOrWpxApvpFCaZpxxeR5oIg4V8n8xfwboS7VqNZybtF1bKDfaFKnTzpp78upazkoOvqjFgOKT9urtjm2YZ/qu+HPbS1/R80wTnNuHnQzN4nx284fgZUQwnfqmqaYJ1zfP819s/ilOK9XlcxUb73fRzj2Ewm7dADwd3kalcxeiOBcOJqjOpjXBaaJrvugwvTQ/GvqKijYumcbWWJl4RONIyWIR/qLjesB+5SHdLh+iEeP+Wf68hHQ4ky/u1J22MdAXKpEwqSErZEQwdsU5iKk7AwQRpGE2Jubgn2TdP5n+jnzECLQjjCqnDGxPZWG8b84onINnhlVTC286Y1OjnpiDLLC4mY8Lh0RmC3/LX3z1xlmzy5atBtqm73KeUBgS7uE6nDoaWl/IDvEE+gfGa5fJkrnX9eNKnWZnM7gCIVLZkWjrq9a9aQtwK3JZ3S4wed5QebVtt6Sqi3AbUWJJleL1ZFhTYTcqgJjfGMvo2MY+7izb4jJiMKmZH9RbuI8LxcT2NYbERrHyx6ilGAM=', 'page_age': 'June 22, 2016', 'title': '30 Brilliant Insights and Quotes From Famous Scientists -', 'type': 'web_search_result', 'url': 'https://launchyourgenius.com/2016/06/22/30-insights-famous-scientists/'}, {'encrypted_content': 'Ep0RCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDJtulRdLvp65AQdqghoMwySfI+d2AXGKD86sIjCzOm4HZB9lfG2x48MSkXnassy6pHAr3vw9WPvcfR0UrqtIiG4q3i6VP88R7rBU2CoqoBDm80i584gmnXR1IuOKKOvY0POIPxrHgQxexcsfxjRJri+rxcoltL3bKF6T+TCceSmn+iwYF8//hoAaHGhb52xPo11USGPE/FGKrIGWsYO5SD4Cv4X7x6XrOG9Ykjh5psvXs5Z0W4SyJVAhuWDj/RrhaOBG6egBbuO2iELFKr14PwNm29tur6oTCLb14fxtJ5cOehDxOd7cnW4LjHZP6utCT9hLfFZb96RhCTOMW9AZF36vjkNj1mz21ufDQqBJ6kP72vIKxo40VoX5yDBoTYNnFCOKRb9NP+7RQOOGN13RByIWB6n9Plor9pWlUXJkOaNP+CHBOXk3Gf15OjRt6BWuxr32TdcaFTu0E3L9zJrX1TpVDHqTcMA1zBL5Vw+MwjDafEH6uw/3aE3Mk0Q8owOF+5huUZzMONa2HBsREVtFEqqngbJmtG3WDufXk4rx4vshNWi8aY0GEga6XWf6NSYhaN6HME+NgGkYRvzZshL5mHp1Nfed5Lb5Fkifp/UEhLliQgTdAprdOj3y4bC/+z5KNbPshijqJgK2PZIOY5ZTAoxmWX/PEnAOg7yXKjDsHRfc72KLhFgeUgtEZxTbuOez0sjQW8221TJj/yEJ1uVKRPmhxSTuGNU2+10i76gGNryJMT+GtcKXce0vKygsaIdVLMDHUvbnLmQg5boTeTfZ6K5HN3Q8FqJOSJF2x+7n5Jwv18SCLUXwproJtZmzeIMRB5b5Kn/GnmFpFTr7qclqErhAL8Lq2SUtmQ2Gy3qN2jXmdq23vWUxip2E5xB+FxqEHO3wBnyb6tIryLcT/uzKEc5ePKCQcQ4qQwyTq+3TUkdhgFhOxqMqmaACWUsHUlJeSz2Yi08+DjZYwppYT54B3spiOcmcCVHJj/APl2wr0WAQVhKyhJqLfLRRIOuKzZoaH94Y6jz6/oISuyqTThEQX4iDVzWJuEhRH+npdjZqJz62cUlVLsIjdflpbwtFamB84foEqHdCwnXyZ/Qvl+owEZFGJAYLBfTGgFSY3PBivURfvp0KzuwTaXYsJmnqWtrzOPD8WIrx6+qhBjVjsb8A/u2YNH7sj8r6jfekEfQyUFpqy9l+tu/f2FvD6fu4ekj6WKdRSCElL9L9mQNtMoUK6xHbeyoFIaBK0ymjoNm6cE+SX9+RYL/sXJtBywO3X6Y/W12Kcdo+s8c5FXM6LMIcBw/Pgx5lAJzd7ohLlDV9+lyqDk2H+TXUFFX8adCqioljGbBJTE4IJwvPkgElrpPjnd9nmHmBRpbOf6SQMFe7pI+p9m2jHFscmRBLBW4GhMyNdBQpdj7vRRwgORkwxo/RKhaeg1E580OEttlhvJbsepsZ4bij3gtiASKgA9iJ36N9g0CJehx9K0fb/G/YxVtxG+P7r2Mp/GjCnSL+tfvssxog4ERSyZMxCBL1umxPW//PgsC1ssjP6X65GbZ+skaplCI3a8fIwSutSuWJf2H4oAZRy60R9qyS6Vi/8oOiT9dA+nH34kwUuocIzSsFmOoYiiXSqsg5np7SRDMkcieSrboDOvgDynE9rZMKclJQQXkrHbOL+CeSxfhUhgh6MnDU0cRy+M5T/xbMhXM86E/eNyeenTaoWxkq8iMLaRiSbDKJjThvXLhz0VH0242VhlzmdQUZSHiQw5GDwYOp2il9yeFj7qo96T3qc3PfMkldVwK17vpfB6y2qwDS4TVuqZd+Myw8piXizIsZcpUXQZwHyj43x8PrPJPRHmaI0KzUGGLC5T2EaSIAKV7rc9PCpW85iFaerPoHrk0eK1fDuHZs/O5ph1Q8nxFqzYHFeSsC/vLyqAHPgV+tw82U56aAmbeCZzPj8MGEEYG5qjIs6Z6WHPedbXKU5cMvizS44CblHGF9yAbaUoWeiHXUkRTGqOMzcKP6cL5m01dulvmtX4gow09eat1NaU3Pvqop3IaCUvL2ysGhE074FKn3tA3pfktmVT2gfC4MMV6r2zX0PC+cI8sxtQq77rmWt7RHrgGYd+s4nvUJc2fYRYkhsYgWjsB064pkLBKZwWK7qUHPyHz5W27Nip9k7ORSjdOfbtz9xSZHeb6fa1JkzhSrXr3i3uR1SLG3CjNK96iUyJFYJtI9eOZEbvuuTg3NLMYdRTVwHeEDVOXp2qspTOJxwkc9s7aRiphCfM9ng3qhloGYl7xTx3/TQd83repI8zzagzLDzcHg+7fmpzm6Xj3C0e4PJcbFGJL/m3HL9E+axr9Sy39mUBxrztYJxdrsBydnuVHDujxbowd3okrleP5t7r1CFHHfRvYI/GMzEzCuD0cLNcoC4giYzXPGAzayg0gWIPnu3WHLIMgVul8SLLCLTds0b84BM41iPYNiZY7YyIQA2/YQC/M/QsGF70KqClTribjbS3i16otTIvqRlCxUyo6CZpnl12HQuonWCzMB6BKpxESlSvk11qXrnI/YVGK5hBcXsOvolCtK2g9wh1tndWmU0kVMCECLGffFmyDj3zIy+q/1O6xytL6dyNC8/4P/SJBPwlquCC9e9X34Sb2wOHnLv3Ecp+KgU5bi4O+WqEAcuz2kjXK1WmVGXDFLfXNoS5duR8RLw79OSX0fJBnOgZShiDPLOlrsBNIcUB/WbTfCO8kUsispvJNvrUnpZoObooLVS1g+1kk9RkUdmOmFh1o2h2sjbVEpMaAh8E0Afq9OLVXTrrHmbFMmaEmD+GvEqc9eMnff0S5RdPSYlCUATy2rV4ZIChyL2omABoemQ7RyMHdR3268Uo1oow10IhLuKhA2mIoPGAM=', 'page_age': None, 'title': 'Science Quotes (11903 quotes)', 'type': 'web_search_result', 'url': 'https://www.goodreads.com/quotes/tag/science'}, {'encrypted_content': 'EukeCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDNr/qs5sX0JcCFK0rhoM2oQbDE9+93CKRhgaIjBuVU1fpUIFdQLZIEBiSKvJrabPusaZjnUibiiUMAP8Hakyiw+bwykTrdXsJFpkSqgq7B2rh2Ui0JJnhXDJvyRrLQx1AXC6XKi1Hjy+eyb3GLgqRObvRTt19bJprBgHOfVomPqE+2EaD05+j6q4JGVX1w7oecv22G6m6tlfXPBlv0YHZemWOjP46eE25DbNOWh89I6bdR5MVoaOp3IcELFy7fZwfY+D1Nak+PRrBEABwEkMQ1blg7eG+F8WknOlk2GNALkDu0chBIQU4mmBEGmyxSbjVD6MH6lqITv1IAz04kJkjqGscc7202LGyCYQqKzPYnpSzJJNdIrJ/GpRgjJlL0KECKlLxyMJmmWqwHkUcn09x9trTrjGjVdmPunKkHAR9GhNuSEApFJ5YRVilUk1vx/bUI5NxyoK3z8qqrIOzszmkbD8vH0spaKh7BdPTtKyoorr19nHGUmxD4mVECEtegbwfF4NrBceMbVB6/S39HLp0dY+EdjssFnHjK8ByZJWW1/mfu8UrS9DtoJ2AZge9hYviBVU1JbspspDkzP6V4Y8Jq3qyvFBb++Mizwdquk+ZoT1j1+OeaX1NIPWj8pnWHdij6rla7ghs0l8eubYB0ZqYxYleOREuQcIjkUyXFEvd5DMzqmBzDfRTsGJik5cSmixnd6y1gTFybjIm3JilZVqsZNfGadFLCBFg/2cN4CvZ9Ol3TahiC73iByYUQlDC5jo/Jsxz5P9Rube+LdZTOgjIaiiA+0YG5CHzbVsQOaHogD1iOEJrqvloToT9nRyfX4b/GAipB9FMjOk4dbjbC5UbGmpnEx0KYvaZlaMk5epXDai7OPK9t+3p/I2sYSPHyRJ2YR2zv2uTUcnxstCbFfo5QZwT+QCS2IFGdrDnW3Qq1npCEWki6T4lIX+QwPa1PSj+LSRng/GYyyRX6AqFCIz0BuhYoV7MN95FXdhUB5umDHYu1nJsxxCtI5avVQ2S5y1JzgZfX9+LPZRDkA0kcr7i7Vu7ySXW6IP35Aka4qvXscu8EnD3hJr0Qea1BeGFe1fh6+pNtGQ1z6ECFdaKl1HSstH7Dv2zKVYFooKI+CLU2VRKZklMel78sHtJJLkVzrpVJHQKisBqExARUQeUfyOK4dgHCeHLsNiTO03lvTaKqcwVRoiNADyt5BS+g/XNXcB1HDin9xTv62v1l6u0IZJkyNjEW54bAi7fLLjrURHGeMZvHpKpkwsqo4zIBn70hUNWdBkXkJjsNqmMFtsmbHGnm/X9MLcLh2My/QMw3wrEaXoE6DuQDRVJ3uZbIMY8XuUcbh0bEiG99SIMp2grOLN84Ts+NzZ2aJxyu9jhA2cxmI3cYmd29qJZA/Q2fX8z02hWFef2FNetU+p/E+M6bk4Aut6/3gptvYOe08UPBYn/JsKX/4J7zG8EXq/GLUvfb6wTLDvu6N9S0XYuaAN2T7n0/udVGq2Uw0dx1yUiRsSjMgacoj+OsHeVJM50B1ze4r/Vpu6fKcULY6oedsfoPvz0Z7jK+xBc2GdjSYMkFE6XJjKkHa4uRikd8erRBR5og4GjkotsygEIaRT0R6fIQzgektgPm58ePclRhThmLdx4tHA3ZY31VSdA0cwuUCojY/8+BOgybGMZpfz6DmvMdbnzK5ZXJ95NnBAKiQfO+AHn83xQfo54snj3QWWm4k3X7rfFo5D7JVK1jCix5O3LLFVV1UrE+dALesGUaeNQ2t0gUkNiFDt7fuLDlJCWVcZahPxGt4LoQqMcYyd5QJPpiVMQq80NnI59W4BvZA/GnQTqyinz/ccVgFr191CFC9JprLMKNSb0dI+2swR92VbGXE5ke9vU9zGOzhPNu4L9VvHHRGW8VKEJFiCcVeRnnDw1fgoeR+ERW4R+wOaMKPtWMEwQK0FhvnAkma40F5I26ELK86p9L40ErZV4uNNP/P6gLPrxu1k8Oo9QMi88Ki1NYxmxpGI6tkBE7vjtbpL759cg5hPy8yBMdI+KR9lM7Yk9/im13EcbgEkPtGCguCxjEJzFEFCnSHP6WSMMra+2BVpu9FssHVZAw7mB5frPSpZXupmoh1orhpMOBRg/VapAQ5fBRTDNJaDPX1KFSwcDWhkTxQo3wqXuWyD6stbhbO3QE3nTeapxyxzHLqTdPfYo3JgbDtmMGfGYBQRBz5IxFhXbkxErYhlqmoqjF/a8JwMaqBxbt6W8n4rS1mbcOuQ5CRskzF0KVdI5B7at0Tvs6izOlOeSfTF7Ttg/8lBqBfFBzP3Bbch11xo+sbFIQhSAl8GbXOwwR9op8FiR7TSUIIr/Sxv9gHUMI31XgarpdsBwVcaHzCmyXi+/m3m2ZxLLByyZoWDwKsmSDYXFEyoG/LF15WyvqsSjWSXPmx5vz/s9gEoe8KZSlWwg/IYoiTSJ3NOpV0hUMf9c0GQDkJ4jYAoi5p6RfvwGq5XufAFDsXjWWCfYwTWok/QOGZpGek8H1DfNKEym34+0ZhJuaJ7XUWU4KUHf3bDdFUvC1z5OMru/VFcGajJr1rHsVhRycs/VRIswS+AEyAB07C8D1p2hEDCytvLw9Lvo3xp73HN7ROnjDe/KP8YJjD3izwvYglki3tFyyy3jhemHwC43dcw5Kfil18j8FndWZqNKiqD7EESo2EPNTTxRBBowdKjkB0Vi+8Gc51r7OsvH/bI2PO80BPN4PY6oWrxFfrPtMkSB2Bf3muEcD89B7LXWPP30QTOYq2xTwf3dQsqgS9ShUe65Bj0Ue07tTiaMKB0toDSk90aaA9itboDED8lySn+zz5e7n7N/D/3Hwjmbc+4kw5inDdm/tP/xCgdy/vaugaLtaIM6YEPKQUR6nXk6HB3Zk7q+BJg757S/Xwx3t90HiyTr9pqYMXuAcwAxBUrRqnH2bhZjCGJZzxz6E6CAEvbBXhAKwN8+3hL3KjuGSbohzHbSYunMDlnZb1vb/+5qbEnRsea2o/2Sal0wp8IQH/YTvp73BLmDrG9m6rnkNkkakqveYOI1UNQj6uAU5FRU2pudUMrvLPgoy7HY+k7LwxIpHYAKEANka65rd1eC+4HqdVaMZdBU7XRrX1oTntYZmMZg58+IDLMkST51/DtIkkNZXEnqEQLvd+GSv2I/bxFPvYnnyyjNJKQceWGYRQaW8JpNK2exCfjGXItAUsGzrX+AXBvbcj/nPLJ9Xc0lih3oyZ5WIX2sUoEB4wmcTQ58fj9DvNZU3cgPRCsHEP4zpbnD2k/F09naJDvlsrxxY863CX9jeOG+GNMi9g2lLo/kbYK+/pM06o06HDp7F+CHGUWzbFzwte9Cg3V1ug7AjlZI3zaIpgpJZNW5eLb421zodSnwgn3TLyPwAXnVL+zIfWU7C3HuVb0K5HJs2/xBNWe9PxZizehIiOIn9Ud9HlaW1GXnXhmFtYLKExH/oj736BfkVVd7O+CMMAfYSFQx9HAa4JThJnVAHFon5dB10dmw9BqiaNdydK+Jlq9xMWS6RIz148CChl7aFwy7i7YbRszEyuHvx9cBOwQi50X8qshni3YXrchZ8xMsOikajBfq502520aYGa+U3lAeu/ndH8+PQUcAKsouxjN9xEFNBxHkmbbAqGStGkS8JyyWRFV5Mq8Li5tfEAvFBy0ZdU912S2ayUKo85EeOxaIDufVanp6jaIptLW3hq92YC6BHfmtWXT6KGOg9zhWholpycz+0x+7Pab7rc1A9XlbLkoeB/wOTR0qhFCxaXfd17s3qPKTGmV4jThaIWmEwft65a7oRpc0ewy0Hl0oDtoTCn6jDTd/mqw9NaM+/NoasiGjWeXzp05kWe98HmW/UmotnQ7cD0hX/cRvFL4rBEnu7yyauvldn6FQtlmGAUsHk3FGLiwAMdkNS03cFu2ebUu6tHTYzXJpj3j31tZfZIfD+LfYP9e/GU7L3uxeKfcxrUdK970gWvZ11EUecSCZwXpxnhkSfCvINTQXeBxQmkr1NMTa18fwi748dXZlvaHzhH02Vhfrz1CKSNN2R9RHXydE1abUqKzsS5GNyfD3tB8BJmYwEBo01F47pxnyRi849hBMc3Rl6QTReRRJR5NF7gGpkcChdrTfsoIG2k36EuNjv95fRyJFbJc+nPp6VAvG8glnL8K76tx/zmTjwxLmR9r4xhAZKAZbe2k0mNtjUb1GS8ExXUFUHAR7snqcAxYUqEIbQ7V+/YqC1ousTs5b+eEzGjk2U76lur5R27fSo6AyoYwtA4nVUKdL2jT8QPbQxvWMuKBzr04l1JEuWv0XoaAPw70CtABPzYj8qYbctEEmkxMklkORRJzFNJiZjrdqsqjpE/ZXl2lK7GcBUfLxknmr67icbNC2WUqPLqhNyA3LB3sP39HMNitjMP8+nmz10xeDyUTBxdFha4yItVEfQTZdX5K7sG2VQ/iTFnZMGk1PvTqTECw4WTXgo1V+HfkuCGifCAzi2lTXrnohgqbTudGZsdbBP6WXYMiQ5QPRdBIQugM5mC029mBkUn/8cTcIyN3EizV5/bxzgDrb0YMPZA2t2HJphQ55IdzIDl4qHDSrqyiT18pC+wtQlY4uPzsJoLcwgBlVbbWk3J2EJla+XdZ0pJb6w/TxkdiCDiH0SN85KC7gGxbCSrtcxVMtFvY4aLmDgx3pQ6r2Lzr5il3t1MldbLnvMgfVe1jdUEJvmUVsl0TfdmIsBq/ko3BPYd7RUHuDeYqwlsL7vxSc+2cAn2lUVqowJf0wLQZHV7pEZyJUFcjbZ8LCe5NuqVsnmnpihsr70D/uUTF/mvos6vGFXlg9irX+3eC32Am6axmrOFQs6cCgFNx8hHStWCN7meeMqfybameV9Zdh+n5jqDU6/4I/7go1ao1mi7xNEmckKHQW7tBLt9z2cwFFs5G7xbg4lG75pSbm0YqjmE9HEV67LzIH1qjOOXsUcDXvZ0FLSShszCumGvbZ8/ntt220vTeca/3GYZCtOiRLxz0H6hnnbZ3hzNugGe/vKpn76ES+j0IHI3swt2Lk5TCOOWcx/zA1iX16fnF/Lt2CxQokKVSRoYHHJGoW5GAY124u+LukVCuUBPjnSxtqhWllU7Dz7EB5cMjbsR7tJkWHZTPqvnz+VhUJaThN23tZLZgh85Cor92wifGCXHyEYJLGgKrTdRW+9v0GHVzg8jiMDERGAM=', 'page_age': 'August 14, 2024', 'title': '20 Best Science Quotes From Researchers, Authors, and Leaders - HubPages', 'type': 'web_search_result', 'url': 'https://discover.hubpages.com/literature/20-Best-Quotes-about-Science-from-Famous-People-and-More'}], 'tool_use_id': 'srvtoolu_013GLmxiDRgqR9hda6yHPEsk', 'type': 'web_search_tool_result'}, {'citations': None, 'text': 'Let me search more specifically for scientific research quotes about otters.', 'type': 'text'}, {'id': 'srvtoolu_01X36BnBixLaQdE3H8E8hr25', 'input': {'query': 'marine biologist otter research quotes scientists'}, 'name': 'web_search', 'type': 'server_tool_use'}, {'content': [{'encrypted_content': 'EsINCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDL9/rC2RWqwfAI20cxoMzASPelkdHgyKsya+IjC8UjxplXsZZ/26Dn5+nRmwvvjcO33q3ZoGsV3BmwhHV1FyP8EnFfYS1CdoY/6fI/gqxQzsJ1qSeWUdfZL35Q9GCFMSG2//mikYizR8g/tf3RwvjFp3VI8rAyxYw7/TZNxmhyoBVSiNP3jLjuB9D3uR/z1kd1qaCE4SBtYeeD8S9KEGRiw0O3NyMEFUruwaGcQFfFHEx25G3kL20X+0aT+r1PITk+geXy00R8F2MUXKdrEr8uMNkpVMzwtt8H/OruNRApeapmT365uaDdusZogAcwAUxrKfjQnLXbUxbW4DVzOCu8qSco2+FlZOInsxMLnSfdQdTTYwe0ap+J1PEZbiOmeYS4CH8TAIXIhIIQh8eX3l5w1Y83EQ/xQCPJWd8qVPK5vRHghhJqq4lZiSST8DM4kp7HxQt86bI7tORVHDjJeP9VjOLyWEwdbDyL6q9Y+ogI85pl+CHvzYNJEQrAYCGkcJNo1Ve43fhxsCaiFhnA++sKpHFFfrjUyfAxMUbPwEnbqyrmsUU85nAwwYKzV1yj1XyWPOuf7W39mmRdv1dyDZYSI0k+wwBbKVoTtwTPnOjD9Gxun7H9QEN9qncGXcBTExgt9J65WpZw9enlCIfSOIQVaRW21HLGlC4qmiLCEGiDkl+FW8YLPWrlimXKhfZaa4LqptdDN9ZlN5jzWyB3KIyrccAvzEsQs0CX2wldutrc1SPnBxdBBprzkfYH71d+RG/f7H+8alw2Dyc96Q2EmEKDt0/0ssY17sFWO6w00gF00Gyg7/QnjF64la6rzqzUmSuriLpZi5A1xB6yCrHlV/SmBqIfTJPdsQANE4reIwvyYOFRvCF+Wi3Mt2cMcZUzuPfsSmeTKV+sSTUASFzpCj+4w9pbPQKGG+c3pvGwRbg2bMsazLMlozl394UsN7DVcp6GO9BSG83zuZf7jcJGBkt4cE5KfufvP4BzrhfveyLGqhswiAzky6GNN45lwzA1hq+7Izybs3n9ej2AOY3gyoT4jo4ej3e1AnO0MPXGesqGzIXUf2tH454vdABmLB0vrMQ/XU4An07TmRbhreQkc39nAqDs3GgwG7vkFxTclewjmTaAGiA2eoifwrFviGuG5XVTxEZDhE57WTMc0LjDXtf9x2yTzgWm7viOfUQRz0DfU9X8CCCBSmSGwKYxlmyE/K27GWE3Fse6QygMsJ+lTGD1aupLsBqh7XoV2xiRx3+5aH4/rFPZpRvs98mJPH9xWA66faZVvAaCEwXsHMn1agOG5WqSnObMnThR0hTQdI8yAA2deeCAzxnc8/Exih5V56ZQmDqIfnK+GDOPSA5nqaFNqICPuJTXVBV3UhFVkFXACqxlEX7buOHzslJm99gW+DuKeacikI7jkj1B9OtY4RAzRKi5/KeQQdHhJzK3MC6P1HAKoEXVJNEIERTutjkjarVcpMTQgpZYRvJVx9DuyXKozQyBaYvmBJNJBIl9KFXJzPAr7IJAzQk2EaiuOD4qiAFM3vVZAbdJw8b76L+XXxb1BHha6ATiVp5VX7cZjd8IYk0OejPNqVKh3pgQc1wiMQIS0MVk+JEwOGYJc3WjtOR6W+RGAtLcP/6uIoGyCIzE3IstAhBVtY1JqBlTMkOGL19kxP5W/83iOVRppk0rcfe+/atBS6PTNPGhVxUCjUW8XG/ZSNr1Vb6LRaA2FcMxpeZkWmNTnzIMGqkLfIiSyxP/Nw17fz3QUNWfX3BMg8JTP6LIXjZ1nNTXVhvkmpNrsGx3+NrV+flYalMdJNxS4N38npO/10Vm8wvDe5JyHRqHHdKqSLik1ysM4ofWAMQoKKcYaMbPeO9+uO3Kr5+GTlOEj0QBuqd9F6fody5daoGauzetxgr8k8O5Urra5DomhOuihC003c5uJ/sLqO0iGIy0bhbj63Aw/BZc51uXthBY9J/xQlY9WWedGC29/VL87ZWSEm3xh1xwhCG46qWpu1+RQExHkM3tALL9e3KSf28JroMisJGTOiPxuBfcYluGAirrIlBo6oRiLQ/Oe3PwD3qwvcg4gIcYAtUyRnUHGfPa6opv4qK60JY64bul7MBAFJUVty4ozQVw2MTjAcfl67iOIE3w4OxghOHQ5tkpDo+hUsH7jrGoBpOpV9Ls/dirXHASTNVsVLGdmehwBOuI8xUpK1kzj0R/RhkoHY8wt7GReEQblWFaEoZFuzcPGotJ1tFbHg+RUYAw==', 'page_age': 'December 9, 2021', 'title': '6 quotes about the ocean that marine biology graduates will love | by HireWindow #STEMjobs | Behind the Job | Medium', 'type': 'web_search_result', 'url': 'https://medium.com/behind-the-job/6-quotes-about-the-ocean-that-marine-biology-graduates-will-love-98a8c86a8c70'}, {'encrypted_content': 'Eo0dCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDMfzv7kO8y2vQ/7W/RoM7MAfjvwAzsnyGPYcIjDXyALINcMG8x5gA5oX7J4YLAID9O/ecHc51NfNWWBA5DgHPVcjtie9zKV4VUvtObIqkBwiKcaCxshlK18M4PVzled+WYL+MXiZno7C6stA46htHfn4R9+WwqImJbASEkVYcVdV0ADJv2sJl1aiEcbTyNgl4v5lq3TUVyQMPMPWnUPgea44dwLPyGCiMZ8//bFRxnrZDXr0Yhi/JY0DCMhCz+FfbVYAU3xjc9J7WfJh/meqHiEIcnlINbg64aPMYQDSoKM5Lw0Nj8ePPqXq6ZsLbqOtsnT+85RycuoQuYgiTfC7wQCLyJqVMeOQ0I2XjWUDB9WBXNRP23sUbA2tSta82xuzKhD9f0IYQ82jnxoSkH7JAjL2hi/QAJmcLxcK1Rr/KKPL0DyqAEK11VsEsxS8jLbPNnNHJSGQjiyNhKDfHqRihMJ+FtbztBFreWiiP8M3JQy2Y2LndRK4xPm858sREXHfd3KdJIlk+pA5rYKXiGZad2In9WuypfN468SvGsWYBtww6ijPACqUAOZJCQPlIpUnncTfrh+9/iSkKg4O7eQuErU5mF6QulWBwFja8zdPEJmhoAYoDGMZgVtY+sGOFJqPKFwmwUOzrn/HsP1UpUMbVvU160Rsp+OdC1WFf38pycxsOvP3Ln2ASjDJ/vCIjt45xCiCF+QaXKdcAyERSx5aX2O7kI7gvEcX3VuRosDMIE/VyVrbwleaBKIe+NCq6WudNw1sm3mzKoi7/mhu9FtpGZs4BWQa+vafwC0ObaBJoIUIrFK8Ie+kY8v/7P/x+y9ouex0vvScw4XPwI6tQ1nvhvwm+yd15n3GtIS2hXPfJ5Lbmh8ExaqFLutWKcIKJ6HkHV8fNbDdGFtww9GMDDXrYHPY8t7SZg9xpVqb3o8H1zijp5b772Ii7aASz9qYnEd60CATWf82vKp1Vb8pC1IbZm+S4y41fSvVZeJwc1O8Z24ren/1ll9pemlxIJqWeZXsxKAFig4u2HcSNJrlTwzMKfehcqOKC9dwEPWm2DVaAsKgRDUoEp0gRsXwLj+8DSxXIoTGAWoyuKsdgW8/oRLdycssor3tWlR5LgB3OmvEK0yo6nz4vLWDQBHAllpx80vD0qC9LVUvolImhtKjsv0G3jVWEndsvXexiPZYOD800sP5jPhWugVWn2J1oY3wTdAL52oEA3qz37thLxJP7bqcDDgeHjY/pZ1mn2G+Oxj3DK/mnZkfFGe01mvXNIfZk7wMAALfCwVsgUaYUtasBCVtaE4jkFcqllmsGpcjQf5nOF1ItF65BXxx+N6SHLACHS/bvPJGQl/R9CsXU7lxs7k0hyweuL4uUg9xwhAVRlxTmTnrMgXJrC0b3vYVqoVc5ASRHfNlH5BkNyaRXCxp8jObtbut1fULGiSDzYl4Ue1W3QxbIcu0Cz66STpqgKaXIG3w/EL7i0T0yTODN/TaZBv6aPlh1RVpSKCEC7CAvmzgMN6ClP8Uw8JoArG2vqggjwMqC8JxkWhup1XmvXLl9Q8qvqNHhr4c21aH6xvCFFcOABPU0Ybi8WgRwEbeOCZRbBwHTJL1k7jcIIIfoWmF8i2ri4/79LJei69rPKCRCEH9LDy3ziYvKOJgRlgs94Ev99rxrvW3CizATheVCOKuxrA6KX35cxjBtaD0+uHfo3GlRb930xMgwe6x83saYyXDWeKBcawzy4b8cWuDGeEufwndSWflon4PY8xqlfUDBu7Sj+7AWXhgnrAt5XbBTz2QB/H2zBrxda2qB9keudXCkGafABKRa5Uq4SW9IEl6is1aGFytsF8Em/lWdD3unhwANE6jWjBriWGek9okBLZqyF3Y3freJXTOavKmcN2RD5E3gInLOqR+wFrcPF0ErJB3WSXAVf30DjLGMqSzEW6XoSeuucrAlvGVNPZela5iEEsBjKu0ISjdu6fr2P5hSrc58Z/+teQ5b+qNveJGgfkzA8kFk435WLaj0VNe/Knt0Br7Hy2WFk+0kx8fe0FyQ/cgxOH1QoRD3FYxLA7FVGT1aou69eTCQCc0aEq6KOpIJ7Izor3alF9/OW6hoWmrguJlgSMMXfkYJuXOKtUlN3kCrf7gZK8toJW3G0QeZCSnkrS9bLZuhbc51q3CEgMsfEzKwE5E112CyfhhfaBGYqoFL1WLWQWUTXz/Vwq6kS+0STYxiWaERYeMqCjyGHnFsG4a49XtVddAYT/W1HfYw8f2dC6Zh07zQfvr0AQJ/jFDEb3vlJPjZlCuVnhvdPF8+R1eA/lpXzR/1M8siF1rgAozrq6cH3K6q2TEAmFGzMaGai7D/6Y6OZ3CwCphiYs6NvO/rD/r9Ti99tZGiVInZim3uSIe/ZoeoaI8qkN+x0SV7hiAjsvW+zYU7v+tjSbYV4RK4vdTD+xDJEfbqQgJHzVJ5ppJNnXmqGbM2zjMIsHTqktf+Olf7FlA4v+1ck1grWlDPfgG2wr803RFBnvhC3tqaKwdy2yvDN0wiENuL5EEreQ+wzpbfzSaXtMnP0GBrXTCkaQeuILIMikUJYrOpS7cXIKlOub5MDGsCsT5jSBpK6sHFNeH49Ux1N21QNkHK1lQ1m7K0CxW20viwkwZsBAVXosWPwZ3GQtYFW1bd2oB3/278Sij13BU0dF1+BD6wUKt/oig+fsIsahNyKgH9jDPSTNO0+P0QGmbfsgZJVN9nEINIoygS2laieW0G3eKFiReZp8w5Cyul7e5mes07cljGlTW3uMku3/EglvJcTDCdWaJlJakRO7/P4hukb8obrdtbI6Q3rVhHimkUfZhRjZAOoy26Du9thgZpRVM4z3hRRK8Jbmo2txW+nphQLASAzWUCxyfB6CBq1qYzgeNQ1eQ8guT2t3GXezroi8hcT3JTjvkkJGmggqnhEdhfVa/ZqF0iel8IVvuJ8c8PswEO2Hjl4LYt5jOiim52N5yA318V4SNOLLUc9G3Mhqw9YZcpzHJ3bc7pe/o0z+aFayDyh1g8ukwmQ8AEHxH2xXbyJPrOX6BfJlk7sMy0TiYmb0Xi5d0BzRTSAY8pNy2sEruyqHYFhrM0WK3vJ0ZqIyUEB1ZYhpkTLajlAns2Pablys5Bk2zLhzEb8iBqBsuL1cjA1NR/fsTP5Hy3b7BFousZRI6sizk3xBaF93xOggg7xJAPAlE9o3SYeNzF413BfGNLRtBCXvOAff62aY3jDfAPcOgRdmb9QXCi9c5K1qyRi2cr0TsHrbv1JkPExX+Oyn7yJuBxcmpjvTirUzjCsfsxFNB/Cbllm/SWc7NkUsjVnSkqQ2INd+gVFX49BbGY2xsCIu7P2Pz7TvG61PECKp3QLQxSh8S1akFa5jUCCgVzHEKqJqmcmlnqqng2zjbPU+Dog6ymPyjAMsvMPVi1IA20FYERr08lSbB3SlbTThS5Nh7VS8lZ61LhQzerQNqysep2k9uWliy9hDe/PnmKHUKHsmhjfipobuceO9IpRes5haHQqwbjnOhXalkGuxIyaeIshrazqmH51cDhdXBJCpYJEN+HRTEztU+DjmQi/aov+uop0/zHhBedbS8H91v69W+QGI1MF5zheKhIE7vSuHf1P9CD9ZxcJAKfV3mTHZSyw1os2sBOAlJ3Fk5xp6a1G/DYxAFzfqmcxjFIEnfVq6b6MlYOkAvIhPI3E88vaaBPz15fZpYM+HiHF6L4H9RVMW4f/LhquT7yFCDik1BkIHaeHMsh+XWAT2PbFt4DAffAP3Q/jvCnfmgPYvyM9R45XSeUi7+2WdL7BAFP7nO8iokbOAjEihTESyqeXbhBZFcqG1BFjGWzV3jTMs+SW/iPRpGihisnmc9ec7CsBGOx968VMDrngrA0aeTMDwcLNjdWIh2HtvVYOBiuegIY22QkXlhh1QfRoyZ8vtfwFsXROuJr35IoYa/px+tTkkCs4xtjmNRKCXE0K8glpqxQNEZ0DMZI4YTTHhNY6KHq3SsGYa7NDKKIaLFRKtsYIFSNjyQ5/nSU8+wAvc5UDaYCmyHDBM0oZ1oLXOHtuGM/rqWtkVfCT+uB2ernwV4fVJcqqS08pzDTjMtMcgSpfXsDMRav9HL2t4NmiCPSkOoF8ruc8vZQsqmiXz1V/kpNw+OPyn8fbNU69xjPVlgxid2RNg8WVOwOurTgUZ55G8UJWzjvgKysveBM8ntHpIfT0q88L5zrDcuEz2JaMVeu5SsIeyLLlD7pdldQgnKsva2EkgXzYcVf6o5PKnUudgaP96jDaQJxTiK05QFQlWLsaujHDfoTfcwnBkE1GrM/1G7Jhbgz5sr9YaMnh2/1p4Xms9PZ3QGpFU5hJ5aoqTOJ0mcet29zHqJJPq4aAOMeI2/lhB+S8OHs8kDCKsGNP+2uB4iYyHWbCFAb+qAsPQ2lqsL4csaFOxmtICcqipcfzu1UElDoCraypRn/DaTohZPj44EKasivagNaMTLudGBvhl6Fk3QBztc9C23BOeMfE9KwIWMp45K1+BpiHUamBIwqgULgdA8UtW/QsnVNkVeLhfas/Hq/W5LXTsNBUx5fUK5RWVwJTmf+zRHuReFmdK32WpeUKkyYS8mrJR8+kJUxoB5M0jmP4XODARHBrJfrsUBQr4/fHXYAz/Ku0N/r7rk+23YI2eUmIcXvpxwpMS2ER4CAxirU8QknW/Ap1IXibtjOgg+07GBiutypN9lVMTIukSRcsSeDW+y7eyMWzG9/AEGTHHcDXeCSRQUoU3crbfyYH4rLPVxG0K+MXPIVJH2ldZHLNG5L9OCUCSWRM+9LwFAyTWy6JydaOcFYPfCPo32p3yDiAU+2fFVdaIZg7NGvyczD0XGTrrR7l/Njxdt413D1SSJKHllMRCvXxgRZABywaY8WPBuzcxY+Moef0oYAw==', 'page_age': None, 'title': 'Meet The Sea Otter Savvy Team', 'type': 'web_search_result', 'url': 'https://www.seaottersavvy.org/our-team'}, {'encrypted_content': 'EogcCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDPBLZI98OtOWPCnbbBoMQZTnvZzM1brlB/WDIjCT3pV7hLnrZDf9mz6pOVqBJ6aLXIXZymNEopmbLH0Hslu3BgAqx6KuRoXvRNMGlbQqixumiIjbUygI5m1xYWrDelX+/K6u+SOdcn33WZ6QFnJ8Au4wYpx2ob9lMQbnhdDh+DQT8VZSc5SYSGEhrf2B5JZL5p2zDxhzY5x4riktVztKalROmnV2ImOi8s3z2lq3aIqwkixQwqVZmiDkdLsDz1Z+OYl9YWxlJ8R5LHOsHDbL1thLEqCHnHAUPqslss4qoNjSFoREKtBqRB/epjs1vA9He9Q8g6U+UoEIz15vaMQVMzAY+KnPpYvnl+/Y3+Jtu9txHGzYxHmq1zcOYiVQPhioGuniTpFISwox7x9IKzR6feArZ90NAUqrcxFmxQCvL7LIV2WplqyVA+INcr3OK9Qwb1w2YKhdl1aHfBG91Kz04fwiLrh+lp4ocuC1bBAnH3jJCbWKpI3K+cfwEitWnHugPCtB1EQIjSyjTBF8vyjNgandhKTBRbtwbS4WhQlyaSIQczLqJWNeV2KFo6EAKF6Cs9IDIZneILjvOHwYdQ4AXxADVCJoQLAhWFTfRZW4Dj4YUfgYJ/ZY1i5Wz00BB1IF1caaHCVmrvtoUuxgJsekyzQuf+MdUdyyRYafKoEU/fwqp+BnPPEsBugEavOMivYBZyNBqdqJNMBBpDszrSBAIz81sfSpeD6z6bKBp805tN6o+HeMdWYeZ79FfWA7pPvk5iP9AIflo8pWa+gYHe8qvgrtRF3EQtxjHuBDu5gomPMRXyI5IQZOxg8X71e+BYCaz85KAPuWR3FzMDiyH55d5E6LUbl6it2BNMJEuCHSZ7xGD80jkuY0bm3nWnHG689rRUorSMFIrSMDCMxzLY5cNCLapJC3h/LYi8kFsiKVycXD6yyPj41x3ApTjyr3HoS6OiZXs6cPqxd7UgEE5NcHthm087XrchCOhDj2c6ytpn3CyGWAuKVmjBTkADqrJsTsm8ZrbuVDeyj5/t+Covryid9K1oCkpDIqq0NVskwnLqVAfbJHc/7z38ZD4wcrgOdMd/IUegWQiG/srjQ9vBwjitvA6+JTbY5LRHvedzm5f37edmwLGxAZwJusfNNNijnfSp+k/teMlyswvINQAsx8AHJGyDwvwj1w4j9fSirsAWT51Y8f8MpjgzJOyhF5SXQrLtPohEGxj6q4cHHW88taFf3QrJIeTzGillksbcFXuntymp3EyhZCdGE0NkoCNkSP6da9yjSfgxxnSAcL3ZShd/WWewSl2ZYv0ZsibTZUq3GiWhYvonePkEJibcDs4UFHy0iAVBuhR0bkfQyQ8VUabXoadTQiZH3dnAIlSI5bVl7+eAAH27LEKy0mdwwBWU3gd9gJ1a7h3HYKdrin9lLrTsVSG/k6fKDEvfKMK3hEpgojUsD/Ul3FINLtXFS2dj1CEXuwPU+b6n3pwXoxAfiqSRuoQOA7+abl65FRYeI9FL3iKl7g0LAmV0OsuSCahCdOqTcyPf+hrWIB6WKDPCfvV0qTVnA5Htzg4tdu4MEbEI2VU0RIUx5LHKkBn5KYZSm74vWWHalXpdpplEFocakwA0dQ10tE8/4AsKb+dDHlzPRpiFhUW2AL4Gcd1XQjN3T+ODbrPplGgV3XjWU76tcdKjxUYfafLpNSTma2wW3hvzm5tp/c/0fEpDlFQxpPaYesXOiQAoDcBwt35a798ElcUaVTkcFvtdFVuhkfWmkbXB9WCbJ1e2Z2RYELv5xsh9GhUBZ1mXoaT7GFwXogSfsjvTjeyff/y+6wdp7yvid0r6rnB7n/tlwBzkXd1gdX4jXa91zCkbBbFZ/faAJv5CJTo0dteEBwJOPNOHVL2F8ebOVrNhRGM/vjjE+HcwnECKeFs416p9xvND+H1B2UyBvbqkU+gmehjsMxGiIxgvK4jPXWw4UPKhsK8nTL3j1etg0H8OffYTKdF0RRluXVdsQFfxfE0JPXVS09n77K60tyxYZsHeVZZSiVuphtAXnrf38GJ5sTFwcy7LGwX1qsbYjdW5fjrUiyCOzXG9jhH5HFpgOiOKrBMnj1ppZYyvpGDwKsdyuxjoOl6L/f4SKlqXlFNUPZDSxmtuuZoTFN8BuTBHkCVuUsO0yf5r/g4354b+7s9O5l+mV2Ih6gNqAmWDPoa2QUF6DjyHdg0WOFlL7xeX5na9X3uU9zaoMQ/8e1XBTl+3R6PhgAMViD6FV8UcS099E3fAa+64bdaKBpNDCA3/P+XYRHbRD9cu6jCdJUKCswe8U/QcJNYLMTPcjHusStBU+vG4LYj0sneNka6C8cFv/uR/bj73VEW2nwufCsUqN6LL1/0Vv23B+SXBARWIqrNuGnGxii3GPgxzq9hOIMStxB38dNWeH1I8nxAn38qdmAM6x5CZyIV5wlZH7By6AiEq6+rKPPT/96o+VDWAENGARBuPBDW5depc19v53psGOvQ6PXY9tvXollzAOqtawT4W9i9fWlSYqpamOkcaSN7QeS9A5TSFJO+ZADr+x42dn+S3Ju5FrSyN//5jUZc3QxiDNcbZI0qhQUR9NS9NXOLb+n/ytd+uNdDkfKnNtA46DtnJF+xAvZCwPUEd+k15kpnU6T9/3jPheAOyuQddb95UKL7JNKLraU2HwSeByym+lxbmj7L5poA5kl00GSu23cOHuumqPVJlEUVqaqdFq9x5hF38ex8lmAxxFZpqQDA/vW5dS5E+cGuCnQKJ1v120UIfHmUfrdG68GQ6MkQeQHt3L4MZb15JvLuaz0Vbn2ebYAG2y3/jhq6LN572dlHm2Zdsp1nV4iAcHQrL5mCBo4in3gjzdZlexa02R/VCs7p6u3JryfmGlaxIJO9hJ2eaI+M3z2P0o/QsJV+15zO+VhQFhvHP0KtITuC0QhpI8lopzI6Nh19UdEisaXW4tvW+H7QWHDvlF7gnL3fQa0QpMeCNnfH1+iofHg9T1LwoCYYW+Qtodg4fzbMH2fGWEtHwBDIhJYyDwd/7KIFqKAtnhPLm2XEt7wu2ojEyZw0TIuwtJQMhQi4A8j7fIoaI1mdW8AA69TM5FVZv25+dOTgpRZKu+qgHygCDj1bjw2aptASb16mYhnTRKv6vK9LZUVN8GicFT71KPcQDx1FG3o1z1zacFJPzWeNWRPse3pUkARmfvrL56u4KWKM7RWoaNhTGJF6YVP1IquxswfJ87CJye7nVm9UjnPHThhU2TxBcuYBOXGrkooPbYHA6qpxpis3DvY11a253m5oOa5vqbvvTX1KSBqSKTf1xl47oyUN9d7Kh1yc9YWu4IHtP5UyHtR3bZjFfn377cFMYUljSLnnDDBEAsNgYvgIlZzH4/G800BTI1O2m//Z5FhwEbt6K6y/4r2gdzoHHlU5UkmNwt1eHYkuW7IE5sU++8oOZAsDmGCBFM1oNxMF7b11a5ymuuGyiH9LXt/TL+brKDnzgATMYcrFhcnEDLxO66bfmoxegoZg/Vq2dYvLjHqOD84sryk/n2EoVoQ3dJEj7kv6W9xgq/hNdnX++9upm3ijjaCn9QIxy1fkLjtySfNI6shZNqnhaVPGZRTzo+6VKzSwQAfcQeeAkyP3h5b9d3CjPlru6+piN6HcDJP6D/kAf/XdZhYRWgogCvMEeXU3st8ouc9TfdKK9E9Ci+CcB9/aN2tbtlAJr7TLNJZ+VQmjzS9z8JIfgrwqQjcX9f59Zfl+74lnEVVuUAXJSyhzFELRSximYaGuOVhCqUGT6PO4JGNJwpjMj+KojWrOFXvAeG1TCfdDBg11iUGpnNYbcGa3MhmSXP+hDB3g9IACOqHnm/XdnYyV6xc3TuZxOTrbXjpMDhpZkr1aHdckY2ZSV6HtMqDIlsapckdj80h9QsBST/dd3XG3/Cw4AFucMs4V7ctFjSj/hhUpyYhGJ8FRhFZs0ehjDO3lrs/PAOPMYwGN60ekLRjIPhSP7ItXfgrzwHjxAsCCLGbHdCwyBXTffhDdTuTzVejGhR18t/BcxGvMGfHcjiO03JJY1gdwGwF09F7wsSXAlETj1zN2tVqQK4fDGShnVjU1vAOo9n/Vw88GuTLt7LpbvisE5tRA23gYheit1/oTlbB/JdDR+aAifehgKUU7XxVIMY9hlwXagJl6PCZ6dHhcPRF0Wkv2xUiIVZLBFZTWBBae2gnT+AFMDSJrM8QArX/soL14i99obe5ZnIl3bJa+5CsDCRQFdidrQXvLhKOubZsK68iCOSAWdPg+BmAVfhDbdmaUdjWgdSEqBAJ3k85wmKjm1FKUP1hQiCim//RG4xQFlCZ7NcXJmAwjRw67lXi9VWSE4NdbvcwutIE9bnincToVCwoXFvqLqlpNeYKMxO+vWZ60g1mZA2sOG0hnby5Ut8kYOobK7gH+4h8fg7sDPZLAFrMQIKah3H3L0RqG/natpfXFkbSXhLUJsPk+GUGnqOavhcre/esbAdO3y4iPZcDxOefOXpGJJXGr4+HEW/P2GTev7s+LBxwcMXZStT21TAB/AJTcuK20b0Gm33spD5Ku+N4bebjRtx5i/urHA7tfZgNG4tGp3YnTlbrDsDU3kRgpzmnvLC7IR4y9v10f7lzbOuBD4iSDgBELKef8qJA8tnWa6cYusiFK2kMO7h1L1k+6MDd7e/ZEPIBU5ZJqrXSCCH2A9+78NTi3x3FKBITdd94oxomwr3L8NgyvxgD', 'page_age': None, 'title': 'Otter Quotes - 2 quotes on Otter Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result', 'url': 'https://todayinsci.com/QuotationsCategories/O_Cat/Otter-Quotations.htm'}, {'encrypted_content': 'ErMfCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDB9iblgskYJY7pVzqxoM4HunZIUppSdqMZH/IjCLFP69KrzwsLN8ECx5IsVmeGn7DTQkwvDPaDX389dJghYKT6WbSfMt4RfYxzqAW6Iqth4REtEVQ76evQFbKk/ikfgMcKmkAoUrAL3L4dL5oMYnbv9TPpga/I6cxmt4VYgSVqaXPeaeaa9zOEtUjPKprWifyThYs9by0e7I0OD0e6YHo0VkqDLWoCaEZYdIAtDQ7xe548KGzQWbgtZYbG2cFvyQ/to6Iszb1Bs+e/PPgB0wUQBp2Rw+/gBogPcJaQhszVyk0OvICf7rQKrBbjfGnYP1DSOK/YUxZkGObfBqtwBs95SAL6KxCMKvHfmj9Qi9SAjeoyyd0rPzd1Ckt1IQeKFhYl2tjhdy2MMt/knH11F8IZVQg8EAv2du8zrnVpZAiHgiMEhwhs4hxgrNvD3veIoynEi76p4PwP410bJIt/7EcjRaMVDIIeo8DGIrhDPhR3iukztUGUBJiolpuR9QEuQFYu9R8OXLnYcajwpvH3v+OnuRsOL1UkCpc3kPma/7+GloGQpjTgtCYwcJhZMarSL6j5zIyf7j/UEMGWeCxnLPA1hsdbSSaSwmBe1FZEmgGBVTfDtUs0HFeTMWBL2sH04ekUSB2eMIdNzMj6DF05oLtUYNkz9+ZkLD2VvxzdVBNtJzX+9Rb3O3WY6bxV62PEJs++eLU6jjDRuUYOoaEIDV+6mKkc0Ulf4fP7syzw13nCEN2ZDKIdKWOCRfPsstmahARZL8/gz3Tvyi/tf5yQpRnDtUaJrxY2st/AhNN32a8nDuWicsULsZ/ana+riahs4sU97oyaTVkwMFQRcWpUvLzWvHOwalqoWBBjxo6w+hH2suTjEQ8hL43tqrJC+EGeAsddJWhUrNUWkcudj/gEjylG1YF3pPuW/+37g3ck7e8PuhzHcgbVtHKVViVxc2sbwsIr5OcLahEr2Mxawp4O8+UB3CcnNwZJfiw3AyT4TC0KwzIAflA1sqjw4nD5mc7ad9iXiXSaFjRXaNrlXLrnVetXjj5thCUAiPpmfII30G5kqoV3pm+et2t31uq9tMgN8ZSI2I/kvlJ5lElEyp3kHRNJSGr9M3PWxZTC3Vu7JaCChJ6Vzo7rFe8eu5iDK78km4fgCYiUedx+XqLiS2wqY0dHEL22Oq3v0iHt13Bgag1k5QLoD9JDrV/6lsHuuyFpdmyk1bK8FQKyHZak21quqRgK5VlMqJ8Mjwbryr+KZtej7YdAwHaE2lTA0uz4h5j5liTJnYxqKN3XG/d06+ZgCrSsaKBg7rERh2Y2JiQm6RF9mKjdYWnIveHRetdGpnTO5tV8eXbLgpx5+BC+0tYKsQwCGNaLDFvG8pAFhhAqBNHc/BmUElYeA0sZfis4gGUZLSSfcdQGfWDicIJZ6/guQNBgBNBupxUejxYvEWwiFM9gM45R91E4UqWmYwLdm5ab7y+BdONCl3RvnXFvm4Cc46+Us9WGViGyzqXSo5ZZDpWS3rIg8A3EW5U5fWSXN7ZGLNih8svNFXZhT5oHAnATXuAPbVEfBakV6TkFdHadz0r7aB2HJCjFDkJToxI+eOk0BguT/dcdUIRXqQ4s6ZRQ/yS3O50xtLewHaX70wpMAN/3zFpgkXp6L5nfb1LVBTzQe/ZKfVoYTP03TgPY6o26DxGcAfu9/lyDWHkwCH75kcWyJbLBhiLvNqdRqHlOb1OKv8lXRSgOZLiZHGmm59Q8R1E9s5SFK9xTZtRQns+PkqSF8dg9NmpHGRvtTSd4lspclnYTrTd24QRCJqO5SMvsuOaMviCHAlxXl70WDzSvCgBn89367UBbF3mWMU42NIJ+n3tVSn3iT4q1BOq/QW+bGze/nL25xPDCJnL9OxAbEkCzYtGwp/xzV9wEPfqkbbY2AgrHoSO6tEqVN7IP7+rZ1RCZog594Kl7WeyhH1zSGPuxd+/ADhAgNSCWwBFRFzduCDdzRAs/Xa1r/B1x5Ds5xZHJOdKui+qcm+JYrCOiUQhIOE9yywzrSxTnc4sYeC+jVur6Tw6eAHMV2HogheEifvq6vPITmX4NnO8Eynir5gZkTPRqcsFfbpA+uA8T5DVMqDtlVcTqR3aJ03okBYZP+jZYuOWBIpDONqjsGkQEkpHb60IEoWV+ZTqul+zsZvsu4utk2G/rQKI6YvU5dTCQRZTUm3/Xk1uI3HGgo6qOIzEGZKTJwA1n0eJ6YaP3tPAMpiJTIuIJmtW5WfNCw3EOiEsRY6iaOC71aLethsdVb4t2VJpKw1DDZBg3kDhHW3cpmf+PFODDoseaKflZ4cio0GvhL/x8xWp8pLrrFrA33LcJUh0djb2Ar6T7H36RnOo2DvDGlvkI7fFH0c7rfNWBw0Ge74sjt5oeDCRhX/9rNsbTmJy8bZ3TozSKFxRVaMMFkQ/qPK6C4bK9tex/BRaviVYa2yssu/pCBth/4hKx4TZW0yLrc82nU54DNpPoGu6xGZ7LeAkdd6jq3ugOCIm68AYSLzhMPmn+gMuNG7O2yaV04MtAcyMpHazL0jW9cID+4xtUYxjP1QvzlZ7QL/Clgz0IOERINZ1dIGs+XmMYFL/sJYacDkEMAfPKZWmCAMpco33DZjtlzMTpJqBP8SIU5WcRlrWjZ4RERLcWtK+DXN6qet08zPyf35q47YGlREKqwh6saOo21Y70dx78MrSacjYmpQunqzA/6mMFFUgKUdlxKkg8N5hwIrCLSr5Tqyj0qq/yX/1FFgKBwHBHQ4GgKDaNuUzkjWou8kxzQ/RGv9oHj1X2naDG4RdjoAbtDvOBm9xZsmSB1gjrNJE42f9kMDU0R+TywQxIDrowXJILR90f1yXwyHUeYzHJ3rHi6wKcCdYxHHq/UZtaRfgO3nGv20NNPU3cp7upnLnQLXe+42fJ/nMZIc7ZqMlLpqawOQPNwMdiiK/UxhAhXGuUH/9N5v7dFOQfrFbnFP6IGpBAObpDU7jC7TsDazGX/w4LSzu9TgVytyr8/e3Nxb7JTPZs6x6Q+A91vIRDeAPLmdGAKVZ/k56QmqtXxWU8WedHtO3A/XrEMA8QLEzlokT/YuypuMjyWSnOBYbRHF5vcwiOTGg4bVuioqyaqBf1oyEO1WTIOYxiPlZDMrlTbBYL5wUr0vIgYclHmTkLzFHe1JtDjb3czghd3GgHE9DIuDgF5Vmp1oNEsKvyXAA/QHTFhK8visInqwKKupj09iLMAXXJcPvkkO1vCx7TWpbtPrW8QGRjlmOf3jQ9eC7bWO1EsDeN+kl1/MoWGgUgdoPeXNUf5O7WFkxuRds9/jS3B9+LYsuV5m1u9RuytRH4AwVFAj/93xJmYTtO1P9fc2A7ibk0FnuFKxoJaxX3YQQdhYm27W+U58N9uDWfsykF4ioADBF+GIDm7lYGDgjkuR0w3sOpcbrTmC+4ZqxhCaxn45vZqofqQnbhRLawravCkwuGrL5plK9ZNgEZj05MBvn89kS5WpxwHoo70NXLtIxAAAeCga+XJ+MPHkv8dg1Cha97hCMBSGs4ZTv9m3zAl69Z9nOtyNuWLaifHH+JrG2I+4Uv6VM72EvVGDlSFfUwVYMIiYOXZxFhqrEeqOyLMQtiqioBE4Yqa9XO60WN+0a03vMExvfwCvxd0htBFfmmyLX9HE+Z8XRIZGAjHOSoGINMsrsa4nMXd/JvsPd1hCXoBGHVpLEV34A7nHZlpvXbr0tYcgPTBssYrvnISiVgoFYQdcoHzvNe1yJWJLnzt7fq+Nn5mXohzascANk3iaEsXHDRfNv/U5IGiFdzEbRLCM543q70Mc+eL8fVmD1HuAv0TcmCx2SWyhUZ2TxreJHne9wNfYjeW71mbc2S0cmBs5V0XbHDgWxnhhkXlxluBSmY0255eDpN6fYD37kUEDADG6oOgCBsmPXMJKpDLdf97QTWzz7OnhSCHBcPcl83JrlpKN38vE/34Xin2TT1+YREmjV02ZWfue3tSnN58/Dv3nPVnyqnXESbwLWpyAje4MkfQfQ4PIoT30etkiV31ArxpbgUL6sxeZU3d70szfJ7fpBUGDMgJqPkdsw+Fub30k+YRzpUkrDbgZGI5JzNehd2ts0UjtO3WdbihP2gcSBqpFi4sfETV7lZVowvkmFMXnJciaGYQsqoopduJrstyNwJ+mOfqNGOdBRzMPRxGtpWFo7eewjRLmqzvprqckX5grqheRlNIlYOUpg5D9LhIhp2wFj6oU7L4QtjG9lvvW1Uk22ATzlsbL9NyBEdNBz+Mdtwizfs5rSCJc+1Jca1Cf+J8V7YIXxaLgDJw6Dh9Ca+3cam5jHjkf5n3AasJg+ETOqZnc6pEnzyoSJiw2tvSzXJn3xesQINI7MJWVZhroOZ39vCCteO7v5PIY+4IXAgCEMZWodInCzQkm73Lor2ZMvlJOznx4ShQOOPKX9kjcO3VS56bOSt74Jk1Oh8ONApF0BMnC720JXRT7SHz/w9Dgz7pOL20Dtbmlu+ZxVU9qGLSdshFP7WVf83C3g4qx5PYLGtUcAr/FIDnBkhkR1OB3nvMVK1UwqoMNJv+J+JUYa22WzMADrBEnKtBuZsFiS8TX+9TNCksqqhe1kRJNnwRuyzMuXMZckrQPTvSsYTO88hXIvZ+twLn0KGohhEs0eypATkQLnTbJT1Y/Ieef5L6quTSkzAdtDAFgMLWs0mtuKqYn+AeE0SkRcoLHpiAcEuRdD+oT9ftx6IrtZjXBL3XU6bMSYMpMoZlURcAai9kKudawy+KlgfV4csosdrn5K8IVyvZYTF13woNUp2vw1UqWx4nEwKywtF8PXL4jkS8asNOi3UqU+PlB71U1CCLUFfMQExUEvkhp2bJCs1TgllDtk+7YhWyPI1Yb5AHp1E4OOqsy4ewDH7HFU2zmgyRcsW2aYOnl5zcUuj17fKAXj/W1txc2DCypj0AWBtLPFy/d0bC7RiUYnFCJKcxuJegne5iTds5SJXViWWL8i4xr90o1d3TRHGt0SLmRNtNViEBQsUmPEXmzSHZxiNi26xeraxBLUY/1c+7Oma7ejB3FmeBg9KESIXa3iIxx5EALF9pX9nncK/iKqc2/leyf0yl9TpGGbMrdrMGjCDw+B26YdbDFMv63OlHoXO96Ek7+ZK9p7Y2CwWkqsel2cqOJ2tByNETGa2zKNcsSPIsimHHh55FQo6cQrNpXTf1FOoTaj/kf4gtp/Lg03TJR/RpCMK0iAVf6w0O/8UAhhhKeLcXQP++5p4xQa07aVCRVmmk17gyS0nu93O6i7A2pF+/ry+Lz4LAYAw==', 'page_age': None, 'title': 'Sea Otter Savvy | Sea Otter Viewing Guidelines & Disturbance Education', 'type': 'web_search_result', 'url': 'https://www.seaottersavvy.org/'}, {'encrypted_content': 'EsUaCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDCcnTvzisyQaGD8xLhoMJfpgc1xJ+c8G1kgZIjDCGUc25s5DqBwmqWyLKCMYUz416SRTRfabzTJUljhICWimifLIhyTHGEN3iakuE1EqyBlOxmQryrHhWR+Cs+/2b1IQ3zmyFgoWrCP+XEdIVT3MXYfZBzzmksNvQgEOuGHfjr8SAyHnSQtEjEW7xDymwCfsUjjnykuUeDX63j06xDYbkS4HQ7UUsvYTgQIYxdOq8MyU9ko3DFoFgFcEbTuJpipYomfR9OsY+8jWVmQUtaikL3kQXdj7huwH6RBLZOEF2B8d1F9vPXNTAV8cT7VVe+xOAUhZR76k1ldHEGorBwLVAJ1LcPPAyVYTBMWPxpRLR1aUVfwTfkKEkZKfRAqshmOx4zmlZQc6v5+B3JIaD/tfKFkUkMAlyU8VL1sSyW7iIXK/RhlKWgt/tFAzLxNmLjOkfTuQUUg2W/F/687VpdPuDFcNFAZrD5lsnJGmYxOlAu74zn5h4wpzD9d7DEPiEcRXMHGRpIx0K91SFYg2SZIG0S/YOxXXN022UhVF6kb8TXdEIseWc+fWIP5Jq6eLUFAocGPZZD1Fag/CmZaZEvRv64yJM0i0SoxJS4y8L5of9cloDkb/wvSlbpuApS4q5fAj0Sezaflw4pUXeTib9WOiPgx7jg1716pHIAfqN9ezERgIMdzbmi4ItqInQXn8u5GE47h/LYShaJ1A5GsHElLSh50CVoa5jlG+6tOCdbDtWlvU18htMPTAPEfIgb3jUYK4eb6ljzmjnPMDftvDGIPUZDaG7ZelIMVrHIqJq8Fh7PNNo2K+/teC4Ci+1ybn/lWYH54Enk8pk52w8rRT+IB5roVSFdeOsFGbshRg7ZucK99L7iviGCjcJrTsnMOutVrJ1ZoP2hicwJHYiw6uQ4SDtCqjSPG/uXamjdDa86msnUtNZhyIeozHU5FxBGeVp+sCQXzFLe7d1YEoHAqwjVSeVka2BC9XjHwcBfEyTxq4IMTkWd1oxqftd3Qo86EuciTBbcb23aEeVTSxD/gdp+HAlTn0k8RhTP4vlCeON9Umd4GEwX2ink13howm7yJ1ZEnIEGfnf+YcDTejH00fWKRLcCxQoPLAQqZibx5rylKmCNkAO8J3bZmjuVbBVBbhNX5xPSB+TvarNkrzCky6DPk9P+4ZZ/L0qmwpKtmleJ63r/40xuQTQi1qqFR1gtrZngUtr33PysLklJlBSiLHtBsilkqAxOae7vTfGOLX7l1dTyxI5ye8bN4zGPYQ4nScdoq3C67BVjRJewMaplhR12brxxs1ioTyKMCSgo1xeP5nkXBBn8FgGGCkzN8/NRKTrZpVHiOXnD2dpOkEjwpT3CepxfhB/GnSJTtsDx69wdErP235gczOAv7ZfPnudP0oILuY76b4dAJOgI45aed7YhA1D02WRzxmMXE1dZO5B1W2ZuHkqian/oGxCdAbHVVT6OjMmk6JyC9ISyNGfw3xw3takVslwL7wG7TqoaRWDBg6A8boIYGAQ9z80WmPa5a9KVHOIfC50RbC6OO1GQFWuqt2XuP+O8AMkMaSIEjczi/2GATov1NAKS8axSW1Ahdgyj6/VAEDt9n2/VYbah+Bfdh4tAu0OPre4ybi8HjzhKs0NK//0efe2TrYJORcphsOCBAFNKzXbFeG3uKZE2teViyaEFDKhNb4Cel68+uq7bViKE9EEgYoWh58wYKBRXW/yu9kBNe8YWGXXBvpX83v6F+Wfu9vZabn3NKt+e68KNf+7Zz8XEU8OjcWwh8ZS/raPz3AJirvsJDJ1Mz6+I/AkF5e2f7q4XUsZqvQpHiyQJvwVHziypMxOeANS9ZQ3zJ8jUispp3B7ElLJedVqHAJXR1DynjjNuu+LOGgp5To/+zL9SBwaCWBgI7DJOEnGqU32+JdCWElJ4afhvRoA7X4D4Vo0VbMETODmVNeQXVKL9tx/a0tg32rD13kOr8w54Fn+DsH/cJQhKVhiUT5kg9zPwqfSvEH9JFvJkbz3zZyQOjsrZ9aElVVH4PPaQB9lApf+UBivaAkJ3S9jmp8yHm+wSlwftIVBfk1U5DqvEd8eGO9Jeqk1yui2Khet7O38nYn8ZfvfNvgCmvL7FpKK6Ln+7o5PDlRxm+NIueam5xPGLIH6bBXmFZl/q0+2fYFk2tHuUortBLmbV3qPPhfN2PopI9/BPau28bbYr2y+6XaZ11EV1wj6P3aSVSof27+Tw89SDNYidLPmuWIOP6m6cO5eBp/6p1uuJoiLdR0ttcGb9zHSEmziFKXdN3SLAv2VihApjkTobxNlkb6vFrkb6iJPrMp2XzMxjGWvhhWdI4GEei95Akd7lS8pxFx8ehtzrrQobymMxQSAPViWLuX2ki7Th3240t4reRIoR3gR/SQKtJ20dQ5VQFQClBTlQvd8+TQVq0wWSbguat1GxGS5GNv4AvMZTmh/GdV/QUQcCmsDxxx73zoM4M/rknBy4kLFntxSA3+chbYjl0rY+PYKe14SCeLUjgNFNBSYypzvM+nX+brApszjvIR76dOLzCP+2wvP1gAymuZKy+BoEqmAB3WiOO/uMzK6b1oPmXPeQFBsHHjqRiGwP26V4mEeoI0ukh5Itvh7cqyPRA1zdEKoyIfaaPUdFaND4R2f06eIDauOZHSvPqqjPZHXv0BB4eJcmUmUoiAcJE36B7sEuTwakpSazoaQMg7rbNb70NM5s4yKfYJk6v88fr1hMmzzzWWdocCIDxhgdQUDq3A+kWPnh1O4pawSfuLr4Vtwh/wwfgcJ+fW6vNsCqJKqF2JqRs0a+RMH/VpzGn95H/veGcsIC2PoxcgCB0M2i2bCmhaS/FzJ/AoeFE3LgPm+jt9a0clR1oOOTRMpI8DpyZM7jIBdyTJOXh58Cy1AtnkB59ikRiX7kwrtOTtqdb3OREj0GCD/2SKk2SlisVapHwiLavXmjOyuq8rDfaqLwP5MOhV+0/31MbbersBCXqiTTc5gYQEhcjX9mnP/Ti+MCXIO2Vl79uxYOU6qIIWpaSiQSLhrQq/tnrzbU+Uowc7a75+WPGev8RYTiWtsA0f8epEKToc/4v7QIn0ThdcYo0t+B19K1aHCEi+4RLXDK7cEaq+wgzlwnoF2bMWvyFLTXodECwe7MKNVEkQLcJzIqbfVd6ahxZNQRqzayqOYRxqqtIi6LF0BThVK59wYFsHxqrp83W/OBuveO6jBkPZ3EAVqirKEus09FhlTpbyQX5rVl507T23TGv1GBWwz+KPBFejbMaBy60+TGg+KeyhKfC0CUsFOS9JgSKxoY5Eb0EtuTSLYU2n+EyFnAHfINExc5OOeE1AALQjrI0B+iHWyGeNduw9wW4XV+u4hs3EByiqrRIYTckymnx718dalfWVWSwuLuzJ8MrSPxooHIxb5wokqh/0/dnH1JudirEZegvixgLFnfYv29tOGNhSkO4IHBl8+G9oLdpByr0nBKiXGVcxBFVoLYrxDMK/1NMDitZR1Gsh2IkixmtQ2O1AzutNLVtY5gWvHR0OFf+uP38At4h9G2UA9rnFlo81+DQYgZUrVCly490td1nlhEdE6AEQ8IKwGPmghrcMFUmZmFonKzLBjQBX77tUCq2juF0ITztkt2cSAFWBhNDtlT3MsPFIKeTiFIVPR57M3jMSWg6pC6r1XqfCAztofTAalygGMOyQWj6cQCQLhF58+gERFM6Uc1Xb401qqw5hVJbZQAPONmkMhJxAubWIOR0dVbcl7yHSOBtI+5nAC3UhDhL9QGoVXoOQAXNxDtgCAWYz3bCbZHltjWkUAE/bSuuQCDYynl+EeWW/K6gpYOsgieM3rl2DzB1vMs9T8AeeHt90qqlbuxUKZD4LWoGSRVvVBoS7Is3ltE8ntS6GLol98wAvIXHf1GA57k0arJKuQZTAcdctOW+xhN1WV23yEQmyKDZfhNypsy2IPyPF7aHhGrLqqr4a6BNx0vDDW55mEdXvSJJeEfZ8QOGU0EpDQ9JwqlC2HBw9HfNbHpERxPMu/EDIzerOeuY7M8lU4Nuj9sHThYIFuJ5F8poMp96coaGkCVgDW4oPSBLqz+ROJllueTeUVC/zt/5e3SAgJmhWx/2Kai/kF82xM9r4m6t5eprDswFuE7g3XR8VvckRJxxralWOj1ZdBfzX+5wkYen2pKmp9glvU29iZppB/EzQAvUzA1VoawdOnpZm3UhKvJ8B2JUUDUk4esbWk0zSv3+e0I4f82B1g6nD8wXT9gDgiwws80JFc3em8uDNip1sRqxCvyrgrnWJyg4/qCOlWIe0Rld0myvUMSYBxFdGSUxJv2T7S0y1BeQB+hOFngNELMqHE6J1dZ1MGx/uBTob4NJQlatqdNTdDTgihy9XM6Q819YF+mMNfJFavGYUDAw/lDA4jWDCB8WzqqNb0kCnQnzkNIpFAPTl2jB3VsTtIMztMJTbamoJ+alt6qBoLhgD', 'page_age': 'January 20, 2025', 'title': '20 Otter Quotes about the Energetic Slender Mammals (2025)', 'type': 'web_search_result', 'url': 'https://www.trvst.world/biodiversity/otter-quotes/'}, {'encrypted_content': 'EuAdCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDLMl6OJWtlsTwULb8RoM1oc5epdsGBtDX7upIjAJyr60944Ef3aYC1j2u6bCvyBhCdvSXnbWUqwG3bfcGumiCdWJnr4qDSbi+rt8YdIq4xxFIHLuirpcINoa8w07mXy/Lk2XC3r3GrUzcZ6danFXglRSLClzHlpxcRwr/Mn29cP2tjm14O4/jrw7z5VMsapxRjPHZhBTo+jRUwsdP/afGVhi6QDcf80lhdB7NH8dodbRJ4PWEf2FOLxK5kpl3/zeqfngyuNsA4CVYFqbw20gseHTZ0XyQzkFSzd9LcEonmTNhlOd+jg+AzEiPPFQLp62IOBDEyfKcSbJxvneMikemzheomgNs/PbQ8ScoRKW4wM2Wx/Yeq9ZOLvBTukXMmkiONRU8KIJOKCdPRHjQWmU8HqJr7SpDPNoPIzbHOJu+DaFFGxoHSesyW+uCSOeM+McTkQ73eUzPj4yrGiVqSXYlzSvlStiyTqfxwpTqueAJNie/tMRuJmJvMBBBtdUQEP6lQNO3WgbBOUcVsvualGBhk6ihAVjJZhkTZwDmCxxSQ5H+0UKsHOGMXHGBZWwd7blCc7Qd5Us+SnTKuLMdkIU9cs/NhVZixT6LT40mVL584D65AtR+Mn8olsenfkUkYQsLYgH032nTtvJphpclSdU2T+7qi75ESw1udE7ceTuemt+G5YgT4891kTBMCEbvIT4eq4gyKTpFlv8mhEFMxCn0eL1XYScXLPvVGjPnwc3rZF9RQwsOtsm7bbPiD1EluvwkbonFgDvxKlOVv3SnfadtxfbbWKoWrdEy0TvIervTNlSHO+eqROexrkA9AoIcScrrF4B6+5GH6ERz+9ePloxMwph8GH8wUP8q9pyP7Ha61Dh1NHteQapo5griPPoHdYabYiXlHM3B4kJLN563xl4+4b6rzP4Bw7gcn90U9sF84/rSwDEE8oe84tYV4g5oAGUIRzL+0LeTNx029bs6RDdVA1/ikcD6IIrcpykixvFZ0G+wXnuM/Buc8bt9kztKhmGow20difwbozGLFFrwLHr28GY5fNHFC4yEJ+lP5IR5fBPdRyLMEDpZX8fkSx0reK9SxptapEzuflp9134YKOpmsda35n23Eys5vCe6Cq5rTAkQ8s/fkdkOSoTd0EKPtefdn6n2HXF60d8rDtkzSI3Vb80HoWuKBYHlJs6ljffiKPRh05OiWgPnk1EWdofltyaPG/gqO0Ez5R2DEdbEflrkMwnfmQygWrDsLhd2QOlqj/bMe96GpBrC80chdP8PQuEArKgKZMMO34txyVbWjE/w5C7ShNXlr560muQ8+wNW3uwOu/1V3suPhgRAzmMmQhayd4JcutXMCaEXCxQWGAOU2mCjy2hRxwxQSfspypcPGywUCNqWg/tniHsKK8fcXEO+/ZCrfDhI8t6zF2qgHlswU88i6V8TmQtq5d+qgLgpwXQjIrttlTn9mqL6/RsuZzp76oIks/w+8j06I0KD5PHzfe/oxYrK/EZ4xg/Q9rzg2+5yaXO1kbSpd3BHO6t/0a2XpNxZI9rWZFy2N4bkxnxHgxcx8IVAfhgZ5okq5uKd9mHKeMnGPpvgtbUMGHr4oydizvBT383+VJWEUOtcKF6HY86cKGkqIuvuzAD3v9vfSpc4Znuyh4deE8TuNag9JXK/FTWNgkib4O1QuyW1LJALfDGlxLzihpFjpQLY9z2F9Eyc8GoDeVVSVnndjj9GmDmHKFs2OOZLMqFly9dva6yIIzqarehFx5Wse7kVx6qnib1kJlv+G3wYvOkBOVuXnF/fxPG8wydfOSLBd4zAFIGC96Q6V+0UHpMQaT2a2GRdqCRe9+qCFqVjGMvvrhznoMrP6kgUMHU+eomvOk0zWI+bKAm68D3tbORNABSIY2jl+rzGFzTq2oEwewJ5ajW4SNT9eSlDVT97LObnECbBMiDr2N6bmabTVLov+QG7nlUn8gB5pCixdlsbrG9PDZKYSfHFIKNHCtObjBupQqzNpv6W1wkAjlGYgozP7bJPU3xJ3psMdh/mrsPYbkd/S07/M54BCcIkB+Jq3Uw5lv4p9YEbXPJaJTHnxcpmzC23S7AN1WXCjLwaqReLL4sVBV4USqttXJ85E1D50CwuXni0FfXXKHYRZKrsWtOtYTwLXtOA1HABIn6wKGgXgVI6lcO+zUy4hziEc1uHN4tEcpMe+jkuSJrcO6v0KoxcjNkhDoshM9jFiy++kJwq42DETTLx9SIio5+CeWague5IoLBZ7u9fI9CDZk8PkFfiPIBa80b62uXdts10hj/A1O5TCNtAdNd2JchFPhcn3fKYr7h4NidBZaVmaa1+NtExjPhBAtAQsL/aH/rgVJ9BqznJYyWow6OXjmaOWuEo0dQH90xUAoUj1fUyWlPAU7UvUzPGekeqwDzgF0kuTEqeV69k4nekbJnkPcY1ebxZjYNwK10M4Dy4oVmwpEJPAsg2DteBq9ti7Ky0t1FANH1fkEuNw/XfyIBQbNmx6xa+uGdejT6YDfuKvkdnIqR1VAIEhZSMRuDR+wbxAwpzj//9V5TKd2VUYvwBirwi3qjP/0T6mr3URYyICatubNSJSfsk2SGLz6CAvkGYP1tipZVosc9Qqn35mp2/Yo+gK8DfDaRjVVI2Ez87MCmUJkvhQKSVBFavUvRpn06FTjDOCE2gFk3hWkbGysYet8wydEiKTC0FjZI2Q4zXLa7d7ht8yZuRGVp0evty9HH+QYsYTG8eKVh5frofqdoTknO8lZIlUNSWiJ0im4dWEnixv2czMfwY/WVpbdNC3y4letGYvo1RHdT+OarsA7S9vq0fR3M+JUXIfemlTA/rkKZcgvkOWb9f24cwWA58w7OTHOKbK6iyCv9iMzd0GElSlVvchwooPhxC26KdcNJVhhUSO5bwqawwM0AlGrp/Um4B7YadMxoK7cEuyWafwayxn/NCOHekp33OcapRdJm2aYxZDXCDAPwmuIMQ3pkZ7O+sotMevnCxOt1MXKTJTad9Oi7L5CY5kuawWm1AMQd9xy0d4hw9dsAiuUNpCfNuo2FfAEOjpkG3hCH1IZByDaf6tT0Zab9eocbQvnAiAIAkHu21h8NwbTT8mhrF3cjMl8iZ5SEDI0HSAdTKwwzTguWiiqXH28kdmRuumcRhc4OqQ0bR8DD9FynFUe0gxpbmgiTv/h7r9dUNA6LyTGiTCTmiO7Ypsu5VoqyLDtzux/uBMoiTF4/UTrPkxiIVFXXec2l9J7iE81TTukr6ZwgDvJHCv189MAY9Rs7+dblqHPKCm7vHLvNCzECTzTZ9IGqUJTMu0kQvuogvcuGB8lzyfATpK7k51iA4nqoFD/g/6Q35hA5pcQelarVP6N7CIDlbydtUAsqhn7opbYuAOaxYKASOyiA02NqrGMay+BtTB3iiy5xxhMMAyVRunJsiA97pYvFBtvhko0MmwfVYHYD7ZxYJ/MMjWEHkQzDn2AS0i31Z6K3MqOrXEN+XrY8h+ZXxOZi0x02TlTNMzOEO2PZ8j9ilEjWzMhsfmQR7f0iYfXnjiDhmsvq/11/JSPeOS+7hcGYBj/HwV8QhOXz/MsQqKir66o/6n9fJS3vmTNwdeCR3ZS89I86KhkAlxhyaPA3G63tVXI+PnzSnwsKWGDe7uqt8VE0cQEH/Gcky2MXEW25ytINcxZdQlNn9l2ekXSqXvg2WIh5rhxTf7kON+mv2O/v2XXl3d5JTOjIOdiWD3Oz3n0svk0TbfkQu0wuKPqlo+0Dx1hRUtVzlchEuVgI8xJ/yAB7Ut3miYLVzQedjvGVs/ZnxvMYiyN3rnD6WEIKaNqCw/0y2tZvIakWdmZ15jsF+VNov3DdZKvHiyrFT43IYfmzq7ekk5fX3MZuDnKn5vh8PFzfZJBQwN1i/yovl8fwd3u8unowrmhctnWjLnF+qY+dXUIGCFzb0xm9ECyFAkDaWt3CM1BDehzHDXc/LVOAK4f4og4udBjwRF/n52X4Azz6EMLB5icSEHNvwqvirQBaTJncu9h9QkzrUKv9Q799RauWh4ph/kk4YNht/5Tx9U+ZoeUTI+TYgVbBixQ0/1BjJhiCpFbi+0+JoDNB+xdMnRRa8GhiWIc92D5uIfgoUdsqtpWqf3HfZJnd8sftztHPsXTZyN4ntS5Ufv/+wcFQ++ioeVNp734LEidpkvaVC31vvl7V7xD+JyfJEE/cJtu4XK9ocXzBRI6oGHNXjvpI9+WMJGB5580fAFy6msCbSnu9cs7p8kjedgG1zuMpkGyuqcH9ZkrbwhszNpmDh2fUaYqNr7z2VEeCdLfXA3Rdln+8vndOr+CsfT3Wv3UwCm+l8zT10ypDU119VJJ8VQNsRXe+PW9d3ox85buzJHYAp8fPRsYG6CXyv0wKTh4CIyI8Toy4jMuWwWnV7HfUjUT87w032s8g/Zmb5t8OcUf4W1mUUeYz7fLAQA6Sjaybbjlh/KLR6+bxOuRBpXvkyvyU4Y4BzN0v/muuW2TY/7u/Juzu6PS+hhfQM3ilZIetxyM3wou4/BW9Cy5hPbIadUVbTyh+Wdv8eQIbOrCViCTSk89i8FNAqMR6zucXPVO4iTrNPgQzS27f2D32SlluDUhYg44osBirpbRJsyb8kuvFq/tgX0iJymzoGOYdiItqk4v1Nn8Gb84+wPKsqFT/xr4AOgz7JMk8v8tyerfPf4pGb5vksOIDQgKnbTmTnLabccTleKgdCyVRPBynOi4m1SqTb9Zj65x4YeL6GtGU4H+8WVHMlfXuOLl7jetBi5uMQZzYdafPOYkTuGxUB4WpNE5MMSbDcYwvuYHxpqCBCFxnLZA9sm/TXoMlqBXA2uMxfovI5cPhK8u+SFrKn23E/MX8DY8e9ZI8AgMu9BAgSw7uOIK0Z85AzCpVhLDPoXvEecPtygkOCmX4oE5GM1T4VTZNGG1xpRZGts0bWUS/nzRgPMc/5CcTwZaC5rk3jJ5ebFNjFwYaj8hSJ+AVcb4IDEWiGjG7VZOKcbAR5Qno1xq5J4mJ+xgD', 'page_age': None, 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}, {'encrypted_content': 'EpMgCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDDdX/ZcOgjNRfwk9thoM7hIvPkvovmnjrwlXIjDjrSoG5SF4RRFc0mFrgKpOTDFP2qw5D/eZT1q/nLrkMcmgl4GWqKGZMLrcSVkZikAqlh+EqFTteFvQUs/Lg7s8BZF9idZLkg9jZLdb4yRIwcemqqMxnfitejYQuVcPx3A+98d3m4Ut4nwD/+/6p8BDTBkbQtaOC6vrwFzkj62beWBVp6Me+jt/r+UO4y2L7/8HqGH4NXs/IWtgDIvRcjRk01sWjPvvrAUHiDNW38nymtZjKpb/1GLwDAyVrG4O17K78Wzeo8HSjjJ8C/g7aD4X14pUMttcAeou3rMDNjrMf+VjLBiiLWEIq8fVYhR04/Cx6esdhqzxCouxv8iFcZMDm4zvxdUJY1qgX3CWa9zYK5Fk3fjyAF05QU7tgf8ozGH7nII2N/IyrN6AEqu2l9V0QUZSRXOebcoiAdJSYqcuIg06hdaf9cgYgYOXdBfdEFWH2hflBBlBJLiNuA8SCYvwLd58Sn2G+0WcwuZZIOipl7ZGCN2tiLZfuhrS/bSpkBPfk1aSUUHOWuyerBT5awXTl6XBfjlzC4BpeiizU8NKYX3tETt2l83aUL8e/KWmhMvbcp2ydbM9mMqsRPm00rurwkfb/CY/wTjLHcX5Ag14OxtLqaVQO+PDcbcOh73XKVzV2ouPM5eJJkoHsaI5NXiApSuv0Zj/0PWJ/jaga+IsVo1juc8a47MyLCZoW4nyjlNpkbyLYHfS7bdtHedSECMEUT7Q0TQsrQII/9dHyr0v/SF2MgcCkqEWSkTvjUZaQw7DhajwjYn2xLXGh7zkh0vQvOA3k3c2rvX8if/+BckYdo8XzAFG6wu6jiZI5I9K7ckjfsJr5YImapBKWz5orBt017q6sbjgKj7q1eOXDEUHzdbuUux4lHiBj/sNKyI+zt150O2UwFmYkFWTvlENA6mXfwGu5U5nTb7t0Nr9b2XMou0Im58WEt3XHUCFa+yVBG6lpVHSEA/GBBvUkPQY/lGXIEq3fyd5JxCyJ6pu+n6oTsdMSQRJmb/gx56El+OmVQPxukTc6tnKHMmHZTZEKkdjtlmEApUVhnFN+1WLS1I41LCDg/r4aJ+WD2uu/LPKoUecFREFNP30ZpzTScvD8t1g6z+9yOEBRBVI4ki967vPDnz/82cazpSvvf5RohqS9ZnV9Sf/PwgkzMQDBpTgj6LBVob4gk7+X5YEKe29w0Gde2XZAra6lVsrX5xRL2ychO0F4oB2w3jcjVGFE8TsRF0gVxDLkGLOI6TVJvLjiTZaPuQE6JR51es9gWu5H3+rw+Knzw/nPl1Pc3WzJza6f3M2JprPr1sGN6BsWGgKKaqNpP4haqyJ6pm7QYwvivLO2UlktQ7U6j/YwjRywzTTlKqd443xqlnl13Ygr8pypSok3b/uZ3/w+7K32ov3a6Q3LB5PExjj//RvQJk0yw+DbpkOoCoYUSE/+tBmfJejT7RUCw9m/h4QO77jwQkd6FGTpIeuvNOaTaJ+lI8zhpr4voxHOtLr1EeM4DL3TqrAhhc0H44LH/Z6VJ1wrkMS4RnULJa5qP42QjDFunyC0JpjlXB7fvn4OzY3YOSEJAIa56iFO+WR/9cvn8frF68BP5JdWOIbJq+d/cQ7RThiJ4gl1Lm1pDyBArzgCr6m8qWlOz/QwN5nYmIGQPP1TYeR2xXOKz2X/h7oSpp6S4az2sVH4R6/AfSTTJobJ/fFIFAsy7J2+iXV7ns3ZyRhQZfafhjzq2cCnTkewBIFr6OCmWBrJeeutULNEBRczfoegmj37JEDAgPkckWT9BDS6WGO4+ctl8ZdR2Ui+KU5d3Qs5iDxkEEqadl7xvACnJM/mk7WAGVPTMe7bKLHHno9Nd37hZgIj0RmXj/2CVBjAImy7t1ngP9lnMBVUXrE8fMCHGZokIyWXUGjg0HoyWQmE4Wz+FVQ1tMpRfah3z4CwmBVJnIzZKED8ZgasWgA/e9AXyV0uwIvUixMr9GwV5yiiOLGlDGNtfmbnqe+aLJ8ikFHTK4UlKJbeyDYOHpHSNOhJACF1YrtYVGhcRzfZy+ipwi3lZzoQ6N8Y2rnFgAjUuqeIKHWagf/qch+wknV0edOTBar18S64R3bzORTBE4YPQFfnDfcBSuQiSxhuEK4T/xWV3MPUkz2jEFa+Ayl9eXKepA19Nu3JTi49sPIqGNbL8tbbL9qEa/EHi/ZOXGQwFOkGUiFVFpfEop/iQ1dYTw0C1zoj0giPuLWJSKztroYNHu/n+ovlb9qo6+xqsuS/r/fX0rpDGSfacm95ABGXwb7B9A0WZsO3vP93lHJrR6pbxoEIkpXZFPO68pD5c9rsXlHkXKkPJU+pXvkMbejZOJsKMA6B+u2NsbkEhudd6felqimA4GCloJJ8iTLrB2rEYBWJLdaR+iaMuSQwBDMDFEGx6oRl9vikrE92sUnAsxlwLef2ApOJAmklRmC1LfH0DvpPHkLep/WQ+1aazsDnJEaDawrulsQ5M5M+Glxl6pod+Xr4dXFcnAbNmNJaxqC1SDVi7dc8R1Dx59dn1UHsVwGkKIRL6xR2o6Fw7PBsZILJnQCHnQ2m/+bh+ptOEIa42hic5VdnEoELuj1j/Za9IAVbdpleON8qgl6zjEfKqGbXItb7G4vI+33yBrYHIgQO4DkS044EoKeu7Yb9w4JhtAv60FllkZtXEnZey4VJ1TK2VIDAZ8RWdGEvYPAncDFw3xjJxsNR7ny2ZOdhjhlU47uOGZfyD/aGcGvNyUUDi7Jfst3uVh9Aa+dSPs5QWUiLfJLF+0KI5s8YCtVAed16fCxNqJeoIe2O03M3z8RiPYl9sKx4wqe6EHv+/9r3MFqJTGsAxgMsOi9F9NiJ1qycjbP2voXhgcDX3EjfuhxOVthRgstU7oLVUoa+vWuFZdMTUiseYAdTVleqBKYFgBy8qzq55/zqEOTCC5BLTCGh742dZZle/OfIP/M2E+Xgz9KLpAyINS6RiUiDFtE8pFNExzORZS4FyVifW5tl4L0WKqIxYRA8Mubj9NoKylW6DTaPxHY96rDRJY1PY77zHuAwi5QudeAoZ56TKd2KO+i7va0OVNUa65l/lfHIhcDLATuA6gGUWiXHFemoATs7UUHxnauRPmpsZeiyweSwWU9jZ0YekpM3snUdSukbRvYQLZc6LaaBOtJen7+XcJVXTiQmP7u23DBJjF609sIR7ZYqKhfFP759gmoV/IuVCmAbXir4/juDBq6tIkrHvy2mCrsapjjLWszo62ujs6nvtqwpL8npKrFTZONabR08TOvddEOf152ecFvPGYVCYgTp+B7HGxYhv+3sE5WzKDUMj+cMraJHW5MmVnbywuBCOtr92PGUNeTMO7OffrApHVSo3oEW+vjTN7O7uvYroeWN20gYKYYSE0GzqoEkNg0pdxVVs6db93PrEkNn7mz3yRtA6J1/RM7U+QsjXXadDIjstJXOtlRE7naZCI2pBhdhoOO5rKcT/4Me7nTIXUp+KotKJASfKj8w4YiYKnVabzB5aDfNnbT/uHPTH5+ZKH3zL7iGPJretOBPySlPFSqlAdQAOEr8fxreKo79nwfxy4fW3UVWbb0AvqS32hb7ZOUfM2RE8ClYaw35BOEq8FyZ6BjYfoZDQFdkGr7czPKzrZlKODbVvbGXLXgXxZL8WTARMK1kH7nXfNZfRj0XRfCNNnjJ94wm60flv+sdcsr3HeKpW+yiAAljk/xQVOZhH0WLpdpfgm19E78XJQXjwn57IdYex2h6qo7R2fU5iX7+3AY7qXPKvGPZ83dlZ0u1RrMHK2PdDzhCXwtKjPlfnN9wJTRua5edGojV5Pc4ldicxZtFD0PuHQ9xn1Y0R5H1fRs6StMFsuc1pJnR595UjqxpTPYhKfBumwU96Mzhl1zPEhByFw1KLhr0uI4h5uyVntiUhoWX4HCVRrBmSB8HKzuV11ypZtkanOC8W4yBGp4te3lPCl8AgOxb3edl4fRDhw9I/s2GU9wseQKcAE812pT0sAc1UunKudtvgVpD7RM5QPkjn/VHy1IGHq4WnRN2BEzonexk1wXo45Wrdf2qoRb7mNbGRjHopv/Hjfg0SgS+wQX2VeyMr219yQQ8MEVgBGl4xND+H2sy6Lqrs/i//ryCFZFoS0Ps3ejoTZcYZ8lIdCwmovD0bAzoDOIaat6ePqoL4FPKpBwZzqS60R45e02CjARW2k0x0SO9eX02kiwpjUxIW4Lv+SEpedg411GaaOA76cYdDbKQRVBfARIt9Lx1LEQhDGQUVhCjSNL5TUqjKPp8IPnYIu5cj+yxKjkBDs5yFzFPNABcxk47MB2R508bhtAjZoUeOzQDYR0rTmqTH4XH3agksA4e4DmRjL4uqgT/Z6H0IOXcdhQZH4qKJ/XQRo7LPuM3cllKJTLb8zUjuvFXFdEUgQUbsAWsehu13x6FEHwFeF5PUfcJcgbjBEf6I2sY7NxjaWB6szrsV1CGrpJ1OdCcBZVHllKVIjTJDRYzj3ptrHgkF7uTRMeOPvTDrwJYroSQqSzUZMykyXAjSzfWh+aLOHnxEzyquBVEx9t01BUQBZc/bRC2EH/NzDhoErfrAL8eIUmC2FJfJgaSiLwqYIOrVZZjkb8VKOWQbmxWCYHnoP8/yeuVpNyqCgYPRn4cFbc/33YPNIWh9xnVLVpu/YnR+bt+b1HazZSXh6LiPubUzB43MEoy8+zXLKsyjqHQonVBtTvuWThfiDPVzzucTReqozMrYtfCNKriaehYnRJaC5C1UdM0AwxuKe+wuIroOq3QNiBCFkHjhhpZ7X6O8TbLp86xJ1P21uTP46YMlSs17gQlgASkEnP5ani4IC6ZGtwXZ6PPXMVy7ZH5LW2W1CCRpCAhRShPmk7HZ32pboGtgGZ+7IUBOZvuN4c8WDzqfvMUH0sKVMGohyZJqKwJgN48yuDk1PQLBHRbF9CaOXef2p5LsA/jJ2kkU4PftZFU1vOhB1BslVjxymv+j9QiarnX0GqdAJoJ2L60eThJLmiw3psmTYhnqPF0t+8lGlxNcdK488Dkx+q3ztqDW+9mrU96eXp8hmPfQeUTw2M52PRDUadcxF/wFUe49bqxoDvpQkTv4S3nerZxwcvcKacjMSKe0Nc8tPVuiG8iqNJfKqtZkqzQEXUPM2H+/LGo3W5XNvn5JWttCnAZA2ZrqnEHAEBE6Rc76nTtIDv+bk3HAK7hWipD2RvGzvrADuplPjJdxPAdYVWu0Qgfq8r5hdwxv21gOgJrW0sovCDOfGFLXkjzoh/gGS6jmw82r02b7bOjZZqGiFHE156AnbERZwMV8LWv/t7L18EpAPq7uLrkf7+bRnyR1u0rBT6jE9q0OeUtejaU0pMkvJGr/7Rg0gEKWoV+QbbYj2xyamd1AoCC06OPKGCVh4YAw==', 'page_age': 'January 20, 2025', 'title': "Five Inspiring Quotes by the World's most Renowned Female Marine Scientists.", 'type': 'web_search_result', 'url': 'https://www.orcaireland.org/celebrating-international-day-of-woman-in-science-past-to-present'}, {'encrypted_content': 'ErQBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDNX0MpNMv+XZHQPCxxoMQ7YMkRA2t4px1jZGIjB1q0h3LQW5ZKPK2TnwVckFdQyfUyMMolKfK7eW2QDzYImq7B14x5zwASNOJ6DIy+IqOAAD89/ICvFgaOENNpILwshiolYnBN21KX5A+HRpjCaf29Ue6dSrQg4vy8ZzAwqrwiEVK4gVCBz2GAM=', 'page_age': None, 'title': 'Funny otter quotes and sayings from marine biologists', 'type': 'web_search_result', 'url': 'https://www.facebook.com/groups/349404865879919/posts/1063710424449356/'}, {'encrypted_content': 'ErAFCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDPhg2Wt4FS2yg/wYrBoMX3sUQiDXfwCWz0hnIjDwmcYX+B/1SoCOp53z9r1/zE3f343u5ACre446N3QRv1BsuCjfP0s8sxxBgFWSSrwqswQKEz436+6Cy+hbU6FZplSOq0FqcrTc9F8usA6ytBuYrTuoeS8nIDpqnQgXkn06l+qzSleiEctf3ve7y61XG23Yr0urH09PDa0XqUZfeFxnGDZls/o3hJ/geXfnWWrMBtrHl58ri036zdAKw9+3TFLlDgE/zHTZnXJOp4j0Mq2ZnBMoQslYUEpVaiA4NQY+/4BqUsP8j9q8TScz7q0abqN6p6R6to5jONBkxQeE2Jk6432lXNISqPSuiCRXtzdso+XYw8M+ViKzGIkz9ZKbt/otSr6bR1XVD/YfC00CvB35Fx6+oLz0u0MgFrRErV7ZTwkfnjNE04ad0UXqK1wm/C5lxGUfUrlBy8j2DhZKmzHIl4f2OAbuco6KEb3dKZWvSCOcX18sLKmz+W7Z1x0JrDR+XLxQ7WrwWp4ksUvVjOmv0zvPe8Qh0a6F0gBwuN+Jxh6u4RuuYv3iwbhgaxuK7qJ5rQVZm0XVjMaVxL4nJG2iEHOo81sDM5P8m9umASuCHFdUpYOauo/mwfgetgFNdZnJcerr46L0SjA5mWf/iMPVi5LF2nFU+iOR3LdsVKiY2epME7Pg13NiQUBr16AA1tWLy14BNXhHX3jSd4hBg+QvjFkysQ4QX5settAbBSzkvSRQA1nBFeLVaf3Fbi8yPVQCbFkB89mVks8Ve30w+gBAP4hZgNU0Al62RyY5TgTufnesLucRvORnZ2C+DBpeXg5fOFTNh1v6+i62c74Mg/XzVTpWvBgD', 'page_age': None, 'title': 'Marine Biology Quotes (7 quotes)', 'type': 'web_search_result', 'url': 'https://www.goodreads.com/quotes/tag/marine-biology'}, {'encrypted_content': 'ErcJCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDHSzldOGr//P/eVIixoMox9DuJC/+sTPfSP6IjB7G0xF+2tdeYNaYYbhxUOnQsssCkmMTstMeP0/ZqmVOLiVNzXorxp1141M6aGU+bsqugitdsezWfinVP6/jI9/STBxkS2pfgHaFeOODgX5j2Ak3hJYq3eHVFbqd7BOWWMIyT46SQxOw5C+FdaCFrpLQPgVUrjwpGeoug/hZ0Y3F8/e4LbEIzejLhDhDt5Prjek5raXItctig01iPfGB55c4/zkvaltBcezpx4V4oyLFQ5KojOJCQ18hI+VhoWH0U2ycUs5Ql1x2SkurK92E6HTn16EFT91QErMNDWPf0Gyzii424l5h/b9lfQJCS4tfBGbuMeSqGhFoQisvnkFVMIG/YWKTPyhyD6OZbCpwkMYuRcdH+ChVj5Qm07HXJgGtqa9KW0FAmmWWAM19gM+En3q5uv/b6gCvdUmk9jV6wmhx8lmVOCndu53k0bSl+nK07WJSr6tOibk5eU4zVzlIp5Lb7gRpOCLfrISGNxVkX+3EnFdi60jBb6bsFPSGEwcoEn1Ig3oa/4KsZaBqtXcpbv3DJ3E1YfDBpQGzcGexjlocRZsacUXlqMocOGuvkwiHOphIurAuhELyJVWan0A4TWbbu1qyvhlo7sxThHU1abkJb2pSg+c0lArqRhL5r5Z51WjQXlf6l13yDvPdLr1niM/xJDGwk09rY68W5BajrSkR2c8ExBHTaP8C3XwmkPTDph6XpEHfzdYRJ7T5pEbpX0EMUmZLzL6s+Farvo88FKAJHMwWOQmyurDQdrr119s5MqNzKqwMCGdqC6d1OuG2Wg3NwubP70vIaQUBkQWhfJwm4NVx2A9FpdLw4SQoGOr7Ytqk3qOVzf7XL4lCTmBhjopqYpkl0limoCq2WqPqmCqkTpbEt23gJKt/Ga2malz12k22FtjSzJUewqyrd9n59YHqlulSgG/G19wfNTLY1f5OwfOZqKz56/eOlzrFvAdCLF+vnAmiLwjpv7famVB9JWSpsmFj95AEqi8hTm0fwK4ovXdWSM7EPZngEoaFGyq9yLZ3SItObY1fXjeQNmArePXYd4yLkdGDovHN0F2bJ5759trmDYdxBiyfTZYjGJsxXfdj6h2XWgTS4J69QgZCFYRgZ/K2T2qrJAjZISlrm8QjoHbP4tLleRdm2h5ZExIRmC1iJpM+SuG64RODKtzzsdxggoEwRbPn7T39m91QTDDEZZBwEbLvwSzG/Wm2OzoQceQFTHzwOvscWgp5hWHSzO6/9WV52s+w/JZeNLSAclCinI9DNtsgqSmWaKa6C/DmozoMOreESSxLoTKSvoeXg0vwn37qyMp91IThPXFESXrCgn82TeJNyaU0fq6bycT2Ov6HJbZXxGJzSLS5sP6QmKavO3hi/P9wGWqi/3JGxLUPTNkGzW7lJhSDun82CQYkw6WrokiNw5hjy8r1LBZbb0EHnZBgGWwyIvjuZKmIyomVrItOYYBiYKqccOL5Hm5gDOJ/JVV19Vt1ikwO+5lM+uO1Vj2Hcx9UlHY0MAIJRgD', 'page_age': None, 'title': 'Ocean Conservation Quotes - Famous Sea Quotes on Sea and Sky', 'type': 'web_search_result', 'url': 'http://www.seasky.org/quotes/sea-quotes-ocean-conservation.html'}], 'tool_use_id': 'srvtoolu_01X36BnBixLaQdE3H8E8hr25', 'type': 'web_search_tool_result'}, {'citations': None, 'text': "Based on my search results, I can provide you with information about otters and some quotes from scientists who have studied them. Here's what I found:\n\n## About Otters\n\n", 'type': 'text'}, {'citations': [{'cited_text': 'Otters, which are fluffy and adorable, are born for the aquatic lifestyle with their slender bodies, thick fur, and webbed feet. ', 'encrypted_index': 'Eo8BCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDAfXVRi0naNItWQIWxoMCxc4a/jSxMEltgboIjDZOzqMO0pCWijD+lsg4ELcciHEw4HXq1lCe2ZA1uKaCj4leAmhgdUoC+17IB2qi3gqE+2LZkOQhgwUh3tjm4GXJj/XNJEYBA==', 'title': '20 Otter Quotes about the Energetic Slender Mammals (2025)', 'type': 'web_search_result_location', 'url': 'https://www.trvst.world/biodiversity/otter-quotes/'}, {'cited_text': 'They are born for the aquatic lifestyle with their slender bodies, thick fur, and webbed feet. ', 'encrypted_index': 'EpABCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDAalT1xr5hKJ4k8wBxoMovHG+jeL0n9TUuL5IjD0NJu0PTy1lHRTrfn8jxwAa8NNuytnjf/LWHLCM7y97DCzynMIl5e8fa/MMUBWzngqFIh0JerI/qOiM19M7UVPAjaLdi6lGAQ=', 'title': '20 Otter Quotes about the Energetic Slender Mammals (2025)', 'type': 'web_search_result_location', 'url': 'https://www.trvst.world/biodiversity/otter-quotes/'}], 'text': 'Otters are fluffy and adorable mammals born for the aquatic lifestyle with their slender bodies, thick fur, and webbed feet.', 'type': 'text'}, {'citations': None, 'text': ' ', 'type': 'text'}, {'citations': [{'cited_text': 'John Steinbeck had an interest in marine science before he met Ricketts. ', 'encrypted_index': 'EpABCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDPVdgoKeglLQsdxBjBoMrowGXN5nAJ4vHV+wIjDuc0SKXR7UYFcN3/OgLcRMN1JchJghZu+bVRchOi8CAvccM/iJuREPaS5i8ZDKvc8qFOZnHFdGh7XsXAGnZdNd7p7q9YnWGAQ=', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': 'They inhabit all continents except Australia and Antarctica, living in rivers, lakes, and coastal areas.', 'type': 'text'}, {'citations': None, 'text': ' ', 'type': 'text'}, {'citations': [{'cited_text': 'Quoted in Larry R. Squire (ed.), The History of Neuroscience in Autobiography (1996), Vol. ', 'encrypted_index': 'EpIBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDFfxyfwxwpOQOQLcSRoMCPEuLJHauNGWZ2FfIjBanRuaR4ONGcE+DZdYrpohBHNiTlES7vYw/zY2HGM7mARc0yqX6EfgzW1B6g1FymcqFvC8St8nI+vyQ5bDTVuCeFv8sLYt6/UYBA==', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': 'The 13 species of otters live in various habitats. For example, sea otters feed on marine invertebrates and use tools to open clams as they inhabit ocean coasts.', 'type': 'text'}, {'citations': None, 'text': '\n\n', 'type': 'text'}, {'citations': [{'cited_text': 'Scientists define animal play as pure enjoyment and not connected with survival. For example, otters play by sliding, somersaulting, belly-flopping, a...', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDDC4VMylw3UvnxhRthoMd8Kh4Y+bM1MI4sivIjCoGMNU5crpIXF59l94jPUQoqOPGo6ZNRjT5HF5tIAa7XR1d8uhSf4/pSEw8LMCvD4qFTGKhUqqjThFzXIxnMSeIWsy437MghgE', 'title': '20 Otter Quotes about the Energetic Slender Mammals (2025)', 'type': 'web_search_result_location', 'url': 'https://www.trvst.world/biodiversity/otter-quotes/'}], 'text': 'Scientists define animal play as pure enjoyment and not connected with survival. For example, otters play by sliding, somersaulting, belly-flopping, and wrestling. They love playing so much that an early study even observed a group of otters slide 16 times in less than a minute.', 'type': 'text'}, {'citations': None, 'text': '\n\n## Quotes from Scientists About Otters\n\nHere are some notable quotes from scientists and researchers who have studied otters:\n\n**From Marine Biologists and Conservationists:**\n\n', 'type': 'text'}, {'citations': [{'cited_text': '', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDA2Zh0yXO/SAfn/alhoMYQtOx3LON0s2UH/fIjAZK1Y2iqkyooEu0XjxVf1YAEd9VnHywv0QX0ily6AziuAUkoEUtD33trpDKR53F64qFXSTgWpZ2pF3FqB0dZNEQUQ/OKHsqRgE', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': '"The sea otter may be one of the most widely studied and intensively managed marine mammals."', 'type': 'text'}, {'citations': None, 'text': ' - Sea Otter Conservation\n\n', 'type': 'text'}, {'citations': [{'cited_text': '', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDOwiVZM3Wp37ko4kNRoM7lRit3qceSWoFtfSIjDcLwkRVXzztLbm3yHBhcpKMIu5IA3PRaAq/k+BJmPIB4+kalM8LO+6P6u2hR/tuZsqFY+fF2Zx8jCP5KUTfUKJB0WEEcUtfRgE', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': '"Healthy otters tell us we have a healthy countryside."', 'type': 'text'}, {'citations': None, 'text': ' - J. C. Tregarthen\n\n', 'type': 'text'}, {'citations': [{'cited_text': '', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDEebcE/nLF5So0n6iRoMVxUaUpV28Z7Yv2O7IjA29RSXnvSaw/mOOh3CQrL9FJFODHTNasNBMPCRmxcLA2NpSf09TmtJ+Y6D9w7lz/MqFYXJNcmkS0prSOp3ecDQqrDsD08+CxgE', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': '"Even an otter picks up a rock once in a while to open a clam."', 'type': 'text'}, {'citations': None, 'text': ' - Chuck Lorre\n\n', 'type': 'text'}, {'citations': [{'cited_text': '', 'encrypted_index': 'EpQBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDMMbLx5afZU02CpeGBoMnYpYzXLTgKWan5X3IjBy/PsS2XRUf4KDiYuJLgSbtbnNy2yBfM0iSxeR6lRDrV9s0NvPeYOobZwihOaYtxQqGEpWrbLvREZ121e9JicktyOIu+hzOXB+lBgE', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': '"These little fur balls are very good swimmers and can stay underwater for three to four minutes at a time. Otters are very much like you boys and girls, because they spend most of their time playing."', 'type': 'text'}, {'citations': None, 'text': ' - Michael R. Greyson\n\n', 'type': 'text'}, {'citations': [{'cited_text': '', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDLxlLqLc2BvKPijQYBoMeVkEigZEngna4O1PIjCFHcflBGO+J0qZiZDyZ7+dfcFJ+L2Rs6B6sYsRtDWbMgXo9Q7COV7TWRL9CUF0YMgqFS5oT8NwwLX+HrnxcE00LkbZk96wDBgE', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': '"Few people actually see the Eurasian otter in the wild, yet it has become one of the most popular species."', 'type': 'text'}, {'citations': None, 'text': ' - Daniel Allen\n\n**From Historical Scientific Observations:**\n\n', 'type': 'text'}, {'citations': [{'cited_text': 'Yet otters have not been hunters in water long enough for the habit to become an instinct. ', 'encrypted_index': 'EpABCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDNN6qAp8NdOs1honLhoMVRV/nup/MX+cBeRpIjDBEsoL2lsBggQp2SLDi5+bdubBZIBc84sp2KB0Y+0915x8V0p/paDewpawC/aA2UIqFGQVuBPhFPJ2+8q5z/OvWJ/hGyHjGAQ=', 'title': 'TOP 25 OTTERS QUOTES | A-Z Quotes', 'type': 'web_search_result_location', 'url': 'https://www.azquotes.com/quotes/topics/otters.html'}, {'cited_text': '', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDDuNrBQdB/+PL7qOdxoMBKo26Vhh8eZcjxYmIjDIxHq1U8Zd83EJMcY4G+Px4hQM+QmHcYJy/DrnjilMRcaDOnahb9anbdkevlRW3TUqFb9ayyV57wxHpGr51Fb/EGXXpZ5iTxgE', 'title': 'Marine Biology Quotes - 24 quotes on Marine Biology Science Quotes - Dictionary of Science Quotations and Scientist Quotes', 'type': 'web_search_result_location', 'url': 'https://todayinsci.com/QuotationsCategories/M_Cat/MarineBiology-Quotations.htm'}], 'text': '"Yet otters have not been hunters in water long enough for the habit to become an instinct."', 'type': 'text'}, {'citations': None, 'text': ' - Henry Williamson\n\n**From Contemporary Research:**\n\n', 'type': 'text'}, {'citations': [{'cited_text': 'In 2018 we asked Jim to talk about his picks for the most important milestones in sea otter research over the span of his career. ', 'encrypted_index': 'EpABCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDARHlPFprBbX/K46ixoMAm+o8o0+2g8o+kPeIjCu98FVKoLhZNb4onnLT8belgp3euyKX9MYoC3JYfskQeNWyRMJA+L0bCdL9Y5jbPkqFC3LH/mK8m4EIEKifB54TmBClzd+GAQ=', 'title': 'Sea Otter Savvy | Sea Otter Viewing Guidelines & Disturbance Education', 'type': 'web_search_result_location', 'url': 'https://www.seaottersavvy.org/'}], 'text': '"I wish there was a verb to otter, ottering around in pure play, to honour Otter ludens, which plays in my mind long after I\'ve seen one."', 'type': 'text'}, {'citations': None, 'text': ' - From a contemporary scientist writing about otter behavior\n\nThe research shows that ', 'type': 'text'}, {'citations': [{'cited_text': '- Chuck Lorre · "These little fur balls are very good swimmers and can stay underwater for three to four minutes at a time. ', 'encrypted_index': 'EpEBCioIBRgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDPihoaeeQGMYhYf/KRoMJyRqidaM69gVDk+bIjBvPhFvLgdoS6h+k2MjneQK9hWLvxR+gOHT6Sx5qS+nU/2i8wMPWIVTe7Sk4euXgCQqFVem5BV83+dDeTAPLr496bMCfQ5VxhgE', 'title': '20 Otter Quotes about the Energetic Slender Mammals (2025)', 'type': 'web_search_result_location', 'url': 'https://www.trvst.world/biodiversity/otter-quotes/'}], 'text': 'working in federal science made it possible to maintain a long-term research program that provided a deep understanding of just how important sea otters are to marine ecosystems.', 'type': 'text'}, {'citations': None, 'text': " Scientists like Jim Estes have been instrumental in understanding otters' role as keystone species in marine environments.", 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 25534, 'cache_read_input_tokens': 17660, 'input_tokens': 20, 'output_tokens': 920, 'server_tool_use': {'web_search_requests': 2}, 'service_tier': 'standard'}`

</details>

## Third party providers

NB: The 3rd party model list is currently out of date--PRs to fix that would be welcome!

### Amazon Bedrock

These are Amazon's current Claude models:

In [ ]:
models_aws

['anthropic.claude-sonnet-4-20250514-v1:0',
 'claude-3-5-haiku-20241022',
 'claude-3-7-sonnet-20250219',
 'anthropic.claude-3-opus-20240229-v1:0',
 'anthropic.claude-3-5-sonnet-20241022-v2:0']

Provided `boto3` is installed, we otherwise don't need any extra code to support Amazon Bedrock -- we just have to set up the approach client:

In [ ]:
ab = AnthropicBedrock(
    aws_access_key=os.environ['AWS_ACCESS_KEY'],
    aws_secret_key=os.environ['AWS_SECRET_KEY'],
)
client = Client(models_aws[0], ab)

In [ ]:
chat = Chat(cli=client)

In [ ]:
chat("I'm Jeremy")

### Google Vertex

In [ ]:
models_goog

['anthropic.claude-3-sonnet-20240229-v1:0',
 'anthropic.claude-3-haiku-20240307-v1:0',
 'claude-3-opus@20240229',
 'claude-3-5-sonnet-v2@20241022',
 'claude-3-sonnet@20240229',
 'claude-3-haiku@20240307']

In [ ]:
from anthropic import AnthropicVertex
import google.auth

In [ ]:
project_id = google.auth.default()[1]
region = "us-east5"
gv = AnthropicVertex(project_id=project_id, region=region)
client = Client(models_goog[-1], gv)

In [ ]:
chat = Chat(cli=client)

In [ ]:
chat("I'm Jeremy")

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()